In [ ]:
import os
import pandas as pd
import numpy as np
import boto3
import getpass
import datetime as dt
from datetime import timedelta, datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import re
!pip install xlsxwriter
import xlsxwriter
!pip install openpyxl

from openpyxl import load_workbook
from scipy.stats import norm
import random
#from scipy.stats import kstest, probplot

#from statsmodels.tsa.statespace.sarimax import SARIMAX
#import statsmodels.api as sm
# Suppress warnings from statsmodels
import warnings
warnings.filterwarnings("ignore")

#!pip install pymc3
#!pip install git+https://github.com/pymc-devs/pymc3
#!git clone https://github.com/pymc-devs/pymc3
#!cd pymc3
#!pip install -r requirements.txt
#!python setup.py install
#!python setup.py develop
import pymc3 as pm
!python -m pip install --upgrade pip
!pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose
import pymc3 as pm
import arviz as az
from scipy.stats import kstest
#print(f"Running on PyMC v{pm.__version__}")

In [ ]:
# # find new clients who have their first order received on a given month

# # Choose the most recent available source data
# target_date = first_csv_file_name.split('opg-analytical_cases_P')[1].lstrip().split('_S')[0] #'2023-12-31' # 

# from datetime import datetime as dt

# # The final date the data is extracted
# final_date = str(target_date)
# final_date = pd.to_datetime(final_date, format ='%Y-%m-%d', errors='coerce') #.dt.date
# final_date = final_date.strftime('%Y-%m-%d')
# print(f"final_date_extracted: {final_date}")

# # The start date of the analysis
# start_date = pd.to_datetime('2017-01-01', format ='%Y-%m-%d', errors='coerce').strftime('%Y-%m-%d')
# print(f"start_date: {start_date}")

# # The end date of the analysis
# end_date = pd.to_datetime('2017-11-14', format ='%Y-%m-%d', errors='coerce').strftime('%Y-%m-%d')
# print(f"end_date: {end_date}")

# # The start year of the analysis
# start_year = pd.to_datetime(start_date, errors='coerce').year
# print(f"start_year: {start_year}")

# # The end year of the analysis
# end_year = pd.to_datetime(end_date, errors='coerce').year
# print(f"end_year: {end_year}")

# # The final year of the analysis
# final_year = pd.to_datetime(final_date, errors='coerce').year
# print(f"final_year: {final_year}")

In [ ]:
## ------------------Step 1: DataHandler Class-------------------- ##

class DataHandler:
    def __init__(self, bucket_name, folder_path):
        self.bucket_name = bucket_name
        self.folder_path = folder_path
        self.s3_client = boto3.client('s3')
        self.selected_columns = ["receiptdate", "cases_glueexporteddate", "uid", "type", "casesubtype", "status", "donor_dob", "donor_postcode", "donor_gender"]
        self.combined_df = None
        self.snapshot_start = None #'2007-01-01'#'2006-12-31'
        self.snapshot_end = None #'2023-12-31' #

    def list_files(self):
        response = self.s3_client.list_objects_v2(Bucket=self.bucket_name, Prefix=self.folder_path)
        file_keys = [obj['Key'] for obj in response.get('Contents', [])]
        non_none_file_keys = [key for key in file_keys if key is not None]
        file_names = [os.path.basename(key) for key in non_none_file_keys]
        csv_extension = '.csv'
        filtered_file_names = [fn for fn in file_names if fn.lower().endswith(csv_extension)]
        return filtered_file_names

    def read_csv_files(self, file_names):
        dfs = {}
        for file_name in file_names:
            s3_path = f's3://{self.bucket_name}/{file_name}'
            try:
                csv_obj = self.s3_client.get_object(Bucket=self.bucket_name, Key=f'{self.folder_path}/{file_name}')
                csv_string = csv_obj['Body'].read().decode('utf-8')
                df = pd.read_csv(StringIO(csv_string))
                df_selected = df[self.selected_columns]
                dfs[file_name] = df_selected
            except Exception as e:
                print(f"Error reading {file_name}: {e}")
        return dfs

    def concatenate_dataframes(self, dataframes):
        self.combined_df = pd.concat(dataframes.values(), ignore_index=True)

    def set_dates(self, start_date=None, end_date=None):
        self.snapshot_start = start_date #if start_date else '2007-01-01' 
        if end_date:
            self.snapshot_end = end_date
        else:
            first_csv_file_name = self.list_files()[1]
            self.snapshot_end = first_csv_file_name.split('opg-analytical_cases_P')[1].lstrip().split('_S')[0] #'2023-12-31' # 

## -------------------

# Enter the corresponding S3 bucket name
bucketName = "alpha-opg-analytical"

# Select the corresponding folder includes new LPA data in S3 bucket:
folderPath = "sirius_data_cuts_3"

# Instantiate the DataHandler
data_handler = DataHandler(bucket_name=bucketName, folder_path=folderPath)
file_names = data_handler.list_files()
dataframes = data_handler.read_csv_files(file_names)
data_handler.concatenate_dataframes(dataframes)
data_handler.set_dates()
#print(data_handler.combined_df.head(2))
#print(data_handler.combined_df.tail(2))

In [ ]:

class DataPreprocessor:
    def __init__(self, combined_df, snapshot_start, snapshot_end):
        self.combined_df = combined_df
        self.snapshot_start = snapshot_start
        self.snapshot_end = snapshot_end
        self.lpa_data_sample_imputed = None

    def preprocess_data(self):
        df_filtered = self.combined_df
        df_filtered['receiptdate'] = pd.to_datetime(df_filtered['receiptdate'], errors='coerce')
        df_filtered = df_filtered.loc[(df_filtered['receiptdate'] > pd.to_datetime(self.snapshot_start))]
        df_filtered = df_filtered.loc[(df_filtered['receiptdate'] < pd.to_datetime(self.snapshot_end))]
        df_filtered = df_filtered.loc[(df_filtered['type'] == 'lpa')]
        df = df_filtered[["receiptdate", "uid", "casesubtype", "status", "donor_dob", "donor_postcode", "donor_gender"]]
        lpa_df = df.dropna()
        lpa_df['age'] = pd.to_datetime(lpa_df['receiptdate'], errors='coerce').dt.year - pd.to_datetime(lpa_df['donor_dob'], errors='coerce').dt.year
        lpa_df['donor_dob'] = pd.to_datetime(lpa_df['donor_dob'], errors='coerce').dt.date
        lpa_df['receiptdate'] = pd.to_datetime(lpa_df['receiptdate'], errors='coerce')
        lpa_df['year'] = lpa_df['receiptdate'].dt.year
        lpa_data_sample_imputed = lpa_df

        criteria = lpa_data_sample_imputed[(lpa_data_sample_imputed['age'] < 0) | (lpa_data_sample_imputed['age'] > 126)]
        lpa_data_sample_imputed.loc[criteria.index, 'age'] = np.nan

        most_common_age = lpa_data_sample_imputed.groupby('year')['age'].apply(lambda x: x.mode().iloc[0])
        lpa_data_sample_imputed['age'] = lpa_data_sample_imputed.apply(lambda row: most_common_age[row['year']] if pd.isna(row['age']) else row['age'], axis=1)

        # Ensure all ages from 18 to 106 are present
        all_ages = pd.DataFrame({'age': range(18, 107)})
        lpa_data_sample_imputed = lpa_data_sample_imputed.merge(all_ages, on='age', how='right')

        # Impute missing values in all columns based on the distribution of that particular age in previous years
        for column in lpa_data_sample_imputed.columns:
            if column not in ['age', 'year']:
                lpa_data_sample_imputed[column] = lpa_data_sample_imputed.groupby('age')[column].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.mean()))
        
        self.lpa_data_sample_imputed = lpa_data_sample_imputed

    def calculate_annual_receipts(self):
        df = self.lpa_data_sample_imputed

        def count_working_days(year):
            start_date = f'{year}-01-01'
            end_date = f'{year}-12-31'
            date_range = pd.date_range(start_date, end_date, freq='B')
            return len(date_range)

        results = {
            'year': [],
            'Average Daily Reciept': [],
            'Yearly Receipt': [],
            'Standard Deviation Daily Reciept': [],
            'Yearly Standard Deviation': []
        }

        for year in df['year'].unique():
            df_year = df[df['year'] == year]
            daily_reciept = df_year.groupby('receiptdate').size()
            average_daily_reciept = daily_reciept.mean()
            std_dev_daily_reciept = daily_reciept.std()
            working_days = count_working_days(year)
            yearly_reciept = average_daily_reciept * working_days
            yearly_std_dev = average_daily_reciept * working_days

            results['year'].append(year)
            results['Average Daily Reciept'].append(average_daily_reciept)
            results['Yearly Receipt'].append(yearly_reciept)
            results['Standard Deviation Daily Reciept'].append(std_dev_daily_reciept)
            results['Yearly Standard Deviation'].append(yearly_std_dev)

        annual_receipts = pd.DataFrame(results)
        annual_receipts = annual_receipts.sort_values(by=['year'])

        return annual_receipts

    def create_unique_identifier(self):
        lpa_unique = self.lpa_data_sample_imputed
        lpa_unique['donor_postcode'] = lpa_unique['donor_postcode'].str.strip().str.replace(' ', '')
        lpa_unique['unique_key'] = lpa_unique['donor_dob'].astype(str) + lpa_unique['donor_gender'] + lpa_unique['donor_postcode']

        duplicate_mask = lpa_unique.duplicated('unique_key', keep=False)
        lpa_unique = pd.concat([lpa_unique.loc[duplicate_mask & lpa_unique[['age', 'donor_gender', 'donor_postcode']].ne(0).any(axis=1)], 
                                lpa_unique[~duplicate_mask]])

        lpa_unique['month_year'] = lpa_unique['receiptdate'].dt.strftime('%b-%y')
        lpa_unique = lpa_unique.sort_values(by=['receiptdate'])
        return lpa_unique

    def calculate_unique_donors_per_year(self, lpa_df_index):
        lpa_df_index.set_index('receiptdate', inplace=True)
        annual_unique_donors = lpa_df_index.groupby(['year', 'age'])['unique_key'].nunique().reset_index(name='count')
        annual_unique_donors = annual_unique_donors.sort_values(by=['year', 'age'])
        return annual_unique_donors

    def calculate_unique_donors_count_monthly(self, lpa_df_index):
        lpa_df_no_index = lpa_df_index.reset_index()
        lpa_df_no_index['month'] = lpa_df_no_index['receiptdate'].dt.month
        monthly_unique_donors = lpa_df_no_index.groupby(['year', 'month', 'donor_gender', 'age'])['unique_key'].nunique().reset_index(name='count')
        monthly_unique_donors = monthly_unique_donors.sort_values(by=['year', 'month', 'donor_gender', 'age'])
        return monthly_unique_donors

    def calculate_unique_donors_per_day(self, lpa_df_index):     
        lpa_df_no_index = lpa_df_index.reset_index()
        lpa_df_no_index['month'] = lpa_df_no_index['receiptdate'].dt.month
        lpa_df_no_index['day'] = lpa_df_no_index['receiptdate'].dt.day
        daily_unique_donors = lpa_df_no_index.groupby(['year', 'month', 'day', 'receiptdate', 'donor_gender', 'age'])['unique_key'].nunique().reset_index(name='count')
        daily_unique_donors = daily_unique_donors.sort_values(by=['receiptdate', 'year', 'month', 'day', 'donor_gender', 'age'])
        return daily_unique_donors    
    
    def calculate_unique_donors_age_specific(self, lpa_df_index):
       #lpa_df_index.set_index('receiptdate', inplace=True)
        annual_unique_donors_age_specific = lpa_df_index.groupby(['year', 'age'])['unique_key'].nunique().reset_index(name='count')
        annual_unique_donors_age_specific = annual_unique_donors_age_specific.sort_values(by=['year', 'age'])
        return annual_unique_donors_age_specific
        
    def calculate_unique_donors_count_daily(self, lpa_df_index):
        #daily_unique_donors = lpa_df_index.groupby(['year'])['unique_key'].nunique().reset_index(name='count')
        #daily_unique_donors = daily_unique_donors.sort_values(by=['year'])
        # Group by 'year' and count unique 'unique_id'
        annual_unique_counts = lpa_df_index.groupby(lpa_df_index['year'])['unique_key'].nunique()    
        return annual_unique_counts


# short-term forecast the rest of age-specific count of donor demands for the remaining months in 2024 based on a bunch of short-term drivers like Martin Lewis broadcast 
As we have only first quarter of age-specific count of donor demands (annual_donors_age_2024), to short-term forecast the rest of quarters age-specific count of donor demands for the remaining months in 2024:
- Estimate the short-term forecasts for the remaining of 2024 by first, take the average number for receipts at working day across 2024 which currently is 5600 multiplied by the total number of working days in the year which is 256 give me the total number of expected receipts for the year (5600 * 256 = 1.433 million receipts across the whole of 2024). 
    1. Calculate the number of reciepts across the whole year (for the available data (only 6 months) on average are going to be for 2024 = average_daily_reciepts_2024
    2. Muliply (average_daily_reciepts_2024) to the Number of working days in 2024 (num_working_day_2024) = total_annual_reciepts_2024
    3. Estimate the Ratio of total_annual_reciepts_2024 to annual_donors_age_2024: ratio_reciept_donor_2024 = total_annual_reciepts_2024 / annual_donors_age_2024
- Once you have calculated the total number of receipts for 2024 then by looking at 2022/2023 historical data count how many receipts there were in 2022/2023 and how many donors were in 2022/2023 just to get the overall ratio. The starting point is just the total number for receipts so according to this that would be just over 1.4 million more receipts then scale that back based on total number for receipts 2022/2023 e.g., 70% then that would suggest that would be 1,000,000 donors roughly. E.g., just to keep it simple we consider the same overall ratio in 2022/2023 (e.g., 70%) should be use in 2024, as 2022 and 2023 are the nearest year to 2024. This will give us an estimate for the total number of donors that we might expect in 2024 based on what we're seeing about average receipts across the year in 2024.      
    1. Calculate the number of reciepts across the whole year on average for 2023 = average_daily_reciepts_2023
    2. Muliply (average_daily_reciepts_2023) to the Number of working days in 2023 (num_working_day_2023) = total_annual_reciepts_2023
    3. Muliply (average_daily_reciepts_2022) to the Number of working days in 2022 (num_working_day_2022) = total_annual_reciepts_2022
    4. Estimate the Ratio of total_annual_reciepts_2023 to annual_donors_age_2023: ratio_reciept_donor_2023 = total_annual_reciepts_2023 / annual_donors_age_2023
    5. Estimate the Ratio of total_annual_reciepts_2022 to annual_donors_age_2022: ratio_reciept_donor_2022 = total_annual_reciepts_2022 / annual_donors_age_2022
- Then scale that back to work out however how many donors that is expected to be in 2024 and then using the historical age distribution in 2022/2023 just to break that down by age so that we can plug in those figures for 2024. So once calculated the number of donors in 2024, then need to find out of those donors what percentage would be aged 18 what percentage the age 19, 20, 21, ..., 106 based on the previous distribution so 2023 or 2022 and both years as well as the total number for receipts for 2022/2023. Then once you've got the total number of estimated donors for 2024 by considering 2022/2023 and we say how many donors were aged 18, how many were aged 19, how many were aged 20, … , etc (we should have all ages incremented by a year from 18 to 106. Then take those figures for 2024 and the plug come back in the age-specific number of annual donor demands. 


# Monte-Carlo simulation 
To ensure a future-proven forecasting model with much more reliable forecast outputs by applying uncertainty, 
especially when there are short-term volatilities in the average daily receipts in the pattern of data due to some short-term variation of external factors / drivers 
(e.g., advertisement), which has an impact on the short-term,  to capture these changes in the pattern of data in the short term and then apply the uncertainty related 
to this to a long-term forecasting of age-specific donor demands, e.g., as it is evident that the number of receipts changes in the short-term 
(e.g., from the range of 4000 to 7000 (increment by 100) in the drop-down list, 
so there is volatility in the short term and the range of the average daily receipts? Each time we run the model, the forecasting output  of age-specific donor demands changes. 

# Dealing with Uncertainty & data volatility
THis is to introduce uncertainty into the model by simulating different scenarios for average daily receipts. 
This approach will capture the short-term volatility in donor receipts, leading to more reliable and robust long-term forecasts.

## Set Up Monte Carlo Simulation Parameters
set up the simulation to run multiple iterations, where each iteration randomly selects an average daily receipt value within 
a given range (e.g., 4000 to 7000). The model will then calculate the donor demand based on this value, 
and we'll aggregate the results to produce a distribution of possible outcomes.

## Run the model
- Run the Monte Carlo simulation for a specified number of iterations.
- Randomly generate an average daily receipt within the specified range for each iteration.
- Aggregate and analyse the results to provide a distribution of forecasted donor numbers.

Monte Carlo Simulation Setup:

The iterations variable defines how many times the simulation will run (in this example, 1000 iterations).
lowerBound and upperBound set the range for possible average daily receipts, reflecting the volatility (from 4000 to 7000).
Generating Random Average Daily Receipts:

For each iteration, a random average daily receipt is selected from the specified range using Application.WorksheetFunction.RandBetween(lowerBound, upperBound).
Running the Forecasting Model:

For each random receipt value, the total receipts for 2024 are calculated.
Using the ratios from 2022 and 2023, the estimated number of donors for 2024 is calculated for each iteration.
Storing and Analyzing Results:

The results of all iterations are stored in an array (monteCarloResults).
The mean and standard deviation of the forecasted donor numbers for each age group are calculated across all iterations.
Visualizations:

Two charts are added to visualize the results:
A clustered column chart showing the mean forecasted number of donors by age.
A line chart showing the standard deviation of forecasted donors by age, indicating the uncertainty.
Output:

The final results are saved in the "Forecast_2024" sheet, and the charts are added for visual analysis.
Running the Simulation:
Each time you run the ForecastWithMonteCarlo macro, it will generate different results based on the random variation in average daily receipts. This randomness helps to account for short-term volatility, making the forecast more robust and reliable over the long term.
Interpretation of Results:
The mean values give an expected forecast for each age group based on the Monte Carlo simulation.
The standard deviation indicates the level of uncertainty or volatility in the forecasted numbers. A higher standard deviation means more variability in the donor forecasts for that age group, suggesting that predictions for this group are more sensitive to changes in daily receipts.
This approach ensures that the forecasting model accounts for potential short-term variations and provides a more nuanced understanding of the possible outcomes.

## Additional Columns:

Year: Added to specify the forecast year.
AgeDistribution2022 & AgeDistribution2023: These columns store the age-specific distribution of donors for the years 2022 and 2023.
AverageAgeDistribution: This column contains the average age distribution calculated from 2022 and 2023 data.
RandomReceipt: Stores the random daily receipt value used in each iteration.
MonteCarloResults: Holds the forecasted donor number for each iteration.
Sum of Values and Sum of Squared Values: These are intermediate calculations needed to compute the mean and standard deviation for the forecasted donor numbers.
Final Forecast (Mean) and Final Forecast (StdDev): The mean and standard deviation of the forecasted donor numbers after all iterations.
Monte Carlo Simulation:

Each iteration generates a random receipt value and calculates the forecasted donor numbers. The results are stored and used to compute the final forecast.
Visualizations:

Clustered Column Chart: Visualizes the mean forecasted donor numbers by age group.
Line Chart: Shows the standard deviation (uncertainty) in the forecasted donor numbers for each age group.
Interpretation:
Year column gives context to the forecast year, which is 2024 in this scenario.
Age Distribution Columns show how donor distributions are evolving over time.
Random Receipt column illustrates the volatility and randomness considered in each iteration.
Monte Carlo Results provide insight into the forecasted donor numbers for each random receipt value.
Sum of Values and Sum of Squared Values are used to calculate the final forecast's mean and standard deviation, indicating the expected donor numbers and their uncertainty.
The charts visually summarize the data, showing how different age groups' donor forecasts change under various scenarios and the associated uncertainties.
This detailed output will help in analyzing the Monte Carlo simulation results in a more comprehensive manner.

In [ ]:
# Calculate Uncertainty Ratio: average annual dornor demands and average annual reciepts:
# Calculate the Ratio of Short-Term to Long-Term Demand:
# Compute the ratio of the average short-term demand to the average long-term demand. 
# This ratio represents the relative impact of the short-term period on the long-term forecast.

## -------------------
# Instantiate the DataPreprocessor
#data_preprocessor = DataPreprocessor(data_handler.combined_df, data_handler.snapshot_start, data_handler.snapshot_end)
data_preprocessor = DataPreprocessor(data_handler.combined_df, '2007-01-01', data_handler.snapshot_end)
data_preprocessor.preprocess_data()

In [ ]:
# Calculate the The total annual reciepts
annual_receipts = data_preprocessor.calculate_annual_receipts()
results = annual_receipts[['year', 'Yearly Receipt']]
#print(f"Total annual reciepts: {results}")

total_annual_reciepts_2022 = results['Yearly Receipt'].iloc[0]
#print(f"total_annual_reciepts_2022: {results['Yearly Receipt'].iloc[0]}")

total_annual_reciepts_2023 = results['Yearly Receipt'].iloc[1]
#print(f"total_annual_reciepts_2023: {results['Yearly Receipt'].iloc[1]}")

total_annual_reciepts_2024 = results['Yearly Receipt'].iloc[2]
#print(f"total_annual_reciepts_2024: {results['Yearly Receipt'].iloc[2]}")

In [ ]:
lpa_df_index = data_preprocessor.create_unique_identifier()
monthly_unique_donors = data_preprocessor.calculate_unique_donors_count_monthly(lpa_df_index)
daily_unique_donors = data_preprocessor.calculate_unique_donors_per_day(lpa_df_index)
annual_donor_demands = data_preprocessor.calculate_unique_donors_count_daily(lpa_df_index)
annual_unique_donors_age_specific = data_preprocessor.calculate_unique_donors_age_specific(lpa_df_index)
annual_unique_donors = data_preprocessor.calculate_unique_donors_per_year(lpa_df_index)

In [ ]:
# Calculate the Total annual donor demands based on 'unique_id'
daily_unique_donors.sort_values(by=['year', 'age'])
total_annual_donor = daily_unique_donors[['year', 'count']]
total_annual_donor = total_annual_donor.groupby(['year']).sum()

total_annual_donor_2022 = total_annual_donor.iloc[0]
total_annual_donor_2023 = total_annual_donor.iloc[1]
total_annual_donor_2024 = total_annual_donor.iloc[2]

annual_unique_donors = annual_unique_donors.rename(columns={'count': 'demand'})
#print(f"Total annual donor demands: {total_annual_donor}")
#print(f"Total annual donor demands for 2022: {total_annual_donor_2022}")
#print(f"Total annual donor demands for 2023: {total_annual_donor_2023}")
#print(f"Total annual donor demands for 2024: {total_annual_donor_2024}")

In [ ]:
# Impute missing values in all columns based on the distribution of that particular age in previous years

# Create a DataFrame with all ages from 18 to 27 for each year
all_ages = pd.DataFrame([(year, age) for year in annual_unique_donors['year'].unique() for age in range(18, 107)], columns=['year', 'age'])

# Merge the original DataFrame with the all_ages DataFrame
df_full = pd.merge(all_ages, annual_unique_donors, on=['year', 'age'], how='left')

# Fill missing values using the age distribution

#pivot_table1 = pivot_table1.apply(lambda x: x.fillna(x.mean()), axis=1)
#df_full['demand'] = df_full.groupby('year')['demand'].transform(lambda x: x.fillna(x.mean()))

# Fill missing values using the previous three ages' demand values or next three ages' demand values
def fill_missing_values(group):
    for i in range(len(group)):
        if pd.isna(group.iloc[i]['demand']):
            prev_values = group.iloc[max(0, i-3):i]['demand']
            next_values = group.iloc[i+1:min(len(group), i+4)]['demand']
            if not prev_values.isna().all():
                group.at[i, 'demand'] = prev_values.mean()
            elif not next_values.isna().all():
                group.at[i, 'demand'] = next_values.mean()
    return group

# Handling Missing Data
#pivot_tbl = pivot_tbl.fillna(0)
df_full = df_full.groupby('year').apply(fill_missing_values).reset_index(drop=True)
df_full = round(df_full.fillna(0), 0)
# # Create a pivot table
# pivot_table = df_full.pivot_table(index='year', columns='age', values='demand', aggfunc='sum')

# # Add the column label 'year' in the first row and first column
# pivot_table.index.name = 'age'
# #pivot_table.reset_index(inplace=True)

# donor_pivot_trns = pivot_table.transpose()
# Print the pivot table
#print(donor_pivot_trns)



# Remove the age label in the first column and first row
#pivot_table.columns.name = None
#pivot_table.index.name = None

# Reset index to make 'age' a column again
#pivot_table.reset_index(inplace=True)

#print(pivot_table)


# # Create a pivot table
# pivot_tbl = df_full.pivot_table(index='year', columns='age', values='demand', aggfunc='sum')

# # Example operations
# # 1. Aggregation
# pivot_sum = pivot_tbl.sum()
# pivot_mean = pivot_tbl.mean()

# # 2. Filtering
# filtered_pivot = pivot_tbl[pivot_tbl[22] > 10]

# # 3. Grouping
# grouped_pivot = df_full.groupby(['year', 'age']).sum().unstack()

# # 4. Reshaping
# stacked_pivot = pivot_tbl.stack()
# unstacked_pivot = stacked_pivot.unstack()

# # 5. Handling Missing Data
# filled_pivot = pivot_tbl.fillna(0)

# # 6. Sorting
# # if 22 in pivot_tbl.columns:
# #     sorted_pivot = pivot_tbl.sort_values(by=22, axis=1)
# # else:
# #     sorted_pivot = pivot_tbl

# # 7. Applying Functions
# applied_pivot = pivot_tbl.apply(np.log1p)

# # 8. Pivot Table Customization
# pivot_with_margins = df_full.pivot_table(index='year', columns='age', values='demand', aggfunc='sum', margins=True)

# # Print examples
# print("Sum:\n", pivot_sum)
# print("Mean:\n", pivot_mean)
# print("Filtered:\n", filtered_pivot)
# print("Grouped:\n", grouped_pivot)
# print("Stacked:\n", stacked_pivot)
# print("Unstacked:\n", unstacked_pivot)
# print("Filled:\n", filled_pivot)
# #print("Sorted:\n", sorted_pivot)
# print("Applied Function:\n", applied_pivot)
# print("With Margins:\n", pivot_with_margins)

In [ ]:

# Create the pivot table
df = df_full
pivot_table = df.pivot_table(index='age', columns='year', values='demand', fill_value=0)
#print(pivot_table)

pivot_tbl = pivot_table
# Reset index to make 'age' a column again
pivot_tbl.reset_index(inplace=True)

# Remove the second row
pvt_tbl = pivot_tbl.drop(0)

# Remove the second column (2007.0)
pvt_tbl.drop(columns=[0.0], inplace=True)
#pvt_tbl = pvt_tbl.pop(0.0)

# Set a label to the first column
pvt_tbl.rename(columns={'': 'Age Group'}, inplace=True)

# Remove the index
pvt_tbl.reset_index(drop=True, inplace=True)
print(pvt_tbl)
#pvt_df = pvt_tbl.pop('index')
#pvt_df
#pivot_annual_unique_donors = pvt_tbl.drop(columns=['year'], inplace=True)
#pivot_annual_unique_donors


# Remove the age label in the first column and first row
#pivot_tbl.columns.name = None
#pivot_tbl.index.name = None
#print(pivot_tbl)

#pvt_tbl = pivot_tbl.pop(1)
#print(pvt_tbl)
# Reorder the columns to move 'age' to the first position
#pivot_tbl = pivot_tbl[
# Insert 'age' as the first column
#total_life_tbl.insert(0, 'age', first_column)

pivot_annual_unique_donors = pvt_tbl # donor_pivot_trns
#pivot_annual_unique_donors = pivot_annual_unique_donors[(pivot_annual_unique_donors['age'] >= 18) & (pivot_annual_unique_donors['age'] <= 106)]
pivot_annual_unique_donors.to_csv('pivot_annual_unique_donors.csv', index=False)

In [ ]:
# Save to CSV file
historical_data_df = df_full
#historical_data_df = df_full[(df_full['age'] >= 18) & (df_full['age'] <= 106)]
historical_data_df.to_csv('annual_unique_donors.csv', index=False)

annual_receipts_historical = annual_receipts[['year', 'Yearly Receipt']]
annual_receipts_historical.to_csv('annual_receipts.csv', index=False)
annual_demands_historical = historical_data_df
annual_unique_donors_age_specific = historical_data_df
annual_demands_age_specific_historical = annual_unique_donors_age_specific
annual_unique_donors_age_specific.to_csv('annual_unique_donors_age_specific.csv', index=False)

monthly_unique_donors = monthly_unique_donors.rename(columns={'count': 'demand'})
# Filter the data to include only ages from 18 to 106
historical_demand_monthly = monthly_unique_donors[(monthly_unique_donors['age'] >= 18) & (monthly_unique_donors['age'] <= 106)]
historical_demand_monthly.to_csv('historical_demand_monthly.csv', index=False)

daily_unique_donors = daily_unique_donors.rename(columns={'count': 'demand'})
# Filter the data to include only ages from 18 to 106
historical_demand_daily = daily_unique_donors[(daily_unique_donors['age'] >= 18) & (daily_unique_donors['age'] <= 106)] 
historical_demand_daily.to_csv('historical_demand_daily.csv', index=False)

In [ ]:
# Filter rows where 'year' is 2023 or 2022
filtered_annual_receipts = annual_receipts_historical #[annual_receipts_historical['year'].isin([2023, 2022, 2021, 2020])]
#print(f"annual_donor_receipts is: {filtered_annual_receipts}")


filtered_annual_demands = annual_demands_historical #[(annual_demands_historical.index == 2020) | (annual_demands_historical.index == 2021) | (annual_demands_historical.index == 2022) | (annual_demands_historical.index == 2023) | (annual_demands_historical.index == 2024)]
#print(f"annual_donor_demands is: {filtered_annual_demands}")

filtered_annual_demands_age_specific = annual_demands_age_specific_historical #[annual_demands_age_specific_historical['year'].isin([2023, 2022, 2021, 2020])]
# Extract and save data into a csv file
filtered_annual_demands_age_specific = filtered_annual_demands_age_specific.rename(columns={'count': 'demand'})

#print(f"annual_donor_demands_age_specific is: {filtered_annual_demands_age_specific}") 

# Save to CSV file
filtered_annual_demands_age_specific.to_csv('annual_demands_age_specific.csv', index=False)

In [ ]:

# Load and Preprocess Data: Load the historical data and filter it for the age range 18-106.
# Calculate Annual Receipts: Calculate the total annual receipts for 2022, 2023, and 2024 based on the provided daily averages and working days.
# Estimate Ratios: Calculate the ratios of receipts to donors for 2022 and 2023.
# Forecast Donors for 2024: Estimate the total number of donors for 2024 based on the historical ratios.
# Age-Specific Distribution: Use historical age distributions from 2022 and 2023 to distribute the forecasted donors by age for 2024.
# Combine Forecasts: Integrate these estimates into the final forecast for age-specific donor demands for the remaining months in 2024.

# class to provide short-term forecast the rest of age-specific count of donor demands for the remaining months in 2024 based on a bunch of short-term drivers like Martin Lewis broadcast 
# As we have only first quarter of age-specific count of donor demands (annual_donors_age_2024), to short-term forecast the rest of quarters age-specific count of donor demands for the remaining months in 2024:
# - Estimate the short-term forecasts for the remaining of 2024 by first, take the average number for receipts at working day across 2024 which currently is 5600 multiplied by the total number of working days in the year which is 256 give me the total number of expected receipts for the year (5600 * 256 = 1.433 million receipts across the whole of 2024). 
#     1. Calculate the number of receipts across the whole year (for the available data (only 6 months) on average are going to be for 2024 = average_daily_reciepts_2024
#     2. Multiply (average_daily_reciepts_2024) to the Number of working days in 2024 (num_working_day_2024) = total_annual_reciepts_2024
#     3. Estimate the Ratio of total_annual_reciepts_2024 to annual_donors_age_2024: ratio_reciept_donor_2024 = total_annual_reciepts_2024 / annual_donors_age_2024
# - Once you have calculated the total number of receipts for 2024 then by looking at 2022/2023 historical data count how many receipts there were in 2022/2023 and how many donors were in 2022/2023 just to get the overall ratio. The starting point is just the total number for receipts so according to this that would be just over 1.4 million more receipts then scale that back based on total number for receipts 2022/2023 e.g., 70% then that would suggest that would be 1,000,000 donors roughly. E.g., just to keep it simple we consider the same overall ratio in 2022/2023 (e.g., 70%) should be use in 2024, as 2022 and 2023 are the nearest year to 2024. This will give us an estimate for the total number of donors that we might expect in 2024 based on what we're seeing about average receipts across the year in 2024.      
#     1. Calculate the number of receipts across the whole year on average for 2023 = average_daily_reciepts_2023
#     2. Multiply (average_daily_reciepts_2023) to the Number of working days in 2023 (num_working_day_2023) = total_annual_reciepts_2023
#     3. Multiply (average_daily_reciepts_2022) to the Number of working days in 2022 (num_working_day_2022) = total_annual_reciepts_2022                                                                    
#(Note: assume we have already calculated the above figures:  
# total_annual_reciepts_2022: 751967.5346260389
# total_annual_reciepts_2023: 970035.0282485875
# total_annual_reciepts_2024: 1124615.1902439024                                                              
# Total annual donor demands for 2022: count    675089
# Total annual donor demands for 2023: count    852627
# Total annual donor demands for 2024: count    568367  
#     4. Estimate the Ratio of total_annual_reciepts_2023 to annual_donors_age_2023: ratio_reciept_donor_2023 = total_annual_reciepts_2023 / annual_donors_age_2023
#     5. Estimate the Ratio of total_annual_reciepts_2022 to annual_donors_age_2022: ratio_reciept_donor_2022 = total_annual_reciepts_2022 / annual_donors_age_2022  
# - Then scale that back to work out however how many donors that is expected to be in 2024 and then using the historical age distribution in 2022/2023 just to break that down by age so that we can plug in those figures for 2024. So once calculated the number of donors in 2024, then need to find out of those donors what percentage would be aged 18 what percentage the age 19, 20, 21, …, 106 based on the previous distribution so 2023 or 2022 and both years as well as the total number for receipts for 2022/2023. Then once you've got the total number of estimated donors for 2024 by considering 2022/2023 and we say how many donors were aged 18, how many were aged 19, how many were aged 20, … , etc 
# (we should have all ages incremented by a year from 18 to 106. Then take those figures for 2024 and the plug come back in the age-specific number of annual donor demands. based on the attached data?  

# Load the historical data
#historical_data_path = 'annual_unique_donors_22_23_24.csv'

# Filter the data to include only ages from 18 to 106
filtered_data_df = historical_data_df[(historical_data_df['age'] >= 18) & (historical_data_df['age'] <= 106)]

# # Add a column for the month (assuming the month is in the 'month' column in the dataset)
# # If the dataset doesn't have month data, this step needs to be adjusted accordingly
# # Here we assume that the dataset has a 'month' column with values from 1 to 12
# filtered_data_df['month'] = pd.to_datetime(filtered_data_df['date']).dt.month

# # Define a function to calculate quarters
# def calculate_quarter(month):
#     if 1 <= month <= 3:
#         return 1
#     elif 4 <= month <= 6:
#         return 2
#     elif 7 <= month <= 9:
#         return 3
#     elif 10 <= month <= 12:
#         return 4

# # Apply the function to add the 'quarter' column
# filtered_data_df['quarter'] = filtered_data_df['month'].apply(calculate_quarter)

# Calculate annual donor receipts and demands for the provided years
annual_donor_demands = filtered_data_df.groupby('year')['demand'].sum()

# Load the additional data provided in the task
average_daily_receipts_2022 = total_annual_reciepts_2022 / 256
average_daily_receipts_2023 = total_annual_reciepts_2023 / 256
average_daily_receipts_2024 = 5600  # Given average daily receipts for 2024

num_working_days_2022 = 256
num_working_days_2023 = 256
num_working_days_2024 = 256

total_annual_receipts_2022 = round(average_daily_receipts_2022 * num_working_days_2022, 2)
print(f"The total number of receipts for 2022: {total_annual_receipts_2022}")

total_annual_receipts_2023 = round(average_daily_receipts_2023 * num_working_days_2023, 2)
print(f"The total number of receipts for 2023: {total_annual_receipts_2023}")

total_annual_receipts_2024 = round(average_daily_receipts_2024 * num_working_days_2024, 2)
print(f"The total number of receipts for 2024: {total_annual_receipts_2024}")

# Calculate ratios of receipts to donors for 2022 and 2023
ratio_receipt_donor_2022 = round(total_annual_receipts_2022 / annual_donor_demands[2022], 2)
print(f"The Ratio of total number of receipt to donor demands for 2022: {ratio_receipt_donor_2022}")

ratio_receipt_donor_2023 = round(total_annual_receipts_2023 / annual_donor_demands[2023], 2)
print(f"The Ratio of total number of receipt to donor demands for 2023: {ratio_receipt_donor_2023}")

# Estimate the total number of donors for 2024 based on historical ratios
estimated_donors_2024 = round(total_annual_receipts_2024 / ((ratio_receipt_donor_2022 + ratio_receipt_donor_2023) / 2), 2)
print(f"The estimate total number of donors for 2024 based on historical ratios: {estimated_donors_2024}")

# Age-specific distribution based on 2022 and 2023
age_distribution_2022 = filtered_data_df[filtered_data_df['year'] == 2022].groupby('age')['demand'].sum() / annual_donor_demands[2022]
print(f"Age-specific distribution based on 2022: {age_distribution_2022}")

age_distribution_2023 = filtered_data_df[filtered_data_df['year'] == 2023].groupby('age')['demand'].sum() / annual_donor_demands[2023]
print(f"Age-specific distribution based on 2023: {age_distribution_2023}")

# Average age distribution
average_age_distribution = round((age_distribution_2022 + age_distribution_2023) / 2, 6)
print(f"Average age-specific distribution based on 2022/2023: {average_age_distribution}")

# Estimate the age-specific number of donors for 2024
age_specific_donors_2024 = estimated_donors_2024 * average_age_distribution
print(f"Age-specific distribution for 2024: {age_specific_donors_2024}")

# Ask the user to specify the available quarters for 2024
#available_quarters = input("Enter the quarters that data is available for 2024 (e.g., 1, 2): ")
#available_quarters = list(map(int, available_quarters.split(',')))
#available_quarters

# Forecast the remaining months in 2024 based on the first quarter data
first_quarter_data = filtered_data_df[(filtered_data_df['year'] == 2024)] #& (filtered_data_df['quarter'] == 1)]
print(f"The first quarter available data for 2024: {first_quarter_data}")

total_first_quarter_demand = first_quarter_data['demand'].sum()
print(f"The total first quarter available donor demands for 2024: {total_first_quarter_demand}")

remaining_demand_2024 = total_annual_receipts_2024 - total_first_quarter_demand
print(f"The remaining donor demands for 2024: {remaining_demand_2024}")

# Distribute the remaining demand across ages based on the average age distribution
remaining_age_specific_donors_2024 = remaining_demand_2024 * average_age_distribution
print(f"The remaining age-specific donor demands for 2024: {remaining_age_specific_donors_2024}")

# Create a DataFrame to store the forecasted age-specific donors for the remaining months of 2024
forecast_df = pd.DataFrame({
    'age': average_age_distribution.index,
    'forecasted_donors': remaining_age_specific_donors_2024
}).reset_index(drop=True)

print(f"The age-specific donor demands for 2024:")
print(forecast_df)
forecast_df.to_csv('estimated age-specific donor demands for 2024.csv')

In [ ]:
# a function in python that apply uncertainty to a short-term forecsasts of number of demands from April 2024 for 2 years 
# by multiplying the SARIMA forecasted values by two ratios: 1. annual_donor_receipts (average daily demnads per year) divided 
# by annual_donor_demands (average daily reciepts per year) and 2. (age-specific demnads ratio (claculated by estimating the percentage 
#                                                                                               of demands per year for each age from 18 to 106). 
#                                                                   Also if there are no demands in a specific age in the corresponding year then consider 
#                                                                   a zero demand for that age. 
# Here is the age specific count of demnds in 2022, 2023, and January 2024, February 2024 and March 2024.  
# To accomplish this task, we need to:

# Forecast the number of demands for the next two years using a SARIMA model.
# Apply uncertainty by multiplying the forecasted values by the specified ratios:
# The ratio of annual donor receipts to annual donor demands.
# The age-specific demands ratio.
# Here's a step-by-step Python function to achieve this:

# Forecasting using SARIMA: I'll assume the SARIMA model is already built and forecasted values are available.
# Ratio Calculation:
# Calculate the ratio of annual donor receipts to annual donor demands.
# Calculate age-specific demands ratio.
# Apply Ratios: Adjust the forecasted values by these ratios.

# 1. Forecasting the overall demand.
# 2. Distributing the forecasted demand among different age groups using age-specific ratios.
# 3. Adjusting these distributed forecasts using the month and age-specific ratios.


# Load the historical data
#historical_data_path = '/annual_unique_donors.csv'
#pd.read_csv(historical_data_path)

#historical_demand_daily['month'] = historical_demand_daily['month_year'].dt.month
#historical_demand_daily['day'] = historical_demand_daily['receiptdate'].dt.day
# Calculate annual donor receipts and demands
# annual_donor_receipts = historical_data_df.groupby('year')['demand'].sum().reset_index(name='Yearly Receipt')
# annual_donor_demands = historical_data_df.groupby('year')['demand'].sum()
# Annual receipts and demands DataFrames
annual_donor_receipts = filtered_annual_receipts  #pd.DataFrame({'year': [2022, 2023, 2024], 'Yearly Receipt': [7.519675e+05, 9.700350e+05, 1.124615e+06]})

annual_donor_demands = filtered_annual_demands  #pd.Series({2022: 601254, 2023: 752898, 2024: 505219}, name='unique_key')

# Age-specific demands DataFrame
age_specific_demands = filtered_annual_demands_age_specific #pd.DataFrame({
#     'year': [2022, 2022, 2022, 2023, 2023, 2023, 2024, 2024, 2024],
#     'age': [18, 19, 20, 18, 19, 20, 18, 19, 20],
#     'demand': [199, 286, 192, 249, 345, 243, 122, 267, 196]})

# Calculate age-specific demands
annual_donor_demands_age_specific = filtered_annual_demands_age_specific #historical_data_df.groupby(['year', 'age'])['demand'].sum().reset_index()

# # Define a function to fit SARIMA model and forecast future demands
# def fit_sarima_and_forecast(data, start_year, forecast_periods):
#     model = sm.tsa.statespace.SARIMAX(data,
#                                       order=(1, 1, 1),
#                                       seasonal_order=(1, 1, 1, 12),
#                                       enforce_stationarity=False,
#                                       enforce_invertibility=False)
#     results = model.fit(disp=False)
#     forecast = results.get_forecast(steps=forecast_periods)
#     forecast_values = forecast.predicted_mean
#     forecast_index = pd.date_range(start=start_year, periods=forecast_periods, freq='MS')
#     forecast_df = pd.DataFrame({'Forecast': forecast_values}, index=forecast_index)
#     return forecast_df

In [ ]:
start_year = 2007
end_ons_year = 2022 # The last avalilable year extracted mortality statistics from ONS
final_year = 2024
end_year = end_ons_year
# Short term forecast extended years
end_forecast_year = 2026

# Long term forecast extended years
year_forecast_long = 2039

start_age = 18
end_age = 106

# Years from 2007 to 2022 (year_forecast_long the single life table is extracted from ONS and is available)
years_ons = list(range(start_year, end_ons_year + 1))

# Extend the table for years 2023 to 2026 (single life table is extracted from ONS and is NOT available) 
extended_years = list(range(end_ons_year+1, year_forecast_long + 1))

# All years 2023 to 2026
years = list(range(start_year, year_forecast_long + 1))

# Ages from 18 to 100 (age in the single life table is extracted from ONS and is available)
ages_100 = list(range(18, 101))

# Extend ages from 100 to 106
extended_ages = list(range(100, end_age+1))


# Ages from 18 to 106
age_range = range(18, end_age+1)

ages = list(age_range)

years_forecast_long = list(range(start_year, year_forecast_long+1))

In [ ]:

# Monte Carlo Simulation number of iterations
iterations = 1000

# Assumptions for the average range of reciepts
lower_bound = 4000
upper_bound = 7000

# Number of working days in 2024
num_working_days_2024 = 256

# Fixed total receipts for 2022 and 2023
total_receipts_2022 = 751967.534626039
total_receipts_2023 = 970035.028248588

lpa_demands = annual_donor_demands

# Calculate total demands for 2022 and 2023
# Filter the DataFrame for 2022 and 2023 data
data_2022 = lpa_demands[lpa_demands['year'] == 2022]
data_2023 = lpa_demands[lpa_demands['year'] == 2023]

# Calculate total demands for 2022 and 2023
total_demand_2022 = data_2022['demand'].sum()
total_demand_2023 = data_2023['demand'].sum()

# Aggregating age distribution: The age distributions for 2022 and 2023 are aggregated and normalised
# Aggregate and normalise age distributions for 2022 and 2023
age_distribution_2022 = data_2022.groupby('age')['demand'].sum().to_dict()
age_distribution_2023 = data_2023.groupby('age')['demand'].sum().to_dict()

for age in age_distribution_2022:
    age_distribution_2022[age] /= total_demand_2022
# print(age_distribution_2022)
for age in age_distribution_2023:
    age_distribution_2023[age] /= total_demand_2023
# print(age_distribution_2023)

# Average age distribution: Calculates the average across both years for shared age groups.
# Average age distribution across 2022 and 2023
average_age_distribution = {}
for age in age_distribution_2022:
    if age in age_distribution_2023:
        average_age_distribution[age] = (age_distribution_2022[age] + age_distribution_2023[age]) / 2
#print(average_age_distribution)

# # Check for NaN values
# has_nan = total_demand_2022.isna().any()
# print("total_demand_2022 has NaN values:", has_nan)
# has_nan = total_demand_2023.isna().any()
# print("total_demand_2023 has NaN values:", has_nan)

# # Check for NaN values
# has_nan = age_distribution_2022.isna().any()
# print("total_demand_2022 has NaN values:", has_nan)
# # Detect NA values
# na_values = age_distribution_2022.isnull()
# print(age_distribution_2022[na_values])
# # Detect NA values using numpy.where()
# na_indices = np.where(age_distribution_2022.isna())[0]
# print(f"Null values in age_distribution_2022: {na_indices}")

# has_nan = age_distribution_2023.isna().any()
# print("total_demand_2023 has NaN values:", has_nan)
# # Detect NA values
# na_values = age_distribution_2023.isnull()
# print(age_distribution_2023[na_values])
# # Detect NA values using numpy.where()
# na_indices = np.where(age_distribution_2023.isna())[0]
# print(f"Null values in age_distribution_2023: {na_indices}")

# # NA fill 
# # Fill NA values with the mean of the Series
# filled_series = series.fillna(series.mean())
# # Forward fill NA values (propagate last valid observation forward)
# filled_series = series.fillna(method='ffill')
# # Backward fill NA values (propagate next valid observation backward)
# filled_series = series.fillna(method='bfill')
# print(filled_series)


# reciept ratios for 2022 and 2023
ratio_receipt_donor_2022 = total_receipts_2022 / total_demand_2022
ratio_receipt_donor_2023 = total_receipts_2023 / total_demand_2023
# Average reciept ratio
average_reciept_ratio = (ratio_receipt_donor_2022 + ratio_receipt_donor_2023) / 2


# Monte Carlo simulation for 2024
# Monte Carlo simulation results initialisation
monte_carlo_results = np.random.uniform(lower_bound, upper_bound, iterations)
#monte_carlo_results = []
#monte_carlo_results = np.zeros(iterations)
for i in range(iterations):
    #random_receipt = random.randint(lower_bound, upper_bound)
    random_receipt = np.random.uniform(lower_bound, upper_bound)
    total_receipts_2024 = random_receipt * num_working_days_2024
    
    estimated_donors_2024 = total_receipts_2024 / average_reciept_ratio  ##    ???????? should be devided by the ratio or multiply by 1+ratio or multiply by ratio?????
    #monte_carlo_results.append(estimated_donors_2024)
    monte_carlo_results[i] = estimated_donors_2024
#print(monte_carlo_results)


##    ???????? should be devided by the ratio or multiply by 1+ratio or multiply by ratio?????



# for age in average_age_distribution.index:
#     sum_values = sum(monte_carlo_results[i] * average_age_distribution[age] for i in range(iterations))
#     sum_squared_values = sum((monte_carlo_results[i] * average_age_distribution[age]) ** 2 for i in range(iterations))
# Initialise an empty DataFrame for output
output_data = []
results = []

# Add data for each age group for 2024
for age_group in average_age_distribution.keys():
    sum_values = 0
    sum_squared_values = 0
    
    for i in range(iterations):
        age_value = monte_carlo_results[i] * average_age_distribution[age_group]
        sum_values += age_value
        sum_squared_values += age_value ** 2

    # Calculate mean and standard deviation for the forecast
    forecasted_donors_mean = sum_values / iterations
    forecasted_donors_stddev = np.sqrt(sum_squared_values / iterations - (sum_values / iterations) ** 2)

    output_data.append([
        2024, age_group, forecasted_donors_mean, forecasted_donors_stddev, 
        monte_carlo_results[i], age_distribution_2022[age_group], age_distribution_2023[age_group], 
        average_age_distribution[age_group], random_receipt
    ])

# Convert to DataFrame for display or saving
output_df = pd.DataFrame(output_data, columns=[
    'year', 'age', 'Forecasted Donors Mean', 'Forecasted Donors StdDev', 
    'monte_carlo_results', 'Age Distribution 2022', 'Age Distribution 2023',
    'Average Age Distribution', 'Random Receipt'
])
  
# Convert to DataFrame for further processing
forecast_df = output_df

# Print or save the output
#print(forecast_df.head(5))
forecast_df

# # Plotting the Forecast
# plt.figure(figsize=(10, 6))
# plt.errorbar(df_output['age'], df_output['Forecast Mean'], 
#              yerr=df_output['Forecast StdDev'], fmt='o', label='Forecast')
# plt.title("Long-Term Forecast Market Shape")
# plt.xlabel('age')
# plt.ylabel('Forecasted Donors ')
# plt.legend()
# plt.grid(True)
# plt.show()


# Donor Estimates & Actuals

lpa_donors = historical_data_df
lpa_donors = lpa_donors.rename(columns={'year': 'year', 'age': 'age', 'demand': 'lx'})

# Calculate 2024 estimate
short_term_forecast_2024 = np.array(forecast_df['Forecasted Donors Mean'])

# Replace the new estimate of 2024 based on short term forecast
lpa_donors.loc[lpa_donors['year'] == 2024, 'lx'] = short_term_forecast_2024


#annual_unique_donors.loc[(annual_unique_donors['year'] == 2017) & (annual_unique_donors['age'] == 18), 'demand']
data = lpa_donors
# Filter the data for the year 2024 to use as the forecast baseline
data_2024 = data[data['year'] == 2024]

# Only short term Naive forecast (2-year)
forecast_years = range(2025, 2040)

# Initialize forecast_lpa with the original data
forecast_lpa = data.copy()

# For each year in the forecast period, copy the demand from 2024 for each age
for year in forecast_years:
    # Create a new dataframe for the forecast year with the same 'age' and 'demand' as 2024
    forecast_data = data_2024.copy()
    forecast_data['year'] = year  # Update the year to the forecast year
    
    # Append the forecast data to the cumulative forecast_lpa dataframe
    forecast_lpa = pd.concat([forecast_lpa, forecast_data])

# Reset index and remove duplicates (if necessary)
forecast_lpa.reset_index(drop=True, inplace=True)
forecast_lpa_df = pd.DataFrame(forecast_lpa)

# Display the forecast data
print(forecast_lpa)

#forecast_lpa_df = forecast_lpa_df.rename(columns={'year': 'year', 'age': 'Age', 'demand': 'lx'})
print(forecast_lpa_df)


# Create the pivot table
df = forecast_lpa_df
pivot_table = df.pivot_table(index='age', columns='year', values='lx', fill_value=0)
#print(pivot_table)

pivot_tbl = pivot_table
# Reset index to make 'age' a column again
pivot_tbl.reset_index(inplace=True)

# Remove the second row
pvt_tbl = pivot_tbl.drop(0)

# Remove the second column (2007.0)
pvt_tbl.drop(columns=[0.0], inplace=True)
#pvt_tbl = pvt_tbl.pop(0.0)

# Set a label to the first column
pvt_tbl.rename(columns={'': 'Age Group'}, inplace=True)

# Remove the index
pvt_tbl.reset_index(drop=True, inplace=True)
#print(pvt_tbl)

Donor_Estimates_Actuals = pvt_tbl
# df = Donor_Estimates_Actuals
# df.set_index('year', inplace=True)
# df = df.drop(df.index[0])
# df.reset_index(drop=True, inplace=True)
# Donor_Estimates_Actuals = df

# Save to CSV
output_file_path = "Donor_Estimates_Actuals.csv"
Donor_Estimates_Actuals.to_csv(output_file_path, index=False)

Donor_Estimates_Actuals

# Survival Rate
- Calculate the survival rate for each year from 2007 to 2030 and for ages 18 to 106.
- utilise the scikit-survival package:
    1. Prepare the data: input the mortality rates (qx) as an array of probabilities for each age (18-106).
    2. Initialise the number of donors for each year and age: This will include the actual LPA data for the years 2007-2023.
    3. Calculate the survival rate for future years: Using the mortality statistics for males and females, compute the survival rate.

## Forecast Mortality rates (qx) for ages 49 to 106:

- Excel Formula: 
=10^@TREND(Log(average(mortality rate for Age=45:49)), (Age=49:94), (Age=90,Age^2=90^2)))

0. Load the mortality rates (qx) from the CSV.

1. The first step Compute the logarithm of these average mortality rates for age 49 to 94 - Log(average(mortality rate For Age from 49 to 94)):
    - Calculate Calculate the average mortality rates for the specified age groups.
    - Calculate the natural logarithm (base e) of the average mortality rate. 

2. Perform linear regression based on these values: 
    - The regression model uses ages 49 to 94 and their squares as independent variables. The Log(average(mortality rate for Age=45:49)) claculated in the first step is used as the independent variable in the TREND function.
    - The logarithms of the mortality rates, are the dependent variable. It predicts the logarithm of the mortality rate for age 90 to 115.

3. Predict the logarithm of mortality rates for ages 90 to 106: 
    - The model predicts the logarithm of the mortality rate for from 90 to 115 based on regression.
    - This represents the new data point for which the prediction is needed. E.g., if the independent variables is for age=90, then calculate the square of age 90 (which is 8100).

4. Apply Exponential Back-Transformation to get the predicted mortality rates on the original scale.
    - Finally, raising 10 to the power of the predicted logarithm converts it back to the original mortality rate scale. The result of the TREND function is raised to the power of 10, effectively converting the logarithm back to the original scale.


How to calculate the number of Living customers from year 2007 to 2040 for age from 18 to 106, by first computig the multiple years (2007 to 2040) Life tables, Cohort survival table, and finally Living customers tables?
Given the follwoing numbers (mx, qx, Ix, dx, ex) from 2007 to 2022 for each age from 18 to 100 downloaded from the ONS statistics for England and Wales, 
and already calculated the number of customers as well as forecasted from 2007 till 2030 years for age 18 to 106, 
Where:
- mx is the central rate of mortality, defined as the number of deaths at age x last birthday in the year to which the single year life table 
relates divided by the estimated population at that age over the same year.
- qx is the mortality rate between age x and (x +1), that is the probability that a person aged x exactly will die before reaching age (x +1).
- lx is the number of survivors to exact age x of 100000 live births of the same sex who are assumed to be subject throughout their lives to the 
mortality rates experienced in the year to which the single year life table relates.
- dx is the number dying between exact age x and (x +1) described similarly to lx, that is dx=lx-lx+1.
- ex is the average period expectation of life at exact age x, that is the average number of years that those aged x exact will live thereafter
based on the mortality rates experienced in the year to which the single year life table relates.

# Calculate the number of living customers from 2007 to 2040 for ages 18 to 106, using the life tables, cohort survival tables, and the living customers tables:

1. **Life Tables Construction**
   For each year (2007 to 2040) and each age (18 to 106), construct life tables. 
   Given already have mx, qx, lx, dx, and ex values from 2007 to 2022:

   a. **Estimate Mortality Rates (qx) for Future Years (2023 to 2040)**
      Given mx for the years 2023 to 2040, calculate qx using the formula:
      $$
      q_x = \frac{m_x}{1 + 0.5 \times m_x}
      $$
      * If mx is not available for future years, may need to project or assume future qx values based on historical trends or a forecasting model.

   b. **Construct lx (Survivors)**
      Start with an initial value for $l_{18}$ (typically 100,000 for a cohort of live births).
      For each subsequent age and year:
      $$
      l_{x+1} = l_x \times (1 - q_x)
      $$
      * This step should be done for each year and age.

   c. **Calculate dx (Deaths)**
      For each age and year:
      $$
      d_x = l_x - l_{x+1}
      $$

   d. **Calculate ex (Life Expectancy)**
      Calculate the life expectancy $e_x$ using the formula:
      $$
      e_x = \frac{L_x + L_{x+1} + \ldots + L_{max\ age}}{l_x}
      $$
      where $L_x$ is the number of person-years lived between ages $x$ and $x+1$, often approximated by:
      $$
      L_x = 0.5 \times (l_x + l_{x+1})
      $$

2. **Cohort Survival Tables Construction**
   To calculate cohort survival for each cohort (group of customers born in the same year):

   a. **Determine the Initial Population for Each Cohort**
      For each year and each age, determine the initial number of customers in the cohort.

   b. **Apply Survival Rates**
      Using the qx values calculated in the life tables, determine how many customers survive each year by multiplying the cohort population by $1 - q_x$.

   c. **Project the Cohorts Over Time**
      Continue this for each subsequent year, ensuring to track how each cohort diminishes over time due to mortality.

3. **Living Customers Table Construction**
   After calculating the cohort survival tables:

   a. **Aggregate Living Customers by Year**
      For each year from 2007 to 2040, sum up the number of living customers across all cohorts and ages to get the total number of living customers in each year.

   b. **Tabulate the Results**
      Create a table where each row corresponds to a year (2007 to 2040), and each column corresponds to an age (18 to 106). The entries in the table should be the number of living customers.

4. **Example Steps for 2023**
   - **Life Table for 2023**:
     - Use the projected mx values to compute qx.
     - Calculate lx for ages 18 to 106.
     - Compute dx and ex.
   - **Cohort Survival for 2023**:
     - Start with the number of customers for each cohort at the beginning of 2023.
     - Apply the survival probabilities to estimate how many customers survive through the year.
   - **Living Customers in 2023**:
     - Sum the surviving customers from all cohorts and ages to get the total number of living customers in 2023.

5. **Repeat the Process for Each Year from 2024 to 2040**
   By following these steps for each year, you will be able to calculate the number of living customers for each year from 2007 to 2040, 
    broken down by age from 18 to 106. 
This approach combines historical data, mortality projections, and cohort survival analysis to forecast the customer population.

In [ ]:
# Calculate the number of living customers from 2007 to 2040 for ages 18 to 106:

# Assumptions: have the necessary mortality data (mx, qx, lx, dx, ex) for the years 2007 to 2022 
# and have forecasted / estimated data for subsequent years.

end_ons_year = 2022 # The last avalilable year extracted mortality statistics from ONS

# Load the life table data
file_path = 'nltew198020221.xlsx'
xls = pd.ExcelFile(file_path)


years_all = re.findall(r'\d{4}', file_path)
if len(years_all) > 1:
    final_year = int(years_all[1])
    print(f"The final year of the extracted data is: {end_ons_year}")
else:
    print("Less than two years found in the filename.")
    
# Initialise an empty list to store dataframes
dfs = []

# Load life tables from 2007 to 2022
for year in range(start_year, end_ons_year+1):
    sheet_name = str(year)
    if sheet_name in xls.sheet_names:
        df_male = pd.read_excel(xls, sheet_name=sheet_name, skiprows=range(0, 6), usecols="A:F")
        df_female = pd.read_excel(xls, sheet_name=sheet_name, skiprows=range(0, 6), usecols="H:M")
        df_male['year'] = year
        df_female['year'] = year
        df_male.columns = ['age', 'mx', 'qx', 'lx', 'dx', 'ex', 'year']
        df_female.columns = ['age', 'mx', 'qx', 'lx', 'dx', 'ex', 'year']
        df_combined = pd.merge(df_male, df_female, on=['age', 'year'], suffixes=('_male', '_female'))
        df_combined['mx'] = (df_combined['mx_male'] + df_combined['mx_female']) / 2
        df_combined['qx'] = (df_combined['qx_male'] + df_combined['qx_female']) / 2
        df_combined['lx'] = (df_combined['lx_male'] + df_combined['lx_female']) / 2
        df_combined['dx'] = (df_combined['dx_male'] + df_combined['dx_female']) / 2
        df_combined['ex'] = (df_combined['ex_male'] + df_combined['ex_female']) / 2
        df_combined = df_combined[['age', 'mx', 'qx', 'lx', 'dx', 'ex', 'year']]
        dfs.append(df_combined)

# Concatenate all dataframes into a single dataframe
final_df = pd.concat(dfs, ignore_index=True)

# Filter the data to include only ages from 18 to 106
source_life_table = final_df[(final_df['age'] >= 18) & (final_df['age'] <= 106)]
source_life_df = pd.DataFrame(source_life_table[['year', 'age', 'lx', 'qx']])



# Function to add new rows
def add_new_rows(df, start_year, end_year, start_age, end_age):
    new_rows = []
    for year in range(start_year, end_year + 1):
        for age in range(start_age, end_age + 1):
            if age > 100:  # Only add ages greater than 100
                new_rows.append(
                    {'year': year,
                     'age': age,
                     'lx': 0, 
                     'qx': 0})
    return df.append(new_rows, ignore_index=True)

# Create the initial DataFrame
df = source_life_df

# Add the new rows to the DataFrame
new_life_tbl = add_new_rows(df, start_year, year_forecast_long, 18, 106)
# Display the updated DataFrame
#print(new_life_tbl)
final_lpa_df = new_life_tbl
final_lpa_df['qx'] = new_life_tbl['qx']
#final_lpa_df   

# Initialise a dictionary to hold starting population for each age in ages_100 and year in years_ons
initial_pop_dict = {}

# Starting population for each age and year 
for year in years_forecast_long:
    #initial_population = forecast_lpa_df[forecast_lpa_df['year'] == year]['lx'].values
    initial_population = final_lpa_df.loc[(final_lpa_df['year']==year), ['year', 'age', 'lx']]
    
    # Convert yearly data to DataFrame and set multi-index
    initial_pop_dict[year] = pd.DataFrame(initial_population).set_index(['year', 'age'])
#print(initial_pop_dict)

In [ ]:
# use lpa donor actuals and forecasts for the source lx and update it

df = pd.concat(initial_pop_dict.values(), axis=0)
df.reset_index(inplace=True)
df#.iloc[:,:-10,:]

df_new_values = Donor_Estimates_Actuals.loc[:,2007:2039]
df_new_values.reset_index(inplace=True)
df_new_values = df_new_values.rename(columns={'index': 'age'})

# Melt the new values DataFrame
df_new_values = df_new_values.melt(id_vars=['age'], var_name='year', value_name='lx')
df_new_values# = df_new_values[['year', 'age', 'lx']]
#df_new_values['year'] = df_new_values['year'].str.extract('(\d+)').astype(int)
df_new_values

# Merge the original DataFrame with the new values DataFrame
df = df.merge(df_new_values, on=['year', 'age'], how='right')
df['lx'] = df['lx_y']
df.drop(columns=['lx_x', 'lx_y'], inplace=True)

initial_donor_dict = {}

for year in years_forecast_long:
    df.loc[(df['year']==year), ['age']] = age_range
    #df.loc[(df['year']==year), ['year', 'age', 'lx']].set_index(['year', 'age'])
    initial_donor_dict[year] = df.loc[(
        df['year']==year), ['year', 'age', 'lx']].set_index(['year', 'age'])
#df.set_index(['year', 'age'])
#print(df.loc[(df['year']==year), ['year', 'age', 'lx']])
#initial_donor_dict

In [ ]:
#initial_donor_dict
#donor_df = pd.concat(initial_donor_dict.values(), axis=0) 
donor_df = df.set_index(['year', 'age'])
#donor_df.loc[(year, range(18,107)), 'lx']
#donor_df.loc[(year, age), 'lx']

liftbl = pd.DataFrame()

for year in years_forecast_long: #range(2007, end_ons_year + 1):
    for age in ages:
        # fill the qx values based on average of 4 previous ages for that age
        #qx_values = np.array([source_life_df.loc[(source_life_df['year'] == 2007) & (source_life_df['age'] > age-6) & (source_life_df['age'] < age), 'qx'].mean()])
        qx_cur = source_life_df.loc[(
            source_life_df['year'] == year) & (
            source_life_df['age'] > age-5) & (
            source_life_df['age'] < age), 'qx'].mean() if year <= end_ons_year else np.nan
        #lx_cur = source_life_df.loc[(source_life_df['year'] == year) & (source_life_df['age'] > age-6) & (source_life_df['age'] < age), 'lx'].mean() #(source_life_df['age'] == age), 'lx'].values[0]
        #lx_cur = donor_df.loc[(year, age), 'lx']
        lx_cur = donor_df.loc[(year, age), 'lx']
        # If the result is valid (lx_next is not None), append it as a new row to source_life_df
        if qx_cur is not None:
            # Create a new row with the required values, adjust as necessary
            new_row = pd.DataFrame({
                'year': [year],  # next year, 2024 in this case
                'age': [age],    # next age
                'lx': [lx_cur],      # calculated value
                'qx': [qx_cur]      # calculated value
            })
            

            # Append the new row to source_life_df
            liftbl = pd.concat([liftbl, new_row], ignore_index=True)
source_life_df = liftbl.fillna(0)
#source_life_df

# for all years (2023 - 2039) and all age (18 - 106) calculate qx if we have qx (qx_prev) for all ages before and including 2022 (2007 - 2022) and we dont have qx (qx_cur) from 2023 onwards 
and should be calculated based on lx of the current year (lx_cur) and previous year lx (lx_prev) by using formula:  lx_cur = lx_prev * (1 - qx_cur), by using the attached data containing qx and lx, 
where qx=is the mortality rate between age x and (x +1), that is the probability that a person aged x exactly will die before reaching age (x +1). 
lx=is the number of survivors to exact age x of 100,000 live births of the same sex who are assumed to be subject throughout their lives to the mortality rates experienced in the year to which the single year life table relates. 

In [ ]:

# population_forecast_df: Contains the forecasted population for each age group in 2023 and 2024.
population_forecast = pd.read_csv('population_forecast.csv')
# Convert the DataFrame to the desired format
df_melted = population_forecast.melt(id_vars=['age'], var_name='year', value_name='population')
population_forecast_df = pd.DataFrame(df_melted)

pop_f = population_forecast_df#.set_index(['age', 'year'])
pop_f['year'] = pd.to_numeric(pop_f['year'], errors='coerce')
pop_f['age'] = pd.to_numeric(pop_f['age'], errors='coerce')

# Sort the data by year and age for proper processing
pop_f.sort_values(by=['year', 'age'], inplace=True)

pop_f#pop_f[(pop_f['age'] == age) & (pop_f['year'] == year)]['population']

In [ ]:
source_life = source_life_df

years_range = list(range(2023, 2040))  # Extend years range for future years
ager = list(range(18, 107))  # Age range from 18 to 106 (inclusive)

# Function to calculate qx based on lx (survival) values
def calculate_qx_from_lx(source_life, year, age):
    """
    Calculate qx using lx values: qx = 1 - (lx+1 / lx)
    Ensure qx is non-negative and that lx decreases with age.
    """
    lx = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age), 'lx'].values
    lx_next = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age + 1), 'lx'].values
    
    if len(lx) > 0 and len(lx_next) > 0 and lx[0] > 0:  # Ensure lx is valid and non-zero
        qx = 1 - (lx_next[0] / lx[0])
        # Ensure qx is not negative
        return max(0, qx)  # Cap qx at a minimum of 0
    else:
        lx_prev = source_life.loc[(source_life['year'] == year-1) & (source_life['age'] == age), 'lx'].values
        lx_cur = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age + 1), 'lx'].values if age < 106 else lx
        qx = 1 - (lx_cur[0] / lx_prev[0])
        return max(0, qx)  # Cap qx at a minimum of 0
    return None  # Return None if lx is missing or invalid

# Function to get the mean of the last valid qx values (up to 4)
def get_previous_qx_mean(source_life, year, age):
    """
    Get the mean of the last valid qx values for a given age and year (up to 4).
    """
    qx_values = []
    for i in range(1, 5):  # Look back at the previous 4 age groups
        qx_prev = source_life.loc[(source_life['year'] == year-1) & 
                                  (source_life['age'] == age - i), 'qx'].values
        if len(qx_prev) > 0 and qx_prev[0] > 0:  # Ensure qx is valid and non-zero
            qx_values.append(qx_prev[0])
    
    if len(qx_values) >= 2:  # Use at least 2 valid values
        return np.mean(qx_values)
    else:
        return source_life.loc[(
            source_life['year'] == year-1) & (
            source_life['age'] == age - 1
        ), 'qx'].values[0] if age > 18 else source_life.loc[(
            source_life['year'] == year-1) & (
            source_life['age'] == age), 'qx'].values[0]
    return None  # Return None if there aren't enough valid values

# Function to get the mean of the next valid qx values (up to 4)
def get_next_qx_mean(source_life, year, age):
    """
    Get the mean of the next valid qx values for a given age and year (up to 4).
    """
    qx_values = []
    for i in range(1, 5):  # Look forward at the next 4 age groups
        qx_next = source_life.loc[(source_life['year'] == year) & 
                                  (source_life['age'] == age + i), 'qx'].values
        if len(qx_next) > 0 and qx_next[0] > 0:  # Ensure qx is valid and non-zero
            qx_values.append(qx_next[0])
    
    if len(qx_values) >= 2:  # Use at least 2 valid values
        return np.mean(qx_values)
    else:
        return source_life.loc[(
            source_life['year'] == year-1) & (
            source_life['age'] == age), 'qx'].values[0]
    return None  # Return None if there aren't enough valid values

# Function to get qx for previous years, considering missing values
def get_previous_qx(source_life, year, age, lookback_years=10):
    """
    Retrieves the most recent non-zero qx value for a given age
    within a specified lookback period.
    """
    for i in range(1, lookback_years+1):
        qx_prev = source_life.loc[(source_life['year'] == year - i) & (
            source_life['age'] == age), 'qx'].values
        if len(qx_prev) > 0 and qx_prev[0] > 0:
            return np.mean(qx_prev) #qx_prev[0]
    return None  # No valid qx found in the lookback period


# Iterate over each future year from 2023 to 2039 and replace invalid qx values
for year in years_range:
    for age in ages:
        
        # Get current qx value
        qx_cur = source_life.loc[(source_life['year'] == year) & (
            source_life['age'] == age), 'qx'].values[0]
        
        qx_cur = calculate_qx_from_lx(source_life, year, age)

        source_life.loc[(source_life['year'] == year) & (
            source_life['age'] == age), 'qx'] = qx_cur  
    
        # Replace if qx is negative or zero
        if qx_cur <= 0:
            if age < 21:
                
                qx_prev = get_previous_qx(source_life, year, age, lookback_years=5)
                if qx_prev is not None:
                    qx_cur = qx_prev
                else:
                    qx_cur = source_life.loc[(source_life['year'] == year-1) & (
                        source_life['age'] == age), 'qx'].values[0]
                    
            elif (age > 21) & (age < 101):  # If age < 102, try to use the next valid values
                qx_mean_next = get_next_qx_mean(source_life, year, age)
                if qx_mean_next is not None:
                    qx_cur = qx_mean_next
                
            elif (age >= 101) & (age < 107):  # If age > 21, try to use the previous valid values
                qx_mean_prev = get_previous_qx_mean(source_life, year, age)
                if qx_mean_prev is not None:
                    qx_cur = qx_mean_prev
        
            source_life.loc[(source_life['year'] == year) & (
                source_life['age'] == age), 'qx'] = qx_cur   
        
# Save the updated DataFrame to a new file
output_file_path = 'updated_mortality_statistics.csv'
source_life.to_csv(output_file_path, index=False)

print(source_life)


To address the issue of having too many zeros in `qx` when calculating for future years (2023 and beyond), we can use a combination of the **survivors (`lx`)** data and the logic from the first code in my earlier response. This method will ensure that `qx` is calculated based on survival data, which can give a more reliable estimate for `qx`.

### Strategy:
1. **Use the `lx` (number of survivors) values**:  
   If we have `lx` for all years before and after 2023, we can calculate `qx` using the relationship between survivors at age `x` and survivors at age `x+1`:
   \[
   qx = 1 - \frac{l_{x+1}}{l_x}
   \]
   This formula will provide a more accurate estimate of the mortality rate based on the observed survival rates.

2. **Fallback to the previous method**:  
   If `lx` is missing or zero, we can still fallback on the averaging method from the previous years' `qx` values.

### Improved Code:

```python
years_range = list(range(2023, 2040))  # Extend years range for future years
ager = list(range(18, 21))  # Example age range (can extend this as needed)

# Function to calculate qx based on lx (survival) values
def calculate_qx_from_lx(source_life, year, age):
    """
    Calculate qx using lx values: qx = 1 - (lx+1 / lx)
    """
    lx = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age), 'lx'].values
    lx_next = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age + 1), 'lx'].values
    
    if len(lx) > 0 and len(lx_next) > 0 and lx[0] > 0:  # Ensure lx is valid and non-zero
        return 1 - (lx_next[0] / lx[0])
    return None

# Function to get qx for previous years, considering missing values
def get_previous_qx(source_life, year, age, lookback_years=3):
    """
    Retrieves the most recent non-zero qx value for a given age
    within a specified lookback period.
    """
    for i in range(1, lookback_years+1):
        qx_prev = source_life.loc[(source_life['year'] == year - i) & (
            source_life['age'] == age), 'qx'].values
        if len(qx_prev) > 0 and qx_prev[0] > 0:
            return qx_prev[0]
    return None  # No valid qx found in the lookback period

# Iterate over each future year from 2023 to 2039 and calculate qx if missing
for year in years_range:
    # For each age group from 18 to 20 (you can extend this as needed)
    for age in ager:
        # Try to calculate qx using lx (survivor data)
        qx_cur = calculate_qx_from_lx(source_life, year, age)
        
        # If lx calculation fails, fallback to previous qx method
        if qx_cur is None:
            qx_prev = get_previous_qx(source_life, year, age, lookback_years=3)
            qx_prev2 = get_previous_qx(source_life, year-1, age, lookback_years=3)
            
            # If both previous years have valid qx, calculate average
            if qx_prev is not None and qx_prev2 is not None:
                qx_cur = (qx_prev + qx_prev2) / 2
            elif qx_prev is not None:  # If only one previous year has qx, use that
                qx_cur = qx_prev
        
        # Only update qx if the current value is zero or missing
        if qx_cur is not None:
            source_life.loc[(source_life['year'] == year) & 
                            (source_life['age'] == age) & 
                            (source_life['qx'] == 0), 'qx'] = qx_cur
```

### Key Changes:
1. **Calculate `qx` from `lx`**:  
   The function `calculate_qx_from_lx()` computes `qx` based on the survival rates (`lx`). This formula:
   \[
   qx = 1 - \frac{l_{x+1}}{l_x}
   \]
   uses the number of survivors at age `x+1` and age `x` to calculate the mortality rate. This is the primary method for determining `qx` in this improved code.

2. **Fallback to Previous Years' `qx`**:  
   If the `lx` values are missing or zero (due to incomplete data), the code falls back to the previous method of using an average of the past years' `qx` values.

3. **Handling Missing Data**:  
   The code checks whether the `lx` values exist and are non-zero before performing any calculations, ensuring no division by zero or other issues.

### How It Works:
1. **Survival-Based Calculation (`lx`)**:  
   For each future year and age group, the code first attempts to calculate `qx` using survival data. If successful, this ensures a reliable mortality rate based on real survival probabilities.

2. **Fallback Mechanism**:  
   If the `lx` data is unavailable or incomplete, it looks back at previous years' `qx` values to calculate an average or use the last known valid `qx`.

3. **Replacing Zeros**:  
   The code updates `qx` values for years where `qx` is initially 0 or missing, ensuring the calculated values are only applied where necessary.

---

### Output:
After running this code, `source_life` will have the `qx` values calculated for 2023 and beyond. The use of survival (`lx`) data should minimize the number of zeros in the resulting `qx` values, producing more realistic estimates for future years.

---

### Example:
If you have `lx` data for the year 2023 and ages 18, 19, and 20, the formula will calculate the probability of dying between age `x` and `x+1` using the survivor data:
- **For age 18** in 2023, `qx` will be calculated as:
  \[
  qx_{18} = 1 - \frac{l_{19}}{l_{18}}
  \]
- **For age 19**, `qx` will similarly be calculated using survivors at age 19 and 20.

If the `lx` data is missing for 2023, it will fallback to the previous years' `qx` values and calculate the average.

Let me know if this addresses your issue or if you'd like further refinements!

The issue of negative `qx` values arises because the formula for calculating `qx` using `lx` (survival values) can lead to negative results when the number of survivors at age `x+1` is greater than the number of survivors at age `x`. This scenario would imply a growth in the number of survivors, which is not realistic in the context of mortality rates.

### Fixing the Issue:
1. **Ensure non-negative `qx` values**:  
   We need to ensure that `qx` values are never negative. The formula:
   \[
   qx = 1 - \frac{l_{x+1}}{l_x}
   \]
   should be adjusted so that `qx` is capped at a minimum of 0. This can be done using `max(0, qx)` to avoid negative values.

2. **Check `lx` Values**:  
   The code will also verify that `lx` values are properly ordered (i.e., `l_{x+1} <= l_x`) before calculating `qx`. If `lx` values suggest an increase in survivors, the calculation will return `qx = 0`.

### Updated Code:

```python
years_range = list(range(2023, 2040))  # Extend years range for future years
ager = list(range(18, 21))  # Example age range (can extend this as needed)

# Function to calculate qx based on lx (survival) values
def calculate_qx_from_lx(source_life, year, age):
    """
    Calculate qx using lx values: qx = 1 - (lx+1 / lx)
    Ensure qx is non-negative and that lx decreases with age.
    """
    lx = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age), 'lx'].values
    lx_next = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age + 1), 'lx'].values
    
    if len(lx) > 0 and len(lx_next) > 0 and lx[0] > 0:  # Ensure lx is valid and non-zero
        qx = 1 - (lx_next[0] / lx[0])
        # Ensure qx is not negative
        return max(0, qx)  # Cap qx at a minimum of 0
    return None  # Return None if lx is missing or invalid

# Function to get qx for previous years, considering missing values
def get_previous_qx(source_life, year, age, lookback_years=3):
    """
    Retrieves the most recent non-zero qx value for a given age
    within a specified lookback period.
    """
    for i in range(1, lookback_years+1):
        qx_prev = source_life.loc[(source_life['year'] == year - i) & (
            source_life['age'] == age), 'qx'].values
        if len(qx_prev) > 0 and qx_prev[0] > 0:
            return qx_prev[0]
    return None  # No valid qx found in the lookback period

# Iterate over each future year from 2023 to 2039 and calculate qx if missing
for year in years_range:
    # For each age group from 18 to 20 (you can extend this as needed)
    for age in ager:
        # Try to calculate qx using lx (survivor data)
        qx_cur = calculate_qx_from_lx(source_life, year, age)
        
        # If lx calculation fails, fallback to previous qx method
        if qx_cur is None:
            qx_prev = get_previous_qx(source_life, year, age, lookback_years=3)
            qx_prev2 = get_previous_qx(source_life, year-1, age, lookback_years=3)
            
            # If both previous years have valid qx, calculate average
            if qx_prev is not None and qx_prev2 is not None:
                qx_cur = (qx_prev + qx_prev2) / 2
            elif qx_prev is not None:  # If only one previous year has qx, use that
                qx_cur = qx_prev
        
        # Only update qx if the current value is zero or missing
        if qx_cur is not None:
            source_life.loc[(source_life['year'] == year) & 
                            (source_life['age'] == age) & 
                            (source_life['qx'] == 0), 'qx'] = qx_cur
```

### Key Changes:
1. **`max(0, qx)`**:  
   This ensures that `qx` values never become negative. If the calculation yields a negative value, it is set to 0, indicating no mortality (in cases where the survivor count increases or remains the same).

2. **Check Survivor (`lx`) Consistency**:  
   The function now checks that `l_{x+1} <= l_x` before calculating `qx`. If this condition is violated (which would imply an increase in the number of survivors, which is unrealistic in a mortality table), it returns a `qx` of 0.

3. **Fallback Mechanism for Missing `lx` Data**:  
   If the `lx` data is missing or incomplete, the code falls back to the previous method of using historical `qx` values from previous years to estimate the current year's `qx`.

---

### How It Works:
- **Survivor-Based `qx` Calculation**:  
  For each year and age, the function calculates `qx` based on the number of survivors (`lx`). It ensures that `qx` remains non-negative and that the number of survivors decreases as age increases.

- **Fallback to Historical `qx`**:  
  If `lx` values are missing or unreliable, the code looks at the previous years' `qx` values and calculates the current `qx` as an average of the last two valid values.

- **Zero Replacement**:  
  The calculated `qx` values are only applied to rows where `qx` is initially 0, ensuring that only missing or zero values are replaced.

---

### Example:
Let’s assume the following `lx` values for a given year and age group:

| Age | Year | lx  | qx  |
|-----|------|-----|-----|
| 18  | 2023 | 1000 | 0   |
| 19  | 2023 | 950  | 0   |
| 20  | 2023 | 900  | 0   |

For **age 18**, the `qx` value would be calculated as:
\[
qx = 1 - \frac{950}{1000} = 1 - 0.95 = 0.05
\]
This result is non-negative and makes sense, as the number of survivors decreases from 1000 to 950 between ages 18 and 19.

If the `lx` values showed an increase in survivors (e.g., 1000 to 1050), the formula would yield a negative `qx`, but `max(0, qx)` ensures that `qx` is set to 0 instead.

---

Let me know if this resolves the issue of negative `qx` values or if you need further modifications!



## Approach to Calculate Mortality Rates \( q_x \) for 2009

To calculate mortality rates \( q_x \) for 2009 based on the given data, follow these steps:

### **General Approach**

1. **Estimate Deaths for 2009**:  
   The estimated number of deaths \( d_x \) in 2009 can be calculated using the following formula:
   \[
   d_x = \text{Population}_{2009} \times q_x
   \]
   Since \( q_x \) for 2009 is unknown, we will estimate it using the data from 2007 and 2008.

2. **Calculate the Average Mortality Rate \( q_x \) for Previous Years**:  
   For each age, calculate an average \( q_x \) for 2007 and 2008 to use as a basis for estimating 2009 values:
   \[
   q_x^{\text{avg}} = \frac{q_x^{2007} + q_x^{2008}}{2}
   \]
   
3. **Use the Formula to Calculate New \( q_x \) for 2009**:  
   Using the survivor data \( l_x \) and estimated deaths \( d_x \), the mortality rate \( q_x \) for 2009 can be calculated as:
   \[
   q_x = \frac{d_x}{l_x}
   \]
   where \( l_x \) is the number of survivors at age \( x \).

---

### **Step-by-Step Calculation**

1. **Average Mortality Rate for Previous Years**  
   Calculate the average mortality rate \( q_x \) for 2007 and 2008 for each age group \( x \).

   For example, for age 18:
   \[
   q_{18}^{\text{avg}} = \frac{0.000000 + 0.000000}{2} = 0.000000
   \]
   
   You can repeat this calculation for other age groups.

2. **Estimate Deaths for 2009**  
   For age 18 in 2009:
   \[
   d_{18}^{2009} = \text{Population}_{2009, 18} \times q_{18}^{\text{avg}} = 734658 \times 0.000000 = 0
   \]
   
   Repeat similar calculations for other age groups.

3. **Calculate \( q_x \) for 2009**  
   For age 18, since there are no deaths, the mortality rate remains 0. For other ages where deaths occur, use:
   \[
   q_x^{2009} = \frac{d_x^{2009}}{l_x^{2009}}
   \]


### **Explanation of the Code**:

1. **Input Data**:  
   The code accepts data for survivors (`lx`) and mortality rates (`qx`) for 2007, 2008, and 2009, and population data for these years.

2. **Average Mortality Rate**:  
   It calculates the average mortality rate (`qx_avg`) for 2007 and 2008.

3. **Deaths Calculation**:  
   For each age group in 2009, the estimated number of deaths is calculated by multiplying the population for 2009 by the average mortality rate (`qx_avg`).

4. **New Mortality Rate Calculation**:  
   The mortality rate for 2009 (`qx_2009`) is calculated by dividing the estimated number of deaths by the number of survivors (`lx`).

5. **Handling Missing Values**:  
   If any missing or zero values appear in the data, the mortality rates are filled with 0 where necessary.

This code will produce the mortality rates \( q_x \) for 2009 based on the input data. Let me know if you'd like further clarification!

To address the issue where some `qx` values are either negative or zero, we can implement a strategy that replaces these invalid values with the mean of the next or previous four valid `qx` values. Here's how the replacement will work:

### Replacement Strategy:
1. **For ages > 21**:  
   Replace negative or zero `qx` values with the mean of the last four valid `qx` values from the **current and previous years**.
   
2. **For ages < 102**:  
   Replace negative or zero `qx` values with the mean of the next four valid `qx` values from the **current and previous years**.

### Approach:
1. **Identify invalid `qx` values**:  
   We’ll detect `qx` values that are either negative or zero.
   
2. **Calculate mean of neighboring values**:  
   For ages greater than 21, we look for the previous four valid `qx` values. For ages below 102, we look for the next four valid `qx` values.

3. **Replace invalid values**:  
   If enough valid values are found, we replace the invalid `qx` with the mean of these values.

### Updated Code:

```python
import numpy as np

years_range = list(range(2023, 2040))  # Extend years range for future years
ager = list(range(18, 103))  # Age range from 18 to 102 (inclusive)

# Function to calculate qx based on lx (survival) values
def calculate_qx_from_lx(source_life, year, age):
    """
    Calculate qx using lx values: qx = 1 - (lx+1 / lx)
    Ensure qx is non-negative and that lx decreases with age.
    """
    lx = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age), 'lx'].values
    lx_next = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age + 1), 'lx'].values
    
    if len(lx) > 0 and len(lx_next) > 0 and lx[0] > 0:  # Ensure lx is valid and non-zero
        qx = 1 - (lx_next[0] / lx[0])
        # Ensure qx is not negative
        return max(0, qx)  # Cap qx at a minimum of 0
    return None  # Return None if lx is missing or invalid

# Function to get the mean of the last four valid qx values
def get_previous_qx_mean(source_life, year, age):
    """
    Get the mean of the last four valid qx values for a given age and year.
    """
    qx_values = []
    for i in range(4):  # Look back at the previous 4 age groups
        qx_prev = source_life.loc[(source_life['year'] == year) & 
                                  (source_life['age'] == age - i), 'qx'].values
        if len(qx_prev) > 0 and qx_prev[0] > 0:  # Ensure qx is valid and non-zero
            qx_values.append(qx_prev[0])
    
    if len(qx_values) == 4:  # If we have 4 valid qx values, return their mean
        return np.mean(qx_values)
    return None  # Return None if there aren't enough valid values

# Function to get the mean of the next four valid qx values
def get_next_qx_mean(source_life, year, age):
    """
    Get the mean of the next four valid qx values for a given age and year.
    """
    qx_values = []
    for i in range(1, 5):  # Look forward at the next 4 age groups
        qx_next = source_life.loc[(source_life['year'] == year) & 
                                  (source_life['age'] == age + i), 'qx'].values
        if len(qx_next) > 0 and qx_next[0] > 0:  # Ensure qx is valid and non-zero
            qx_values.append(qx_next[0])
    
    if len(qx_values) == 4:  # If we have 4 valid qx values, return their mean
        return np.mean(qx_values)
    return None  # Return None if there aren't enough valid values

# Iterate over each future year from 2023 to 2039 and replace invalid qx values
for year in years_range:
    for age in ager:
        # Get current qx value
        qx_cur = source_life.loc[(source_life['year'] == year) & (source_life['age'] == age), 'qx'].values
        
        if len(qx_cur) > 0:
            qx_cur = qx_cur[0]
            
            # Replace if qx is negative or zero
            if qx_cur <= 0:
                if age > 21:  # If age > 21, use the previous 4 valid values
                    qx_mean_prev = get_previous_qx_mean(source_life, year, age)
                    if qx_mean_prev is not None:
                        source_life.loc[(source_life['year'] == year) & 
                                        (source_life['age'] == age), 'qx'] = qx_mean_prev
                
                elif age < 102:  # If age < 102, use the next 4 valid values
                    qx_mean_next = get_next_qx_mean(source_life, year, age)
                    if qx_mean_next is not None:
                        source_life.loc[(source_life['year'] == year) & 
                                        (source_life['age'] == age), 'qx'] = qx_mean_next
```

### Explanation:
1. **Negative and Zero `qx` Detection**:
   - The code checks if the current `qx` value is zero or negative, which are invalid.

2. **Replacement for Ages > 21**:
   - The `get_previous_qx_mean()` function looks back at the previous four valid `qx` values for the current year and age group, and it calculates their mean.
   - If four valid values are found, the invalid `qx` is replaced with the mean.

3. **Replacement for Ages < 102**:
   - The `get_next_qx_mean()` function looks forward at the next four valid `qx` values and computes their mean.
   - If four valid values are available, the invalid `qx` is replaced.

4. **Edge Cases**:
   - If fewer than four valid `qx` values are found, the function returns `None`, and the replacement is skipped.
   - The function ensures that values are only replaced when valid neighboring data is available.

---

### Example:
If you have the following `qx` values for a given year and age:

| Age | Year | qx  |
|-----|------|-----|
| 18  | 2023 | 0   |
| 19  | 2023 | 0.005 |
| 20  | 2023 | 0.006 |
| 21  | 2023 | -0.002 |
| 22  | 2023 | 0.007 |
| 23  | 2023 | 0   |

- **For age 18**, since `qx` is zero, the code will look for the next four values (`qx` for ages 19, 20, 21, and 22), calculate their mean, and replace `qx` for age 18 with that mean.
  
- **For age 21**, since `qx` is negative, it will look back at the previous four values and calculate their mean to replace the invalid value.

---

This approach should resolve issues with both negative and zero `qx` values, ensuring that all `qx` values are based on the neighboring valid data. Let me know if you need further adjustments!

In [ ]:
# LPA SURVIVAL TABLES

life_tbl_src = source_life
#year_life_tbl = 2007
first_column = []
lpa_survival_tables_all = pd.DataFrame()
lpa_survival_tables_all_vals = pd.DataFrame()
lpa_survival_tables_all_aged = pd.DataFrame()
total_lpa_surviving_donors_df = pd.DataFrame()
#years = list(range(start_year, end_forecast_year + 1))

# Initialise a dictionary to hold lpa_surviving_donors data for each year
#lpa_surviving_donors_dict = {}
#lpa_surviving_donors_dict_changed = {}
lpa_surviving_donors_all_dict = {}
total_lpa_surviving_donors_dict = {}
total_life_tbl = pd.DataFrame()
surviving_donors_lpa = {} #pd.DataFrame()
summed_lx_dict_by_year = {}

# Define the age groups
age_groups = {"age_groups": age_range}
age_groups_df = pd.DataFrame(age_groups)


lpa_life_tbl_dict = {}
qx_lpa_surviving_donors_all = pd.DataFrame() #{}


# Extend life tables for all years:
for year_life_tbl in years_forecast_long:  #list(range(2007, 2010)): #
    
     
    lpa_surviving_donors = pd.DataFrame()
    lpa_surviving_donors_df = pd.DataFrame()
    lpa_surviving_donors_tbl = pd.DataFrame()
    lpa_surviving_donors_all_df = pd.DataFrame()
    lpa_surviving_donors_dict_changed = {}
    lpa_surviving_donors_dict = {}
    qx_lpa_surviving_donors_dict = {}
    qx_lpa_surviving_donors_df = pd.DataFrame() 
    qx_lpa_surviving_donors = pd.DataFrame()
    qx_lpa_surviving_donors_tbl = pd.DataFrame()
    qx_lpa_surviving_donors_all_df = pd.DataFrame()
    qx_lpa_surviving_donors_dict_changed = {}
    qx_lpa_surviving_donors_dict = {}
    
    # Years from the year_life_tbl to end_forecast_year
    years_life_tbls = list(range(year_life_tbl, year_forecast_long + 1))
    
    # Iterate over each year for which life table needs to be generated
    for year in years_life_tbls: 
        
        yearly_data = []
        
        for age in ages: 
            
            qx_current_vals = life_tbl_src.loc[(life_tbl_src['year'] == year) & (
                life_tbl_src['age'] == age), 'qx'].values
            
            lx_current_vals = life_tbl_src.loc[(life_tbl_src['year'] == year) & (
                life_tbl_src['age'] == age), 'lx'].values
            
            # Calculate population for each age group, shifting cohorts forward each yea
            if (year == year_life_tbl):  # Starting from the initial year
                
                # Calculate / Update the survivors (lx) based on the lpa donor data

                if (age - (year - year_life_tbl) >= start_age): # for shifting for the next year and next age (age+1) 

                    qx = qx_current_vals[0] if len(qx_current_vals) > 0 else np.nan
                    
                    # Get lx for the current age and year  
                    lx_current = lx_current_vals[0] if not np.isnan(lx_current_vals) else np.nan 
                    
                    lx = lx_current# * (1 - qx) #if np.isnan(lx_current_vals) else 0 #np.nan
                    
                    #qx=life_tbl_src.loc[(life_tbl_src['year'] == year) & (life_tbl_src['age'] == age),'qx'].values[0]
                    #lx = life_tbl_src.loc[(life_tbl_src['year'] == year) & (life_tbl_src['age'] == age),'lx'].values[0]
                    #print(f"NON-ZEROS: year:{year} year_life_tbl:{year_life_tbl}, age:{age}, qx:{qx}, lx:{lx}")
                    
                    
                else:
                    lx = np.nan
                    qx = np.nan
                    #print(f"ZEROS: year:{year} year_life_tbl:{year_life_tbl}, age:{age}, qx:{qx}, lx:{lx}")
                            
            else: 
                # Access the data for the previous year (to calculate forward cohorts) 
                if (age - (year - year_life_tbl) >= start_age): # for shifting for the next year and next age (age+1) 
                    
                    #qx_prev = life_tbl_src.loc[(life_tbl_src['year'] == year-1) & (
                    #   life_tbl_src['age'] == age),'qx'].values[0] 
                    qx_cur = life_tbl_src.loc[(life_tbl_src['year'] == year) & (
                        life_tbl_src['age'] == age),'qx'].values[0] 
                    
                    #lx_prev = lpa_surviving_donors.loc[(lpa_surviving_donors['year'] == year-1) & (
                    #    lpa_surviving_donors['age'] == age-1)].values[0] 
                    lx_prev = lpa_surviving_donors_df.loc[(year-1, age-1), 'lx']
                    #lpa_surviving_donors_df.loc[(year-1,  [age-4,age-3,age-2,age-1]), 'lx']
                    #lx_cur = lpa_surviving_donors_dict_changed[year_life_tbl].iloc[age-start_age,0] if age < end_age else lpa_surviving_donors_dict_changed[year_life_tbl].iloc[age-start_age,0]
#                     lx_cur = life_tbl_src.loc[(
#                         life_tbl_src['year'] == year_life_tbl) & (
#                         life_tbl_src['age'] == age-1),'lx'].values[0] if age < end_age else life_tbl_src.loc[(
#                         life_tbl_src['year'] == year_life_tbl) & (
#                         life_tbl_src['age'] == age),'lx'].values[0]
                    
                    #if (lx_cur > 0) & (qx_prev > 0): # & (age <= 100):# & (age < 106): #(len(qx_prev) > 0)

                    qx = qx_cur

                    #ex_qx = life_tbl_src.loc[(life_tbl_src['year'] == year) & (life_tbl_src['age'] == age),'qx'].values # qx_lpa_surviving_donors[year-1][age-start_age]
                    #print(f"year:{year} year_life_tbl:{year_life_tbl}, age:{age}, qx:{qx}, lx_cur:{lx_cur}, lx:{lx_cur * (1 - qx)}")
                    
                    
                    # Calculate current year's lx based on previous year's lx and previous qx
#                     lx = lx_cur * (1 - qx) if not np.isnan(qx) else np.nan
                    lx = lx_prev * (1 - qx) if not np.isnan(qx) else np.nan
                                                       
                else:
                    lx = np.nan
                    qx = np.nan

                
            # Append the data for the current age to the yearly data list
            yearly_data.append({'year': year, 'age': age, 'lx': lx, 'qx': qx})

        # Convert yearly data to a DataFrame and set multi-index for easy access
        lpa_surviving_donors_df = pd.DataFrame(yearly_data).set_index(['year', 'age'])
        #lpa_surviving_donors_inx = lpa_surviving_donors_df
        #lpa_surviving_donors_tbl = lpa_surviving_donors_dict[year]
        lpa_surviving_donors_dict[year] = lpa_surviving_donors_df[['lx']]
        qx_lpa_surviving_donors_dict[year] = lpa_surviving_donors_df[['qx']]
        #lpa_surviving_donors_dict[year].to_csv(f"lpa_surviving_donors_{year}.csv", index=False)
        #print(f"lpa_surviving_donors_dict({year}): {lpa_surviving_donors_dict[year]}") 
         
        lpa_surviving_donors = lpa_surviving_donors_df[['lx']]
        lpa_surviving_donors = lpa_surviving_donors.reset_index() #pd.DataFrame(lpa_surviving_donors).reset_index()
        lpa_surviving_donors = lpa_surviving_donors.rename(columns={'lx':year})  #, inplace=True)
        #print(f"year: {year} - lpa_surviving_donors: {lpa_surviving_donors}")        
        qx_lpa_surviving_donors = lpa_surviving_donors_df[['qx']]
        qx_lpa_surviving_donors = qx_lpa_surviving_donors.reset_index() #pd.DataFrame(lpa_surviving_donors).reset_index()
        qx_lpa_surviving_donors = qx_lpa_surviving_donors.rename(columns={'qx':year})  #, inplace=True)
        
        lpa_surviving_donors_tbl = lpa_surviving_donors #lpa_surviving_donors[['age', year]] 
        qx_lpa_surviving_donors_tbl = qx_lpa_surviving_donors #lpa_surviving_donors[['age', year]] 
        #lpa_surviving_donors_tbl = lpa_surviving_donors.assign(Age=pd.Series(range(18,107)).values)
        lpa_surviving_donors_all_df = lpa_surviving_donors_tbl.set_index(['age'])
        #print(f"lpa_surviving_donors_all_df: {lpa_surviving_donors_all_df}")
        qx_lpa_surviving_donors_all_df = qx_lpa_surviving_donors_tbl.set_index(['age'])
        lpa_surviving_donors_dict_changed[year] = lpa_surviving_donors_all_df[[year]]
        qx_lpa_surviving_donors_dict_changed[year] = qx_lpa_surviving_donors_all_df[[year]]
        #print(f"year: {year} - lpa_surviving_donors_dict_changed: {lpa_surviving_donors_dict_changed[year]}") 
    lpa_life_tbl_dict[year_life_tbl] = lpa_surviving_donors_df
    #qx_lpa_life_tbl_dict[year_life_tbl] = lpa_surviving_donors_df
    # Concatenate all yearly data into a single DataFrame
    lpa_surviving_donors_all = pd.concat(lpa_surviving_donors_dict_changed.values(), axis=1)   
    
    qx_lpa_surviving_donors_all = pd.concat(qx_lpa_surviving_donors_dict_changed.values(), axis=1) 
     
    #print(f"lpa_surviving_donors_all from lpa_surviving_donors_dict_changed: {lpa_surviving_donors_all}")
    lpa_surviving_donors_all_vals = lpa_surviving_donors_all
    qx_lpa_surviving_donors_all_vals = qx_lpa_surviving_donors_all
    qx_lpa_surviving_donors_all = qx_lpa_surviving_donors_all.reset_index()
    lpa_surviving_donors_all = lpa_surviving_donors_all.reset_index()
    lpa_surviving_donors_all_vals = lpa_surviving_donors_all_vals.assign(age = pd.Series(age_range).values)
    qx_lpa_surviving_donors_all_vals = qx_lpa_surviving_donors_all_vals.assign(age = pd.Series(age_range).values)
    lpa_surviving_donors_all_dict[year_life_tbl] = lpa_surviving_donors_all_vals
    lpa_surviving_donors_all_dict[year_life_tbl] = lpa_surviving_donors_all_vals
    #print(f"year_life_tbl in of the life_tbl: {year_life_tbl} - lpa_surviving_donors_all_vals: {lpa_surviving_donors_all_vals}")
    
    # Ensure lpa_surviving_donors_all_dict[year_life_tbl] exists and has valid data
    if year_life_tbl in lpa_surviving_donors_all_dict:
        lpa_surviving_donors_all_aged = lpa_surviving_donors_all_dict[year_life_tbl]

        # Ensure lpa_surviving_donors_all_aged is not None
        if lpa_surviving_donors_all_aged is not None:
            # Reorder the columns to move 'age' to the first position
            first_column = lpa_surviving_donors_all_aged.pop('age')

            # Insert 'age' as the first column
            lpa_surviving_donors_all_aged.insert(0, 'age', first_column)

            # Check the DataFrame before saving
            #print(f"Data for year of life table: {year_life_tbl}:")
            #print(lpa_surviving_donors_all_aged)

            # Save to CSV
            lpa_surviving_donors_all_aged.to_csv(f"lpa_surviving_donors_{year_life_tbl}.csv", index=False)
        else:
            print(f"Error: lpa_surviving_donors_all_aged is None for year {year_life_tbl}.")
    else:
        print(f"Error: No data found for year {year_life_tbl} in lpa_surviving_donors_all_dict.")
    
    # Calculate total living customers for each year by summing across ages
    total_lpa_surviving_donors_df[year_life_tbl] = lpa_surviving_donors_all_aged.iloc[:, 1:].sum(axis=1) #(axis=1)
    #total_lpa_surviving_donors['Total_lpa_surviving_donors'] = lpa_surviving_donors_all_aged.groupby('age')[year].sum().reset_index()
    
    surviving_donors_lpa[year_life_tbl] = lpa_surviving_donors_all_aged.iloc[:, 1:].sum(axis=1)
    #print(f"total_lpa_surviving_donors_df[{year_life_tbl}]: {total_lpa_surviving_donors_df[year_life_tbl]}")
    
    
    
    #total_lpa_surviving_donors_df['age'] = range(18, 107)  # range for ages 18-106
    #total_lpa_surviving_donors_df.columns = [years_life_tbls, 'age']
    #total_lpa_surviving_donors_df.columns = [year_life_tbl, 'age']
    #print(f"total_lpa_surviving_donors_df[year_life_tbl, 'age']: {total_lpa_surviving_donors_df[year_life_tbl, 'age']}")
    # Check the number of columns in the DataFrame
    #print(f"Columns before renaming: {total_lpa_surviving_donors_df.columns}")

    # Ensure 'age' is part of the DataFrame if needed
    if 'age' not in total_lpa_surviving_donors_df.columns:
        # If 'age' needs to be added, ensure it exists as a series
        total_lpa_surviving_donors_df['age'] = age_range  # range for ages 18-106

    # Check if the DataFrame has 2 columns now
    if total_lpa_surviving_donors_df.shape[1] == 2:
        total_lpa_surviving_donors_df.columns = [year_life_tbl, 'age']
    else:
        #print(f"Warning: The DataFrame does not have 2 columns. It has {total_lpa_surviving_donors_df.shape[1]} columns.")
        #print(f"Warning: The DataFrame has age column already.")
        total_lpa_surviving_donors_df = total_lpa_surviving_donors_df
        
    #Rename the column name by the year
    total_lpa_surviving_donors_df.rename(columns={'Total_lpa_surviving_donors': year})  #, inplace=True)
    
    total_lpa_surviving_donors_dict[year_life_tbl] = total_lpa_surviving_donors_df.set_index(['age']) # pd.DataFrame(total_lpa_surviving_donors).set_index(['age'])
    #print(f"for year_life_tbl: {year_life_tbl} {total_lpa_surviving_donors_dict[year_life_tbl]}")
    total_lpa_surviving_donors_dict[year_life_tbl].iloc[:, :-1] = pd.NA
    #print(f"After NA applied to the prior years: for year_life_tbl: {year_life_tbl} {total_lpa_surviving_donors_dict[year_life_tbl]}")
    
#Concatenate all yearly living customer tables into a single DataFrame
total_life_tbl = pd.concat(total_lpa_surviving_donors_dict.values(), axis=0)
#total_life_tbl = total_lpa_surviving_donors_dict[end_forecast_year]
total_life_tbl = total_life_tbl.reset_index()
# Reorder the columns to move 'age' to the first position
total_life_tbl.pop('age')
total_life_df = total_life_tbl
# Insert 'age' as the first column
#total_life_tbl.insert(0, 'age', first_column)
print(f"LPA LIFE TABLES : {total_life_df}")
total_life_df.to_csv('LPA_LIFE_TABLES.csv', index=False) 
LPA_LIFE_TABLES = total_life_df

LPA_SURVIVAL_TABLES_individual_years_tables = pd.concat(lpa_surviving_donors_all_dict.values(), axis=0)
print(f"LPA SURVIVAL TABLES (for each individual year table): {total_life_df}")
LPA_SURVIVAL_TABLES_individual_years_tables.to_csv('LPA_SURVIVAL_TABLES_individual_years_tables.csv', index=False)

surviving_donors_lpa_df = pd.concat(surviving_donors_lpa.values(), axis=1)
# Rename the columns to years from 2007 to 2023
surviving_donors_lpa_df.columns = [str(year) for year in years_forecast_long]
print(f"LPA SURVIVAL TABLES (TOTALS):") # {surviving_donors_lpa_df}")
surviving_donors_lpa_df.to_csv('LPA_SURVIVAL_TABLES.csv', index=False)     
LPA_SURVIVAL_TABLES_TOTALS = surviving_donors_lpa_df
LPA_SURVIVAL_TABLES_TOTALS
## ########################################################################################## ##        
    # Sum based on number of years before year_index (e.g., year_index(2023) - year(2007) = 12)
    # To calculate the sum of the 'lx' values for each year_life_tbl in lpa_surviving_donors_all_dict[year_life_tbl],
    # iterate over each year_life_tbl, 
    # sum the values based on the conditions/assumptions provided, 
    # adjust the number of years that should be summed depending on whether the current year_life_tbl is less than or equal to year_index. 

    #             x = lpa_surviving_donors_dict_changed[2007]
#             y = x.reset_index()
#             #y.loc[y['age' == 18]]
#             y.iloc[0,1] 


# Forecasting model

## Previous Forecasting usig exponential smoothing:
I have had a look now and the approach that I previously used is really quite simple, but may not work that well with the reduction in LPA demand following the pandemic.

If X is the smoothing factor, and t the most recent year,  then a simple exponentially smoothed forecast is generated as follows

(LPAs as a % of remaining customers)t * X + ((LPAs as a % of remaining customers)t + (LPAs as a % of remaining customers)t - (LPAs as a % of remaining customers)t-1) * (1-X)

Ie it simply takes X% of the most recent value and adds onto it a proportion (1-X)% of the increase in LPAs as a % of remaining customers over the previous year. 

This is obviously quite simplistic but has worked well whilst these %s have increased from year to year. 

However this approach would not really work that well if we saw a sudden decrease in the %, over the course of the year, as this approach would only then allow for the trend to decrease. This may be the case for some of the age specific %’s following the pandemic, and may not be representative of the trend that was apparent before the pandemic. 

So what is probably needed really is a bit of  rethink about how best to do this so that the generated trend is not entirely dependant on the change in the % over the last 12 months .

## New forecasting model
I will leave you to have a think about how best to do this !!! but I guess some possibilities are 
•	Averaging annual increases over more than one year 
•	Applying a Bayesian approach to updating prior estimates .

Fitting the model
You can use a linear regression model to estimate the avocado price elasticity. The regression formula should be:

Volume ~ Normal(beta0 + beta1 * price + beta2 * type_organic + sigma)

Here, will be the price elasticity, that is the impact of price on sales. You will assume that the elasticity is the same for regular and organic avocados. You also expect it to be negative: the higher the price, the lower the sales, that's the case for most goods. To incorporate this prior knowledge into the model, you decide to use a normal distribution with mean -80 as the prior for price. How would you build such a model?

NOTE: Recall that calling pm.sample() for the first time in a fresh Python session takes some time, as Python code is being compiled to C under the hood. To save you time, we only ask you to get the code right instead of executing it.

Optimizing the price
Great job on fitting and inspecting the model! Now, down to business: your boss asks you to provide the avocado price that would yield the largest profit, and to state what profit can be expected. Also, they want the price to be divisible by $0.25 so that the customers can easily pay with quarters.

In this exercise, you will use your model to predict the volume and the profit for a couple of sensible prices. Next, you will visualize the predictive distributions to pick the optimal price. Finally, you will compute the credible interval for your profit prediction. Now go and optimize!

Predicting Profit for Each Price:

The code iterates over a list of potential prices: [0.5, 0.75, 1, 1.25].
For each price, it calculates the predicted mean volume using a linear model: pred_mean = (intercept_mean + price_mean * price + organic_mean). This formula likely comes from a previously fitted Bayesian model where intercept_mean, price_mean, and organic_mean are the posterior means of the model parameters.
It then simulates the predicted volume using a normal distribution centered around pred_mean with a standard deviation sd_mean, generating 1000 samples: volume_pred = np.random.normal(pred_mean, sd_mean, size=1000).
The predicted profit for each price is calculated by multiplying the price by the predicted volume: profit_pred = price * volume_pred.
These predicted profits are stored in a dictionary predicted_profit_per_price with the price as the key.
Visualizing Predicted Profits:

A forest plot is created using pm.forestplot(predicted_profit_per_price). This plot visualizes the distribution of predicted profits for each price, helping to identify which price might yield the highest profit.
Calculating the Highest Posterior Density (HPD) Interval:

The code calculates the HPD interval for the predicted profits at the optimal price, which is chosen as 0.75 in this solution. The HPD interval is a Bayesian credible interval that contains the most probable values of the parameter.
The function az.hdi(predicted_profit_per_price[0.75], credible_interval=0.99) computes the 99% HPD interval for the predicted profits at the price of 0.75.
Finally, it prints the HPD interval, which provides a range of values within which the true profit is likely to lie with 99% probability.
This solution effectively uses Bayesian analysis to predict profits for different prices, visualizes these predictions, and calculates a credible interval for the optimal price, thus fulfilling the exercise's requirements.



In [ ]:
# Bayesian Hyperparameter tuning with Hyperopt



In [ ]:


# Linear regression formula
formula = "volume ~ price + type_organic"

# the MCMC model
with pm.Model() as model:    
    priors = {"price": pm.Normal.dist(mu=-5)}    
    pm.GLM.from_formula(formula, data=avacado, priors=priors)    
    trace = pm.sample(draws=1000, tune=500)


# Draw a trace plot of trace
pm.traceplot(trace)
plt.show()

# Print a summary of trace
summary = pm.summary(trace)
print(summary)

# Get each parameter's posterior mean
intercept_mean = np.mean(trace.get_values("Intercept")) 
organic_mean = np.mean(trace.get_values("type_organic")) 
price_mean = np.mean(trace.get_values("price"))
sd_mean = np.mean(trace.get_values("sd"))

# For each price, predict volume and use it to predict profit
predicted_profit_per_price = {}
for price in [0.5, 0.75, 1, 1.25]:
    pred_mean = (intercept_mean + price_mean * price + organic_mean)
    volume_pred = np.random.normal(pred_mean, sd_mean, size=1000)
    profit_pred = price * volume_pred
    predicted_profit_per_price.update({price: profit_pred})
    
# Draw a forest plot of predicted profit for all prices
pm.forestplot(predicted_profit_per_price)
plt.show()

# Calculate and print HPD of predicted profit for the optimal price
opt_hpd = az.hdi(predicted_profit_per_price[0.75], credible_interval=0.99)
print(opt_hpd)

In [ ]:
simplified_source_life = source_life_df
source_life_end_ons_year = source_life_df.loc[(source_life_df['year'] == end_ons_year)]

# Iterate over each future year from 2023 to 2039 and replace invalid qx values
for year in years_range:
    for age in ages:
        # Get current qx value
        qx_end_ons_year = source_life_end_ons_year.loc[(source_life_end_ons_year['age'] == age), 'qx'].values[0]
        # simplified mortalitility based on the previous year (last year qx was available:    
        simplified_source_life.loc[(simplified_source_life['year'] == year) & (
                simplified_source_life['age'] == age), 'qx'] = qx_end_ons_year
        
# Save the updated DataFrame to a new file
simplified_file_path = 'simplified_mortality_statistics.csv'
simplified_source_life.to_csv(simplified_file_path, index=False)

#print(simplified_source_life)
simplified_source_life

In [ ]:
# Simplified LPA SURVIVAL TABLES

life_tbl_src = simplified_source_life

#year_life_tbl = 2007
first_column = []
lpa_survival_tables_all = pd.DataFrame()
lpa_survival_tables_all_vals = pd.DataFrame()
lpa_survival_tables_all_aged = pd.DataFrame()
total_lpa_surviving_donors_df = pd.DataFrame()
#years = list(range(start_year, end_forecast_year + 1))

# Initialise a dictionary to hold lpa_surviving_donors data for each year
#lpa_surviving_donors_dict = {}
#lpa_surviving_donors_dict_changed = {}
lpa_surviving_donors_all_dict = {}
total_lpa_surviving_donors_dict = {}
total_life_tbl = pd.DataFrame()
surviving_donors_lpa = {} #pd.DataFrame()
summed_lx_dict_by_year = {}

# Define the age groups
age_groups = {"age_groups": age_range}
age_groups_df = pd.DataFrame(age_groups)


lpa_life_tbl_dict = {}
qx_lpa_surviving_donors_all = pd.DataFrame() #{}


# Extend life tables for all years:
for year_life_tbl in years_forecast_long:  #list(range(2007, 2010)): #
    
     
    lpa_surviving_donors = pd.DataFrame()
    lpa_surviving_donors_df = pd.DataFrame()
    lpa_surviving_donors_tbl = pd.DataFrame()
    lpa_surviving_donors_all_df = pd.DataFrame()
    lpa_surviving_donors_dict_changed = {}
    lpa_surviving_donors_dict = {}
    qx_lpa_surviving_donors_dict = {}
    qx_lpa_surviving_donors_df = pd.DataFrame() 
    qx_lpa_surviving_donors = pd.DataFrame()
    qx_lpa_surviving_donors_tbl = pd.DataFrame()
    qx_lpa_surviving_donors_all_df = pd.DataFrame()
    qx_lpa_surviving_donors_dict_changed = {}
    qx_lpa_surviving_donors_dict = {}
    
    # Years from the year_life_tbl to end_forecast_year
    years_life_tbls = list(range(year_life_tbl, year_forecast_long + 1))
    
    # Iterate over each year for which life table needs to be generated
    for year in years_life_tbls: 
        
        yearly_data = []
        
        for age in ages: 
            
            qx_current_vals = life_tbl_src.loc[(life_tbl_src['year'] == year) & (
                life_tbl_src['age'] == age), 'qx'].values
            
            lx_current_vals = life_tbl_src.loc[(life_tbl_src['year'] == year) & (
                life_tbl_src['age'] == age), 'lx'].values
            
            # Calculate population for each age group, shifting cohorts forward each yea
            if (year == year_life_tbl):  # Starting from the initial year
                
                # Calculate / Update the survivors (lx) based on the lpa donor data

                if (age - (year - year_life_tbl) >= start_age): # for shifting for the next year and next age (age+1) 

                    qx = qx_current_vals[0] if len(qx_current_vals) > 0 else np.nan
                    
                    # Get lx for the current age and year  
                    lx_current = lx_current_vals[0] if not np.isnan(lx_current_vals) else np.nan 
                    
                    lx = lx_current# * (1 - qx) #if np.isnan(lx_current_vals) else 0 #np.nan
                    
                    #qx=life_tbl_src.loc[(life_tbl_src['year'] == year) & (life_tbl_src['age'] == age),'qx'].values[0]
                    #lx = life_tbl_src.loc[(life_tbl_src['year'] == year) & (life_tbl_src['age'] == age),'lx'].values[0]
                    #print(f"NON-ZEROS: year:{year} year_life_tbl:{year_life_tbl}, age:{age}, qx:{qx}, lx:{lx}")
                    
                    
                else:
                    lx = np.nan
                    qx = np.nan
                    #print(f"ZEROS: year:{year} year_life_tbl:{year_life_tbl}, age:{age}, qx:{qx}, lx:{lx}")
                            
            else: 
                # Access the data for the previous year (to calculate forward cohorts) 
                if (age - (year - year_life_tbl) >= start_age): # for shifting for the next year and next age (age+1) 
                    
                    #qx_prev = life_tbl_src.loc[(life_tbl_src['year'] == year-1) & (
                    #   life_tbl_src['age'] == age),'qx'].values[0] 
                    qx_cur = life_tbl_src.loc[(life_tbl_src['year'] == year) & (
                        life_tbl_src['age'] == age),'qx'].values[0] 
                    
                    #lx_prev = lpa_surviving_donors.loc[(lpa_surviving_donors['year'] == year-1) & (
                    #    lpa_surviving_donors['age'] == age-1)].values[0] 
                    lx_prev = lpa_surviving_donors_df.loc[(year-1, age-1), 'lx']
                    #lpa_surviving_donors_df.loc[(year-1,  [age-4,age-3,age-2,age-1]), 'lx']
                    #lx_cur = lpa_surviving_donors_dict_changed[year_life_tbl].iloc[age-start_age,0] if age < end_age else lpa_surviving_donors_dict_changed[year_life_tbl].iloc[age-start_age,0]
#                     lx_cur = life_tbl_src.loc[(
#                         life_tbl_src['year'] == year_life_tbl) & (
#                         life_tbl_src['age'] == age-1),'lx'].values[0] if age < end_age else life_tbl_src.loc[(
#                         life_tbl_src['year'] == year_life_tbl) & (
#                         life_tbl_src['age'] == age),'lx'].values[0]
                    
                    #if (lx_cur > 0) & (qx_prev > 0): # & (age <= 100):# & (age < 106): #(len(qx_prev) > 0)

                    qx = qx_cur

                    #ex_qx = life_tbl_src.loc[(life_tbl_src['year'] == year) & (life_tbl_src['age'] == age),'qx'].values # qx_lpa_surviving_donors[year-1][age-start_age]
                    #print(f"year:{year} year_life_tbl:{year_life_tbl}, age:{age}, qx:{qx}, lx_cur:{lx_cur}, lx:{lx_cur * (1 - qx)}")
                    
                    
                    # Calculate current year's lx based on previous year's lx and previous qx
#                     lx = lx_cur * (1 - qx) if not np.isnan(qx) else np.nan
                    lx = lx_prev * (1 - qx) if not np.isnan(qx) else np.nan
                                                       
                else:
                    lx = np.nan
                    qx = np.nan

                
            # Append the data for the current age to the yearly data list
            yearly_data.append({'year': year, 'age': age, 'lx': lx, 'qx': qx})

        # Convert yearly data to a DataFrame and set multi-index for easy access
        lpa_surviving_donors_df = pd.DataFrame(yearly_data).set_index(['year', 'age'])
        #lpa_surviving_donors_inx = lpa_surviving_donors_df
        #lpa_surviving_donors_tbl = lpa_surviving_donors_dict[year]
        lpa_surviving_donors_dict[year] = lpa_surviving_donors_df[['lx']]
        qx_lpa_surviving_donors_dict[year] = lpa_surviving_donors_df[['qx']]
        #lpa_surviving_donors_dict[year].to_csv(f"lpa_surviving_donors_{year}.csv", index=False)
        #print(f"lpa_surviving_donors_dict({year}): {lpa_surviving_donors_dict[year]}") 
         
        lpa_surviving_donors = lpa_surviving_donors_df[['lx']]
        lpa_surviving_donors = lpa_surviving_donors.reset_index() #pd.DataFrame(lpa_surviving_donors).reset_index()
        lpa_surviving_donors = lpa_surviving_donors.rename(columns={'lx':year})  #, inplace=True)
        #print(f"year: {year} - lpa_surviving_donors: {lpa_surviving_donors}")        
        qx_lpa_surviving_donors = lpa_surviving_donors_df[['qx']]
        qx_lpa_surviving_donors = qx_lpa_surviving_donors.reset_index() #pd.DataFrame(lpa_surviving_donors).reset_index()
        qx_lpa_surviving_donors = qx_lpa_surviving_donors.rename(columns={'qx':year})  #, inplace=True)
        
        lpa_surviving_donors_tbl = lpa_surviving_donors #lpa_surviving_donors[['age', year]] 
        qx_lpa_surviving_donors_tbl = qx_lpa_surviving_donors #lpa_surviving_donors[['age', year]] 
        #lpa_surviving_donors_tbl = lpa_surviving_donors.assign(Age=pd.Series(range(18,107)).values)
        lpa_surviving_donors_all_df = lpa_surviving_donors_tbl.set_index(['age'])
        #print(f"lpa_surviving_donors_all_df: {lpa_surviving_donors_all_df}")
        qx_lpa_surviving_donors_all_df = qx_lpa_surviving_donors_tbl.set_index(['age'])
        lpa_surviving_donors_dict_changed[year] = lpa_surviving_donors_all_df[[year]]
        qx_lpa_surviving_donors_dict_changed[year] = qx_lpa_surviving_donors_all_df[[year]]
        #print(f"year: {year} - lpa_surviving_donors_dict_changed: {lpa_surviving_donors_dict_changed[year]}") 
    lpa_life_tbl_dict[year_life_tbl] = lpa_surviving_donors_df
    #qx_lpa_life_tbl_dict[year_life_tbl] = lpa_surviving_donors_df
    # Concatenate all yearly data into a single DataFrame
    lpa_surviving_donors_all = pd.concat(lpa_surviving_donors_dict_changed.values(), axis=1)   
    
    qx_lpa_surviving_donors_all = pd.concat(qx_lpa_surviving_donors_dict_changed.values(), axis=1) 
     
    #print(f"lpa_surviving_donors_all from lpa_surviving_donors_dict_changed: {lpa_surviving_donors_all}")
    lpa_surviving_donors_all_vals = lpa_surviving_donors_all
    qx_lpa_surviving_donors_all_vals = qx_lpa_surviving_donors_all
    qx_lpa_surviving_donors_all = qx_lpa_surviving_donors_all.reset_index()
    lpa_surviving_donors_all = lpa_surviving_donors_all.reset_index()
    lpa_surviving_donors_all_vals = lpa_surviving_donors_all_vals.assign(age = pd.Series(age_range).values)
    qx_lpa_surviving_donors_all_vals = qx_lpa_surviving_donors_all_vals.assign(age = pd.Series(age_range).values)
    lpa_surviving_donors_all_dict[year_life_tbl] = lpa_surviving_donors_all_vals
    lpa_surviving_donors_all_dict[year_life_tbl] = lpa_surviving_donors_all_vals
    #print(f"year_life_tbl in of the life_tbl: {year_life_tbl} - lpa_surviving_donors_all_vals: {lpa_surviving_donors_all_vals}")
    
    # Ensure lpa_surviving_donors_all_dict[year_life_tbl] exists and has valid data
    if year_life_tbl in lpa_surviving_donors_all_dict:
        lpa_surviving_donors_all_aged = lpa_surviving_donors_all_dict[year_life_tbl]

        # Ensure lpa_surviving_donors_all_aged is not None
        if lpa_surviving_donors_all_aged is not None:
            # Reorder the columns to move 'age' to the first position
            first_column = lpa_surviving_donors_all_aged.pop('age')

            # Insert 'age' as the first column
            lpa_surviving_donors_all_aged.insert(0, 'age', first_column)

            # Check the DataFrame before saving
            #print(f"Data for year of life table: {year_life_tbl}:")
            #print(lpa_surviving_donors_all_aged)

            # Save to CSV
#             lpa_surviving_donors_all_aged.to_csv(f"sim_lpa_surviving_donors_{year_life_tbl}.csv", index=False)
        else:
            print(f"Error: lpa_surviving_donors_all_aged is None for year {year_life_tbl}.")
    else:
        print(f"Error: No data found for year {year_life_tbl} in lpa_surviving_donors_all_dict.")
    
    # Calculate total living customers for each year by summing across ages
    total_lpa_surviving_donors_df[year_life_tbl] = lpa_surviving_donors_all_aged.iloc[:, 1:].sum(axis=1) #(axis=1)
    #total_lpa_surviving_donors['Total_lpa_surviving_donors'] = lpa_surviving_donors_all_aged.groupby('age')[year].sum().reset_index()
    
    surviving_donors_lpa[year_life_tbl] = lpa_surviving_donors_all_aged.iloc[:, 1:].sum(axis=1)
    #print(f"total_lpa_surviving_donors_df[{year_life_tbl}]: {total_lpa_surviving_donors_df[year_life_tbl]}")
    
    
    
    #total_lpa_surviving_donors_df['age'] = range(18, 107)  # range for ages 18-106
    #total_lpa_surviving_donors_df.columns = [years_life_tbls, 'age']
    #total_lpa_surviving_donors_df.columns = [year_life_tbl, 'age']
    #print(f"total_lpa_surviving_donors_df[year_life_tbl, 'age']: {total_lpa_surviving_donors_df[year_life_tbl, 'age']}")
    # Check the number of columns in the DataFrame
    #print(f"Columns before renaming: {total_lpa_surviving_donors_df.columns}")

    # Ensure 'age' is part of the DataFrame if needed
    if 'age' not in total_lpa_surviving_donors_df.columns:
        # If 'age' needs to be added, ensure it exists as a series
        total_lpa_surviving_donors_df['age'] = age_range  # range for ages 18-106

    # Check if the DataFrame has 2 columns now
    if total_lpa_surviving_donors_df.shape[1] == 2:
        total_lpa_surviving_donors_df.columns = [year_life_tbl, 'age']
    else:
        #print(f"Warning: The DataFrame does not have 2 columns. It has {total_lpa_surviving_donors_df.shape[1]} columns.")
        #print(f"Warning: The DataFrame has age column already.")
        total_lpa_surviving_donors_df = total_lpa_surviving_donors_df
        
    #Rename the column name by the year
    total_lpa_surviving_donors_df.rename(columns={'Total_lpa_surviving_donors': year})  #, inplace=True)
    
    total_lpa_surviving_donors_dict[year_life_tbl] = total_lpa_surviving_donors_df.set_index(['age']) # pd.DataFrame(total_lpa_surviving_donors).set_index(['age'])
    #print(f"for year_life_tbl: {year_life_tbl} {total_lpa_surviving_donors_dict[year_life_tbl]}")
    total_lpa_surviving_donors_dict[year_life_tbl].iloc[:, :-1] = pd.NA
    #print(f"After NA applied to the prior years: for year_life_tbl: {year_life_tbl} {total_lpa_surviving_donors_dict[year_life_tbl]}")
    
#Concatenate all yearly living customer tables into a single DataFrame
total_life_tbl = pd.concat(total_lpa_surviving_donors_dict.values(), axis=0)
#total_life_tbl = total_lpa_surviving_donors_dict[end_forecast_year]
total_life_tbl = total_life_tbl.reset_index()
# Reorder the columns to move 'age' to the first position
total_life_tbl.pop('age')
total_life_df = total_life_tbl
# Insert 'age' as the first column
#total_life_tbl.insert(0, 'age', first_column)
print(f"Simplified LPA LIFE TABLES : {total_life_df}")
total_life_df.to_csv('Simplified LPA_LIFE_TABLES.csv', index=False) 
LPA_LIFE_TABLES = total_life_df

LPA_SURVIVAL_TABLES_individual_years_tables = pd.concat(lpa_surviving_donors_all_dict.values(), axis=0)
print(f"Simplified LPA SURVIVAL TABLES (for each individual year table): {total_life_df}")
LPA_SURVIVAL_TABLES_individual_years_tables.to_csv('Simplified_LPA_SURVIVAL_TABLES_individual_years_tables.csv', index=False)

surviving_donors_lpa_df = pd.concat(surviving_donors_lpa.values(), axis=1)
# Rename the columns to years from 2007 to 2023
surviving_donors_lpa_df.columns = [str(year) for year in years_forecast_long]
print(f"Simplified LPA SURVIVAL TABLES (TOTALS):") # {surviving_donors_lpa_df}")
surviving_donors_lpa_df.to_csv('Simplified_LPA_SURVIVAL_TABLES.csv', index=False)     
Simplified_LPA_SURVIVAL_TABLES_TOTALS = surviving_donors_lpa_df
Simplified_LPA_SURVIVAL_TABLES_TOTALS
## ########################################################################################## ##        
    # Sum based on number of years before year_index (e.g., year_index(2023) - year(2007) = 12)
    # To calculate the sum of the 'lx' values for each year_life_tbl in lpa_surviving_donors_all_dict[year_life_tbl],
    # iterate over each year_life_tbl, 
    # sum the values based on the conditions/assumptions provided, 
    # adjust the number of years that should be summed depending on whether the current year_life_tbl is less than or equal to year_index. 

    #             x = lpa_surviving_donors_dict_changed[2007]
#             y = x.reset_index()
#             #y.loc[y['age' == 18]]
#             y.iloc[0,1] 


Survival Rates: The code now considers the survival rates across different ages (from 18 to 106). 
The survival rate is factored into how the cohort sizes decrease year by year.

Forecasting: For each iteration in the Monte Carlo simulation, the number of donors is estimated dynamically, considering the varying age-specific survival rates.

Market Shape: The assumedMarketShape is calculated based on the specific age group’s donor population compared to the total population. 
The assumedPotentialMarketShape is normalized to a 0-100 scale.

This approach allows for a more accurate long-term forecast of customer/donor behavior by integrating demographic aging and survival rates into the market model.

# To extend the lpa_surviving_donors calculation for ages from 18 to 106 and integrate it into the long-term cohort survival forecasting, we need to make sure that the cohort survival rates are applied dynamically across the forecasted years, adjusting for demographic aging. Below is a refined explanation and code modification to include this requirement in your existing VBA code.

Step 1: Extend lpa_surviving_donors Calculation
To extend the lpa_surviving_donors calculation from ages 18 to 106, you'll first need to calculate survival rates for each age group. This can be done using the life table that provides qx values (the probability of dying within one year).

Assumption: The life table's qx values are already calculated for all age groups from 18 to 106.

Step 2: Modify VBA Code for Cohort Survival Forecasting
Below is the modified code segment that integrates the survival rates into the cohort-based forecasting model:
    
Step 3: Integration and Testing
After implementing this code:
Test the modified code using the existing life table and demographic data.
Verify the accuracy by comparing the results with known historical data or through validation with external models.



In [ ]:

# Load the data
lpa_demands = historical_data_df

# Get the last row in column A
#last_row = ws.max_row

# donors_df: Contains donor data for 2022 and 2023, including year, age, and demand (number of donors per age group).
donors_df = annual_donor_demands_age_specific

# # Sample data for donors_df
# donors_data = {
#     'year': [2022, 2022, 2023, 2023],
#     'age': [30, 50, 30, 50],
#     'demand': [500, 300, 550, 320]
# }
# donors_df = pd.DataFrame(donors_data)

# population_forecast_df: Contains the forecasted population for each age group in 2023 and 2024.

# population_forecast_df: Contains the forecasted population for each age group in 2023 and 2024.
population_forecast = pd.read_csv('population_forecast.csv')
# Convert the DataFrame to the desired format
df_melted = population_forecast.melt(id_vars=['age'], var_name='year', value_name='population')
population_forecast_df = pd.DataFrame(df_melted)

pop_f = population_forecast_df#.set_index(['age', 'year'])
pop_f['year'] = pd.to_numeric(pop_f['year'], errors='coerce')
pop_f['age'] = pd.to_numeric(pop_f['age'], errors='coerce')

# Sort the data by year and age for proper processing
pop_f.sort_values(by=['year', 'age'], inplace=True)
population_forecast_df = pop_f
pop_f#pop_f[(pop_f['age'] == age) & (pop_f['year'] == year)]['population']
#population_forecast_df[(population_forecast_df['age'] == age) & 
#                                              (population_forecast_df['year'] == 2023)]['population']
# # Sample data for population_forecast_df
# population_forecast_data = {
#     'year': [2023, 2023, 2024, 2024],
#     'age': [30, 50, 30, 50],
#     'population': [10000, 8000, 10500, 8200]
# }
# population_forecast_df = pd.DataFrame(population_forecast_data)

# survival_rate_df: Contains survival rates for each age group for future years.
surviving_lpa = LPA_SURVIVAL_TABLES_TOTALS
survival_tbl = surviving_lpa.reset_index()
survival_tbl = survival_tbl.rename(columns={'age':'age'})
# Convert the DataFrame to the desired format
survival_melted = survival_tbl.melt(id_vars=['age'], var_name='year', value_name='survived')
survival_df = pd.DataFrame(survival_melted)

# # Sample data for survival_rate_df
# survival_rate_data = {
#     'age': [30, 50],
#     'year': [2025, 2025],
#     'survival_rate': [0.95, 0.93]
# }
# survival_rate_df = pd.DataFrame(survival_rate_data)


# Merge the DataFrames on 'year' and 'age'
#merged_df = pd.merge(survival_df, pop_f, on=['year', 'age'], how='inner')
merged_df = survival_df
# Calculate the demand/population ratio
#merged_df['survival_rate'] = merged_df['survived'] / merged_df['population']
merged_df['survival_rate'] = survival_df['survived'] / pop_f['population']

survival_rate_df = merged_df[['age', 'year', 'survival_rate']]
survival_rate_df

In [ ]:

# # Prompt user to ask regarding control assumption to apply the market shape or not
# ## Automating the input dates to forecast

# p = getpass.getpass(prompt='Do you want to apply the market shape for forecasting? (Choose Yes=Y OR No=N)')
 
# if (p.lower() == 'n') | (p.lower() == ''):
#     # The market shape = 100%
#     print('You have not choosen to apply the market shape.')
    
# else:    
#     # Apply the market shape based on the ONS population forecast
#     print('You have choosen to apply the market shape.')




# Additional forecast for future years (2025-2029) can be added similarly using previous years' data and survival rates.
# handling age-specific market data
# Dictionary to store results
total_population_age_specific = {}
total_donors_age_specific = {}
assumed_market_shape = {}
max_market_estimate = {}
assumed_potential_market_shape = {}
total_surviving_age_specific = {}
total_survival_rate_age_specific = {}
assumed_potential_survival_rate = {}
assumed_survival_rate = {}
max_survival_estimate = {}
avg_survival_rate = {}
assumed_potential_avg_survival_rate = {}
max_avg_survival_estimate = {}
# Create arrays for total population and donors (106 age groups, assuming 0-105 years)
#total_population_age_specific = np.zeros(106)
#total_donors_age_specific = np.zeros(106)
#assumed_market_shape = np.zeros(106)
#assumed_potential_market_shape = np.zeros(106)


# for age in range(18, 107):
#     population_2023 = population_forecast_df[(population_forecast_df['age'] == age) & 
#                                              (population_forecast_df['year'] == 2023)]['population'].sum()
#     population_2024 = population_forecast_df[(population_forecast_df['age'] == age) & 
#                                              (population_forecast_df['year'] == 2024)]['population'].sum()
#     total_population_age_specific[age] = population_2023 + population_2024
#     donors_2023 = df_donors[(df_donors['year'] == 2023) & (df_donors['age'] == age)]['Demand'].sum()
#     forecasted_donors_2024 = df_output[df_output['age'] == age]['Forecasted_Donors_Mean'].sum()
#     total_donors_age_specific[age] = donors_2023 + forecasted_donors_2024
    
    
# Iterate over age range
for age in ages:

    # Ensure 'age' and 'year' columns are integers
    population_forecast_df['age'] = population_forecast_df['age'].astype(int)
    population_forecast_df['year'] = population_forecast_df['year'].astype(int)

    # Check if the age and year exist in the DataFrame
    #print(population_forecast_df[population_forecast_df['year'] == 2023].head())
    #print(population_forecast_df[population_forecast_df['age'] == age].head())

    # Calculate total population age-specific for 2023 and 2024 (current and last year)
    total_population_age_specific[age] = population_forecast_df[
        ((population_forecast_df['age'] == age) & (population_forecast_df['year'] == 2023)) |
        ((population_forecast_df['age'] == age) & (population_forecast_df['year'] == 2024))
    ]['population'].sum()
    
    # Calculate total donors age-specific for 2023 and 2024 (current and last year)
    total_donors_age_specific[age] = (donors_df[
        ((donors_df['year'] == 2023) & (donors_df['age'] == age)) |
        ((donors_df['year'] == 2024) & (donors_df['age'] == age))
    ]['demand'].sum())

    # Ensure 'age' and 'year' columns are integers
    survival_df['age'] = survival_df['age'].astype(int)
    survival_df['year'] = survival_df['year'].astype(int)

    # Check if the age and year exist in the DataFrame
    #print(survival_df[survival_df['year'] == 2023].head())
    #print(survival_df[survival_df['age'] == age].head())
    
    # Calculate total surviving customers age-specific for 2023 and 2024 (current and last year)
    total_surviving_age_specific[age] = (survival_df[
        ((survival_df['year'] == 2023) & (survival_df['age'] == age)) |
        ((survival_df['year'] == 2024) & (survival_df['age'] == age))
    ]['survived'].sum())
    
    # Ensure 'age' and 'year' columns are integers
    survival_rate_df['age'] = survival_rate_df['age'].astype(int)
    survival_rate_df['year'] = survival_rate_df['year'].astype(int)

    # Check if the age and year exist in the DataFrame
    #print(survival_df[survival_df['year'] == 2023].head())
    #print(survival_df[survival_df['age'] == age].head())
    
    # Calculate total survival rate age-specific for 2023 and 2024 (current and last year)
    total_survival_rate_age_specific[age] = (survival_rate_df[
        ((survival_rate_df['year'] == 2023) & (survival_rate_df['age'] == age)) |
        ((survival_rate_df['year'] == 2024) & (survival_rate_df['age'] == age))
    ]['survival_rate'].sum())
    
    # Calculate assumed market shape 
    assumed_survival_rate[age] = total_surviving_age_specific[age] / total_population_age_specific[age] if total_population_age_specific[age] > 0 else 0

    # Calculate survival rate
    assumed_market_shape[age] = total_donors_age_specific[age] / total_population_age_specific[age] if total_population_age_specific[age] > 0 else 0
    
    #Calculate Average survival rate
    avg_survival_rate[age] = survival_rate_df[
        ((survival_rate_df['year'] == 2023) & (survival_rate_df['age'] == age)) |
        ((survival_rate_df['year'] == 2024) & (survival_rate_df['age'] == age))
    ]['survival_rate'].mean()
    
    
# Normilisation:
# Find max assumed market shape
max_assumed_market_shape = max(assumed_market_shape.values())
print(f"Maximum of Assuming Market Shapes: {max_assumed_market_shape}")

# Find max survival rate
max_assumed_survival_rate = max(assumed_survival_rate.values())
print(f"Maximum of Assumed Survival Rate: {max_assumed_survival_rate}")

# Find max average survival rate
max_avg_survival_rate = max(avg_survival_rate.values())
print(f"Maximum of Assumed Average Survival Rate: {max_avg_survival_rate}")

# # Assuming market shapes based on a certain distribution
# max_assumed_market_shape = np.max(assumed_market_shape)

# Calculate assumed potential market shape and survival rate
for age in ages:
    assumed_potential_market_shape[age] = (assumed_market_shape[age] / max_assumed_market_shape) #* 100
    
    assumed_potential_survival_rate[age] = (assumed_survival_rate[age] / max_assumed_survival_rate) #* 100
    
    assumed_potential_avg_survival_rate[age] = (avg_survival_rate[age] / max_avg_survival_rate) #* 100

    
for age in ages:

    max_market_estimate[age] = total_population_age_specific[age] * max_assumed_market_shape

    max_survival_estimate[age] = total_population_age_specific[age] * max_assumed_survival_rate

    max_avg_survival_estimate[age] = total_population_age_specific[age] * max_avg_survival_rate

    
# LPA MARKET SHAPE    
# LPAs as % of Population Totals = Assumed Market "Shape"
Assumed_Market_Shape = pd.DataFrame()
Assumed_Market_Shape['age'] = pd.DataFrame(assumed_market_shape.keys())
Assumed_Market_Shape['market_shape'] = pd.DataFrame(assumed_market_shape.values())
LPA_MARKET_SHAPE = Assumed_Market_Shape['market_shape']
print(Assumed_Market_Shape)

Assumed_Potential_Maximum_Market = pd.DataFrame()
Assumed_Potential_Maximum_Market['age'] = pd.DataFrame(assumed_potential_market_shape.keys())
Assumed_Potential_Maximum_Market['market_shape'] = pd.DataFrame(assumed_potential_market_shape.values())
print(Assumed_Potential_Maximum_Market)


# Dictionary for cohort survival rate
survival_rate_dict = {(row['age'], row['year']): row['survival_rate'] for _, row in survival_rate_df.iterrows()}


# print(len(total_population_age_specific))  # Should be 89
# print(len(total_donors_age_specific))      # Should be 89
# print(len(assumed_market_shape))           # Should be 89

# Output the final forecast and results
output_market_data = {
    'age': list(range(18,107)),
    'Total Population': total_population_age_specific.values(),
    'Total Donors': total_donors_age_specific.values(),
    'Assumed Market Shape': LPA_MARKET_SHAPE,
    'Assumed Potential Market Shape': Assumed_Potential_Maximum_Market['market_shape'],
    'Total Surviving Customers': total_surviving_age_specific.values(),
    'Max Market estimates': max_market_estimate.values(),
    'Max Assumed Market Shape': [max_assumed_market_shape] * 89,
    'Assumed Survival Rate': assumed_survival_rate.values(),
    'Assumed Potential Survival Rate': assumed_potential_survival_rate.values(),
    'Max Survival estimates': max_survival_estimate.values(),
    'Max Assumed Survival Rate': [max_assumed_survival_rate] * 89,
    'Average Survival Rate': avg_survival_rate.values(),
    'Total Survival Rate': total_survival_rate_age_specific.values()
}

output_market_df = pd.DataFrame(output_market_data)

# save to csv
output_market_df.to_csv('output_market_data.csv', index=False)

print(output_market_df)

In [ ]:
year_forecast_long = 2039
years_forecast_long = list(range(start_year, year_forecast_long+1))


# MAX MARKET ESTIMATES

MAX_MARKET_ESTIMATES = pd.DataFrame()
MAX_MARKET_ESTIMATES['age'] = output_market_df[['age']] #pd.DataFrame(Assumed_Potential_Maximum_Market.keys())

# Calculate max market estimates in long term (till 2039)
for year in years_forecast_long:
    
    max_market_estimate ={}
    market_shape_percentage = {}
    pop_forecast_ons = 0
    
    for age in range(18, 107):
        
        # Fill the missing pop_forecast_ons for age > 104      ??????????????
        
        pop_forecast_ons = population_forecast_df[((population_forecast_df['age'] == age) & (population_forecast_df['year'] == year))]['population'].values[0]
        market_shape_percentage[age] = (Assumed_Potential_Maximum_Market[Assumed_Potential_Maximum_Market['age'] == age]['market_shape']).values[0]
        max_market_estimate[age] = pop_forecast_ons * market_shape_percentage[age]
    
    # MAX MARKET ESTIMATES
    MAX_MARKET_ESTIMATES[year] = pd.DataFrame(max_market_estimate.values())

print(f"MAX MARKET ESTIMATES") #: {MAX_MARKET_ESTIMATES}")
MAX_MARKET_ESTIMATES.to_csv('MAX_MARKET_ESTIMATES.csv', index=False)     
MAX_MARKET_ESTIMATES

In [ ]:
# REMAINING CUSTOMERS

# For 2007 (the starting year): MAX_MARKET_ESTIMATES[year]
# For year > 2007: = MAX(0, MAX_MARKET_ESTIMATES - LPA SURVIVAL TABLES)

LPA_SURVIVAL_TABLES = surviving_donors_lpa_df
LPA_SURVIVAL_TABLES = LPA_SURVIVAL_TABLES.reset_index()

# List of old and new column names
old_columns = LPA_SURVIVAL_TABLES.columns
new_columns = years

# Creating a renaming dictionary
rename_dict = {old: new for old, new in zip(old_columns, new_columns)}

# Renaming columns
LPA_SURVIVAL_TABLES.rename(columns=rename_dict, inplace=True)

remaining_lpa = {}
remaining_customers = pd.DataFrame()

# Calculate the emaining customers based on MAX_MARKET_ESTIMATES and LPA_SURVIVAL_TABLES
for year in years:
    if year == start_year: # for 2007
        remaining_lpa[year] = MAX_MARKET_ESTIMATES[year]
    else: # year > 2007
        yearly_data = []
        remaining = 0
        for a in range(len(ages)):
            # REMAINING CUSTOMERS: MAX(0, MAX_MARKET_ESTIMATES - LPA_SURVIVAL_TABLES)
            remaining = MAX_MARKET_ESTIMATES[year][a] - LPA_SURVIVAL_TABLES[year][a] if MAX_MARKET_ESTIMATES[year][a] - LPA_SURVIVAL_TABLES[year][a] else 0
            yearly_data.append(remaining) 
        
        #print(yearly_data)
        remaining_lpa[year] = pd.Series(yearly_data)
    
remaining_customers = pd.concat(remaining_lpa.values(), axis=1)              

# If 'age' needs to be added, ensure it exists as a series
remaining_customers['age'] = range(18, 107)  # range for ages 18-106       

# List of old and new column names
old_columns = remaining_customers.columns[:-1]
new_columns = years

# Creating a renaming dictionary
rename_dict = {old: new for old, new in zip(old_columns, new_columns)}

# Renaming columns
remaining_customers.rename(columns=rename_dict, inplace=True)


#remaining_customers = remaining_customers.reset_index()
# Reorder the columns to move 'age' to the first position
first_column = remaining_customers.pop('age')
# Insert 'age' as the first column
remaining_customers.insert(0, 'age', first_column)
print(f"REMAINING CUSTOMERS") #: {remaining_customers}")
remaining_customers.to_csv('REMAINING_CUSTOMERS.csv', index=False) 
REMAINING_CUSTOMERS = remaining_customers
REMAINING_CUSTOMERS
#     pd.DataFrame(yearly_data).set_index(['year', 'age'])        
#     # Concatenate all yearly data into a single DataFrame
#     lpa_surviving_donors_all = pd.concat(lpa_surviving_donors_dict_changed.values(), axis=1)  

In [ ]:
LPA_LIFE_TABLES

In [ ]:

# Forecast for the next 5 years (2025-2029)
forecasted_results = []
iterations = 1000  # example number of iterations

for year in range(2025, 2030):
    for age in range(18, 107):
        cohort_survival_rate = survival_rate_dict.get((age, year), 0)
        
        # Retrieve mean from last two years
        last_year_mean = population_forecast_df[
            (population_forecast_df['year'] == year - 1) & (population_forecast_df['age'] == age)
        ]['Donors'].mean()
        
        second_last_year_mean = population_forecast_df[
            (population_forecast_df['year'] == year - 2) & (population_forecast_df['age'] == age)
        ]['Donors'].mean()
        
        prev_year_mean = (last_year_mean + second_last_year_mean) / 2 if pd.notnull(last_year_mean) and pd.notnull(second_last_year_mean) else 0
        
        # Monte Carlo simulation for forecasting donors
        forecasted_donors = []
        for _ in range(iterations):
            forecasted_value = prev_year_mean * (cohort_survival_rate * max_market_estimate[age])
            forecasted_donors.append(forecasted_value)
        
        forecasted_donors_mean = np.mean(forecasted_donors)
        forecasted_donors_stddev = np.std(forecasted_donors)
        
        forecasted_results.append({
            "Year": year,
            "Age": age,
            "ForecastedDonorsMean": forecasted_donors_mean,
            "ForecastedDonorsStdDev": forecasted_donors_stddev
        })

# Convert results to a DataFrame for analysis or saving
forecasted_results_df = pd.DataFrame(forecasted_results)

# Save forecasted results to Excel
forecasted_results_df.to_excel("ForecastedDonors.xlsx", index=False)

# Visualization using matplotlib
plt.figure(figsize=(10, 6))
plt.plot(forecast_df['Age Group'], forecast_df['Forecast Mean'], label='Forecasted Donors Mean')
plt.fill_between(forecast_df['Age Group'], forecast_df['Forecast Mean'] - forecast_df['Forecast StdDev'], 
                 forecast_df['Forecast Mean'] + forecast_df['Forecast StdDev'], color='lightblue', alpha=0.5)
plt.xlabel('Age Group')
plt.ylabel('Number of Donors')
plt.title('Forecasted Donors for 2024')
plt.legend()
plt.show()



In [ ]:




# Initialise the class with the sample data
forecast_model = LongTermForecastLPA(donors_df, population_forecast_df, survival_rate_df)

# Run the forecast
forecast_result = forecast_model.run_forecast()

# Output the forecast result
print(forecast_result)

# The forecast will generate estimated donor counts for 2024 based on the Monte Carlo simulation, 
# and then extend the forecast for the years 2025-2029 using the cohort survival model. 
# The forecast_result will contain the forecasted values for each age group and year.


In [ ]:
import numpy as np
import pandas as pd

class LongTermForecastLPA:
    def __init__(self, donors_df, population_forecast_df, survival_rate_df):
        # Initialize dataframes
        self.donors_df = donors_df
        self.population_forecast_df = population_forecast_df
        self.survival_rate_df = survival_rate_df

        # Fixed variables
        self.iterations = 1000
        self.lower_bound = 4000
        self.upper_bound = 7000
        self.num_working_days_2024 = 256

        # Fixed total receipts for 2022 and 2023
        self.total_receipts_2022 = 751967.534626039
        self.total_receipts_2023 = 970035.028248588

        # Initialize variables
        self.total_population_age_specific = np.zeros(107)
        self.total_donors_age_specific = np.zeros(107)
        self.assumed_market_shape = np.zeros(107)
        self.assumed_potential_market_shape = np.zeros(107)

    def run_forecast(self):
        # Calculate total demands for 2022 and 2023
        total_demand_2022 = self.calculate_total_demand(2022)
        total_demand_2023 = self.calculate_total_demand(2023)

        # Aggregate age distributions for 2022 and 2023
        age_distribution_2022 = self.calculate_age_distribution(2022, total_demand_2022)
        age_distribution_2023 = self.calculate_age_distribution(2023, total_demand_2023)

        # Average age distribution across 2022 and 2023
        average_age_distribution = self.calculate_average_age_distribution(age_distribution_2022, age_distribution_2023)

        # Monte Carlo Simulation for 2024
        monte_carlo_results = self.monte_carlo_simulation(average_age_distribution, total_demand_2022, total_demand_2023)

        # Output the results to a DataFrame
        result_df = self.output_results(monte_carlo_results, average_age_distribution)

        # Calculate total population and total donors age-specific for 2023 and 2024
        self.calculate_total_population_and_donors()

        # Extend forecast to the next 5 years (2025-2029) using cohort model
        extended_forecast_df = self.extend_forecast(result_df)

        # Combine all results
        final_result_df = pd.concat([result_df, extended_forecast_df], ignore_index=True)

        return final_result_df

    def calculate_total_demand(self, year):
        return self.donors_df[self.donors_df['year'] == year]['demand'].sum()

    def calculate_age_distribution(self, year, total_demand):
        age_distribution = self.donors_df[self.donors_df['year'] == year].groupby('age')['demand'].sum()
        return age_distribution / total_demand

    def calculate_average_age_distribution(self, dist_2022, dist_2023):
        common_ages = dist_2022.index.intersection(dist_2023.index)
        return (dist_2022[common_ages] + dist_2023[common_ages]) / 2

    def monte_carlo_simulation(self, average_age_distribution, total_demand_2022, total_demand_2023):
        monte_carlo_results = []
        ratio_receipt_donor_2022 = self.total_receipts_2022 / total_demand_2022
        ratio_receipt_donor_2023 = self.total_receipts_2023 / total_demand_2023
        avg_ratio = (ratio_receipt_donor_2022 + ratio_receipt_donor_2023) / 2

        for _ in range(self.iterations):
            random_receipt = np.random.randint(self.lower_bound, self.upper_bound)
            total_receipts_2024 = random_receipt * self.num_working_days_2024
            estimated_donors_2024 = total_receipts_2024 / avg_ratio
            monte_carlo_results.append(estimated_donors_2024)

        return monte_carlo_results

    def output_results(self, monte_carlo_results, average_age_distribution):
        result_df = pd.DataFrame(columns=["Year", "Age Group", "Forecast (Mean)", "Forecast (StdDev)"])
        
        for age in average_age_distribution.index:
            sum_values = np.sum(np.array(monte_carlo_results) * average_age_distribution[age])
            sum_squared_values = np.sum((np.array(monte_carlo_results) * average_age_distribution[age]) ** 2)

            forecast_mean = sum_values / self.iterations
            variance = sum_squared_values / self.iterations - (forecast_mean ** 2)
            forecast_stddev = np.sqrt(variance) if variance >= 0 else 0

            result_df = result_df.append({
                "Year": 2024,
                "Age Group": age,
                "Forecast (Mean)": forecast_mean,
                "Forecast (StdDev)": forecast_stddev
            }, ignore_index=True)

        return result_df

    def calculate_total_population_and_donors(self):
        for age in range(18, 107):
            population_2023 = self.population_forecast_df[(self.population_forecast_df['year'] == 2023) & (self.population_forecast_df['age'] == age)]['population'].sum()
            population_2024 = self.population_forecast_df[(self.population_forecast_df['year'] == 2024) & (self.population_forecast_df['age'] == age)]['population'].sum()

            total_population = population_2023 + population_2024
            total_donors = self.donors_df[(self.donors_df['year'] == 2023) & (self.donors_df['age'] == age)]['demand'].sum()

            self.total_population_age_specific[age] = total_population
            self.total_donors_age_specific[age] = total_donors

            self.assumed_market_shape[age] = total_donors / total_population if total_population > 0 else 0

        max_market_shape = np.max(self.assumed_market_shape)
        self.assumed_potential_market_shape = (self.assumed_market_shape / max_market_shape) * 100

    def extend_forecast(self, result_df):
        extended_forecast_df = pd.DataFrame(columns=["Year", "Age Group", "Forecast (Mean)", "Forecast (StdDev)"])

        for year in range(2025, 2030):
            for age in range(18, 107):
                survival_rate = self.get_survival_rate(age, year)
                previous_year_mean = result_df[(result_df['Year'] == 2024) & (result_df['Age Group'] == age)]["Forecast (Mean)"].mean()

                forecast_mean = previous_year_mean * survival_rate
                forecast_stddev = np.std(forecast_mean)  # Approximation for stddev

                extended_forecast_df = extended_forecast_df.append({
                    "Year": year,
                    "Age Group": age,
                    "Forecast (Mean)": forecast_mean,
                    "Forecast (StdDev)": forecast_stddev
                }, ignore_index=True)

        return extended_forecast_df

    def get_survival_rate(self, age, year):
        survival_rate = self.survival_rate_df[(self.survival_rate_df['age'] == age) & (self.survival_rate_df['year'] == year)]['rate']
        return survival_rate.iloc[0] if not survival_rate.empty else 0


In [ ]:
# Sample data for survival_rate_df
survival_rate_data = {
    'age': [30, 50],
    'year': [2025, 2025],
    'survival_rate': [0.95, 0.93]
}
survival_rate_df = pd.DataFrame(survival_rate_data)
survival_rate_df

In [ ]:
population_forecast = pd.read_csv('population_forecast.csv')
# Convert the DataFrame to the desired format
df_melted = population_forecast.melt(id_vars=['age'], var_name='year', value_name='population')
population_forecast_df = pd.DataFrame(df_melted)
population_forecast_df

In [ ]:
remaining_lpa = remaining_lpa_df
survival_tbl = remaining_lpa.reset_index()
survival_tbl = survival_tbl.rename(columns={'age':'age'})
# Convert the DataFrame to the desired format
df_melted = survival_tbl.melt(id_vars=['age'], var_name='year', value_name='remaining')
survival_df = pd.DataFrame(df_melted)
survival_df

In [ ]:
population_forecast_df = pd.read_csv('population_forecast.csv')
population_forecast_df

In [ ]:
survival_rate_df = remaining_lpa_df.reset_index()
survival_rate_df

In [ ]:
Sub LongTermForecast_Market_Survival()
    ' Define variables
    Dim ws As Worksheet
    Set ws = ThisWorkbook.Sheets("annual_unique_donors (2)")
    Dim lastRow As Long
    lastRow = ws.Cells(ws.Rows.count, "A").End(xlUp).row
    Dim iterations As Integer
    iterations = 1000 ' Number of iterations for Monte Carlo simulation
    Dim lowerBound As Double, upperBound As Double
    lowerBound = 4000
    upperBound = 7000
    Dim i As Long, j As Long, row As Long
    Dim randomReceipt As Double
    Dim estimatedDonors2024 As Double
    Dim monteCarloResults() As Double
    ReDim monteCarloResults(1 To iterations)
    Dim ageDistribution2022 As Object, ageDistribution2023 As Object, averageAgeDistribution As Object
    Dim totalDemand2022 As Double, totalDemand2023 As Double
    Dim totalReceipts2022 As Double, totalReceipts2023 As Double
    Dim totalReceipts2024 As Double
    Dim ratioReceiptDonor2022 As Double, ratioReceiptDonor2023 As Double
    Dim numWorkingDays2024 As Long
    Dim totalPopulationAgeSpecific(106) As Double, totalDonorsAgeSpecific(106) As Double
    Dim assumedMarketShape(106) As Double, assumedPotentialMarketShape(106) As Double
    Dim maxAssumedMarketShape As Double
    Dim prevYearsPop(4) As Double
    numWorkingDays2024 = 256
    ' Fixed total receipts for 2022 and 2023
    totalReceipts2022 = 751967.534626039
    totalReceipts2023 = 970035.028248588
    ' Calculate total demands for 2022 and 2023
    totalDemand2022 = Application.WorksheetFunction.SumIf(ws.Range("A2:A" & lastRow), 2022, ws.Range("C2:C" & lastRow))
    totalDemand2023 = Application.WorksheetFunction.SumIf(ws.Range("A2:A" & lastRow), 2023, ws.Range("C2:C" & lastRow))
    ' Aggregate age distributions for 2022 and 2023
    Set ageDistribution2022 = CreateObject("Scripting.Dictionary")
    Set ageDistribution2023 = CreateObject("Scripting.Dictionary")
    For i = 2 To lastRow
        If ws.Cells(i, 1).Value = 2022 Then
            If ageDistribution2022.Exists(ws.Cells(i, 2).Value) Then
                ageDistribution2022(ws.Cells(i, 2).Value) = ageDistribution2022(ws.Cells(i, 2).Value) + ws.Cells(i, 3).Value
            Else
                ageDistribution2022.Add ws.Cells(i, 2).Value, ws.Cells(i, 3).Value
            End If
        ElseIf ws.Cells(i, 1).Value = 2023 Then
            If ageDistribution2023.Exists(ws.Cells(i, 2).Value) Then
                ageDistribution2023(ws.Cells(i, 2).Value) = ageDistribution2023(ws.Cells(i, 2).Value) + ws.Cells(i, 3).Value
            Else
                ageDistribution2023.Add ws.Cells(i, 2).Value, ws.Cells(i, 3).Value
            End If
        End If
    Next i
    ' Normalize age distribution
    For Each age In ageDistribution2022.Keys
        ageDistribution2022(age) = ageDistribution2022(age) / totalDemand2022
    Next age
    For Each age In ageDistribution2023.Keys
        ageDistribution2023(age) = ageDistribution2023(age) / totalDemand2023
    Next age
    ' Average age distribution across 2022 and 2023
    Set averageAgeDistribution = CreateObject("Scripting.Dictionary")
    For Each age In ageDistribution2022.Keys
        If ageDistribution2023.Exists(age) Then
            averageAgeDistribution.Add age, (ageDistribution2022(age) + ageDistribution2023(age)) / 2
        End If
    Next age
    ' Monte Carlo Simulation for 2024
    For i = 1 To iterations
        randomReceipt = Application.WorksheetFunction.RandBetween(lowerBound, upperBound)
        totalReceipts2024 = randomReceipt * numWorkingDays2024
        ratioReceiptDonor2022 = totalReceipts2022 / totalDemand2022
        ratioReceiptDonor2023 = totalReceipts2023 / totalDemand2023
        estimatedDonors2024 = totalReceipts2024 / ((ratioReceiptDonor2022 + ratioReceiptDonor2023) / 2)
        monteCarloResults(i) = estimatedDonors2024
    Next i 
    Dim outputWs As Worksheet
    ' Output the results to the "LongTermForecast_market" sheet
    On Error Resume Next
    Set outputWs = ThisWorkbook.Sheets("LongTermForecastMarket")
    If Not outputWs Is Nothing Then
        outputWs.Cells.Clear
        For Each chartObj In outputWs.ChartObjects
            chartObj.Delete
        Next chartObj
    Else
        Set outputWs = ThisWorkbook.Sheets.Add(After:=ThisWorkbook.Sheets(ThisWorkbook.Sheets.count))
        outputWs.Name = "LongTermForecastMarket"
    End If
    On Error GoTo 0
    ' Output column headers
    outputWs.Cells(1, 1).Value = "Year"
    outputWs.Cells(1, 2).Value = "Age Group"
    outputWs.Cells(1, 3).Value = "Age Distribution 2022"
    outputWs.Cells(1, 4).Value = "Age Distribution 2023"
    outputWs.Cells(1, 5).Value = "Random Receipt"
    outputWs.Cells(1, 6).Value = "Final Forecast (Mean)"
    outputWs.Cells(1, 7).Value = "Final Forecast (StdDev)"
    ' Add data for each age group for 2024
    row = 2
    For Each age In averageAgeDistribution.Keys
        sumValues = 0
        sumSquaredValues = 0
        For i = 1 To iterations
            sumValues = sumValues + monteCarloResults(i) * averageAgeDistribution(age)
            sumSquaredValues = sumSquaredValues + (monteCarloResults(i) * averageAgeDistribution(age)) ^ 2
        Next i
        ' Calculate mean and standard deviation for the final forecast
        forecastedDonorsMean = sumValues / iterations
        forecastedDonorsStdDev = Sqr(sumSquaredValues / iterations - (sumValues / iterations) ^ 2)
        ' Output the data for the current age group
        outputWs.Cells(row, 1).Value = 2024
        outputWs.Cells(row, 2).Value = age
        outputWs.Cells(row, 3).Value = ageDistribution2022(age)
        outputWs.Cells(row, 4).Value = ageDistribution2023(age)
        outputWs.Cells(row, 5).Value = randomReceipt ' Random Receipt output only once per age group
        outputWs.Cells(row, 6).Value = forecastedDonorsMean
        outputWs.Cells(row, 7).Value = forecastedDonorsStdDev
        row = row + 1
    Next age
    finalYearLastRow = outputWs.Cells(outputWs.Rows.count, "A").End(xlUp).row
    ' Calculate total population and total donors age-specific for 2023 and 2024
    Dim populationForecastWs As Worksheet
    Set populationForecastWs = ThisWorkbook.Sheets("UnpivotPopulationForecast")
    Dim popLastRow As Long
    popLastRow = populationForecastWs.Cells(populationForecastWs.Rows.count, "A").End(xlUp).row 
    For age = 18 To 106
        totalPopulationAgeSpecific(age) = Application.WorksheetFunction.SumIfs(populationForecastWs.Range("C2:C" & popLastRow), _
            populationForecastWs.Range("B2:B" & popLastRow), age, populationForecastWs.Range("A2:A" & popLastRow), 2023) + _
            Application.WorksheetFunction.SumIfs(populationForecastWs.Range("C2:C" & popLastRow), _
            populationForecastWs.Range("B2:B" & popLastRow), age, populationForecastWs.Range("A2:A" & popLastRow), 2024)
            
        totalDonorsAgeSpecific(age) = Application.WorksheetFunction.SumIf(ws.Range("A2:A" & lastRow), 2023, ws.Range("C2:C" & lastRow)) + _
                              Application.WorksheetFunction.SumIf(outputWs.Range("A2:A" & finalYearLastRow), 2024, outputWs.Range("F2:F" & finalYearLastRow))     
        assumedMarketShape(age) = totalDonorsAgeSpecific(age) / totalPopulationAgeSpecific(age)
    Next age
    ' Find the max assumed market shape and corresponding age
    maxAssumedMarketShape = Application.WorksheetFunction.Max(assumedMarketShape)
    ' Calculate assumed potential market shape
    For age = 18 To 106
        assumedPotentialMarketShape(age) = assumedMarketShape(age) / maxAssumedMarketShape * 100
    Next age
    ' Calculate MAX MARKET ESTIMATES:
    'Calculate maxMarketEstimate by multiplying totalPopulationAgeSpecific(age) by assumedPotentialMarketShape(age) for each age
    For age = 18 To 106
        maxMarketEstimate(age) = totalPopulationAgeSpecific(age) * maxAssumedMarketShape
    Next age
    ' LPA SURVIVAL TABLES (BY YEAR OF APPLICATION):
    
    ' Extend forecast to the next 5 years (2025-2029) using cohort model
    Dim year As Long
    Dim cohortSurvivalRate As Double
    Dim survivalRateWs As Worksheet
    Set survivalRateWs = ThisWorkbook.Sheets("UnpivotSurvivalRate")
    Dim srLastRow As Long
    srLastRow = survivalRateWs.Cells(survivalRateWs.Rows.count, "A").End(xlUp).row    
    Dim dict As Object
    Set dict = CreateObject("Scripting.Dictionary")    
    ' Load data into the dictionary
    For i = 2 To srLastRow
        Key = survivalRateWs.Cells(i, 1).Value & "-" & survivalRateWs.Cells(i, 2).Value
        Value = survivalRateWs.Cells(i, 3).Value
        dict(Key) = Value
    Next i    
    For year = 2025 To 2029
        For age = 18 To 106
            sumValues = 0
            sumSquaredValues = 0    
            ' Retrieve the survival rate
            Key = age & "-" & year
            If dict.Exists(Key) Then
                cohortSurvivalRate = dict(Key)
            Else
                cohortSurvivalRate = 0
            End If    
            ' Calculate prevYearMean as the average of the values from the last two years
            Dim lastYearMean As Double, secondLastYearMean As Double
            Dim lastYearRow As Range, secondLastYearRow As Range    
            ' Find the row for the last year (year - 1) for the current age
            Set lastYearRow = outputWs.Columns(2).Find(What:=age, After:=outputWs.Cells(1, 2), LookIn:=xlValues, LookAt:=xlWhole, SearchOrder:=xlByRows, SearchDirection:=xlPrevious, MatchCase:=False)
            If Not lastYearRow Is Nothing Then
                lastYearMean = lastYearRow.Offset(0, 4).Value
            Else
                lastYearMean = 0
            End If    
            ' Find the row for the second last year (year - 2) for the current age
            Set secondLastYearRow = outputWs.Columns(2).Find(What:=age, After:=lastYearRow, LookIn:=xlValues, LookAt:=xlWhole, SearchOrder:=xlByRows, SearchDirection:=xlPrevious, MatchCase:=False)
            If Not secondLastYearRow Is Nothing Then
                secondLastYearMean = secondLastYearRow.Offset(0, 4).Value
            Else
                secondLastYearMean = 0
            End If    
            ' Calculate the average of the last two years
            prevYearMean = (lastYearMean + secondLastYearMean) / 2    
            ' Apply assumedPotentialMarketShape and cohortSurvivalRate in the forecast
            For i = 1 To iterations
                ' Apply cohort survival rate and assumed potential market shape
                forecastedDonors = prevYearMean * (cohortSurvivalRate * maxMarketEstimate(age)))   
                sumValues = sumValues + forecastedDonors
                sumSquaredValues = sumSquaredValues + forecastedDonors ^ 2
            Next i   
            ' Calculate mean and standard deviation for the forecasted donors
            forecastedDonorsMean = sumValues / iterations
            Dim variance As Double
            variance = sumSquaredValues / iterations - (sumValues / iterations) ^ 2   
            If variance >= 0 Then
                forecastedDonorsStdDev = Sqr(variance)
            Else
                forecastedDonorsStdDev = 0
            End If    
            ' Output the data for the current year and age group
            outputWs.Cells(row, 1).Value = year
            outputWs.Cells(row, 2).Value = age
            outputWs.Cells(row, 3).Value = "N/A"
            outputWs.Cells(row, 4).Value = "N/A"
            outputWs.Cells(row, 5).Value = randomReceipt
            outputWs.Cells(row, 6).Value = forecastedDonorsMean
            outputWs.Cells(row, 7).Value = forecastedDonorsStdDev
            row = row + 1
        Next age
    Next year 
    ' Virtulisation
    'Dim chartObj As ChartObject
    'Dim ws As Worksheet
    'Set outputWs = ThisWorkbook.Sheets("LongTermForecastMarket")
    ' Clear any existing charts
    For Each chartObj In outputWs.ChartObjects
        chartObj.Delete
    Next chartObj
    ' Define the last row for the data
    'Dim lastRow As Long
    lastRow = outputWs.Cells(outputWs.Rows.count, 1).End(xlUp).row
    ' 1. Line Chart: Forecasted Donors Over Time
    Set chartObj = outputWs.ChartObjects.Add(Left:=50, Width:=500, Top:=50, Height:=300)
    With chartObj.chart
        .SetSourceData Source:=outputWs.Range("A1:A" & lastRow & ", F1:F" & lastRow)
        .ChartType = xlLine
        .HasTitle = True
        .ChartTitle.Text = "Forecasted Donors Over Time"
        .Axes(xlCategory, xlPrimary).HasTitle = True
        .Axes(xlCategory, xlPrimary).AxisTitle.Text = "Year"
        .Axes(xlValue, xlPrimary).HasTitle = True
        .Axes(xlValue, xlPrimary).AxisTitle.Text = "Number of Donors"
    End With
    ' 2. Bar Chart: Age Distribution 2022 and 2023
    Set chartObj = outputWs.ChartObjects.Add(Left:=600, Width:=500, Top:=50, Height:=300)
    With chartObj.chart
        .SetSourceData Source:=outputWs.Range("B1:B" & lastRow & ", C1:D" & lastRow)
        .ChartType = xlColumnClustered
        .HasTitle = True
        .ChartTitle.Text = "Age Distribution 2022 vs 2023"
        .Axes(xlCategory, xlPrimary).HasTitle = True
        .Axes(xlCategory, xlPrimary).AxisTitle.Text = "Age Group"
        .Axes(xlValue, xlPrimary).HasTitle = True
        .Axes(xlValue, xlPrimary).AxisTitle.Text = "Distribution"
    End With
    ' 3. Cluster Chart: Comparing Forecasts Across Different Years
    Set chartObj = outputWs.ChartObjects.Add(Left:=50, Width:=500, Top:=400, Height:=300)
    With chartObj.chart
        .SetSourceData Source:=outputWs.Range("A1:A" & lastRow & ", F1:F" & lastRow)
        .ChartType = xlColumnClustered
        .HasTitle = True
        .ChartTitle.Text = "Forecasted Donors Comparison"
        .Axes(xlCategory, xlPrimary).HasTitle = True
        .Axes(xlCategory, xlPrimary).AxisTitle.Text = "Year"
        .Axes(xlValue, xlPrimary).HasTitle = True
        .Axes(xlValue, xlPrimary).AxisTitle.Text = "Number of Donors"
    End With
    ' 4. Heatmap: Assumed Market Shape by Age
    ' Create a new sheet for the heatmap
    Dim heatmapWs As Worksheet
    On Error Resume Next
    Set heatmapWs = ThisWorkbook.Sheets("MarketShapeHeatmap")
    If heatmapWs Is Nothing Then
        Set heatmapWs = ThisWorkbook.Sheets.Add(After:=ThisWorkbook.Sheets(ThisWorkbook.Sheets.count))
        heatmapWs.Name = "MarketShapeHeatmap"
    End If
    On Error GoTo 0
    ' Populate the heatmap data
    heatmapWs.Cells.Clear
    heatmapWs.Cells(1, 1).Value = "Age Group"
    heatmapWs.Cells(1, 2).Value = "Market Shape (%)"
    For age = 18 To 106
        heatmapWs.Cells(age - 16, 1).Value = age
        heatmapWs.Cells(age - 16, 2).Value = assumedPotentialMarketShape(age)
    Next age
    ' Add conditional formatting to create a heatmap
    With heatmapWs.Range("B2:B91")
        .FormatConditions.AddColorScale ColorScaleType:=3
        .FormatConditions(1).ColorScaleCriteria(1).Type = xlConditionValueLowestValue
        .FormatConditions(1).ColorScaleCriteria(1).FormatColor.Color = RGB(255, 255, 255)
        .FormatConditions(1).ColorScaleCriteria(2).Type = xlConditionValuePercentile
        .FormatConditions(1).ColorScaleCriteria(2).Value = 50
        .FormatConditions(1).ColorScaleCriteria(2).FormatColor.Color = RGB(255, 165, 0)
        .FormatConditions(1).ColorScaleCriteria(3).Type = xlConditionValueHighestValue
        .FormatConditions(1).ColorScaleCriteria(3).FormatColor.Color = RGB(255, 0, 0)
    End With
    
    ' Optional: Automatically resize columns for better readability
    heatmapWs.Columns("A:B").AutoFit
End Sub

In [ ]:
#%%
# This script is designed to be modular, allowing you to call specific methods to perform different parts of the analysis.
# split the code into three separate classes:

# DataHandler: For handling file operations and initial data loading.
# DataPreprocessor: For preprocessing the data and performing calculations on the data.
# DataAnalyzer: For analyzing the data and Bayesian modeling.

# Load and Preprocess Data:

# Extract relevant columns.
# Handle missing values.
# Calculate age from the date of birth and receipt date.
# Filter data by year and age range.
# Calculate Age-Specific Demand:

# Group data by year, gender, and age.
# Calculate the number of unique donors.
# Calculate Annual Demands Ratio:

# Use the annual average daily application receipts.
# Apply the annual demands ratio to incorporate uncertainty.
# Forecast Age-Specific Demand:

# Use the Bayesian model to forecast future demands.
# Adjust the model to account for age-specific data.

# use the Pymc Python package and Bayesian criteria while using Monte Carlo simulation for time series forecasting of donor application demands to 
# account for uncertainty and make more robust predictions, especially in scenarios with significant disruptions like post-COVID impacts after March 2021, 
# by having age-specific demand data (column names: year, gender, age, number of unique donors) from 2006 to 2024? Also, how do you make sure that, 
# from the actual data, you choose the right prior based on the distribution of donor demands per year and fluctuation before and after COVID? 
# to learn from the pre-COVID data and post-COVID data to decide which distribution should be used for the priors for pre-COVID and post-COVID parameters? 
# In addition, imagine we already have calculated average annual daily application receipts and then you can use average annual daily application receipts 
# and annual demands ratio (without considering age) to deal with uncertainty to improve long term forecasting of  donor application demands?


## -------------------
#%%



## -------------------
#%%

#%%


#%%
## ------------------Step 3: DataAnalyzer Class-------------------- ##

class DataAnalyzer:
    def __init__(self, data):
        self.data = data

    def exploratory_data_analysis(self, data):
        data.reset_index()
        data.set_index('month_year', inplace=True)
        data['demand'].plot(figsize=(14, 7))
        plt.title('Donor Application Demands')
        plt.xlabel('Month-Year')
        plt.ylabel('Demand')
        plt.show()

        pre_covid = data.loc[:'Mar-21', 'demand']
        post_covid = data.loc['Mar-21':, 'demand']

        plt.figure(figsize=(14, 7))
        sns.histplot(pre_covid, kde=True, label='Pre-COVID', color='blue', bins=50)
        sns.histplot(post_covid, kde=True, label='Post-COVID', color='red', bins=50)
        plt.legend()
        plt.title('Distribution of Donor Application Demands')
        plt.xlabel('Demand')
        plt.ylabel('Frequency')
        plt.show()

    def determine_priors(self, pre_covid, post_covid):
        def test_distribution(data, dist, *args):
            return kstest(data, dist, args=args)

        print("Pre-COVID Data:")
        print("Normal:", test_distribution(pre_covid, 'norm', np.mean(pre_covid), np.std(pre_covid)))
        print("Log-Normal:", test_distribution(pre_covid, 'lognorm', np.std(np.log(pre_covid[pre_covid > 0])), 0, np.exp(np.mean(np.log(pre_covid[pre_covid > 0])))))
        print("Student's t:", test_distribution(pre_covid, 't', len(pre_covid)-1, np.mean(pre_covid), np.std(pre_covid)))

        print("\nPost-COVID Data:")
        print("Normal:", test_distribution(post_covid, 'norm', np.mean(post_covid), np.std(post_covid)))
        print("Log-Normal:", test_distribution(post_covid, 'lognorm', np.std(np.log(post_covid[post_covid > 0])), 0, np.exp(np.mean(np.log(post_covid[post_covid > 0])))))
        print("Student's t:", test_distribution(post_covid, 't', len(post_covid)-1, np.mean(post_covid), np.std(post_covid)))

    def bayesian_modeling(self, pre_covid, post_covid):
        model = pm.Model()
        with model:
            pre_covid_mean = pm.Normal('pre_covid_mean', mu=np.mean(pre_covid), sigma=np.std(pre_covid))
            pre_covid_sd = pm.HalfNormal('pre_covid_sd', sigma=1)
            post_covid_mean = pm.Normal('post_covid_mean', mu=np.mean(post_covid), sigma=np.std(post_covid))
            post_covid_sd = pm.HalfNormal('post_covid_sd', sigma=1)

            pre_covid_likelihood = pm.Normal('pre_covid_likelihood', mu=pre_covid_mean, sigma=pre_covid_sd, observed=pre_covid)
            post_covid_likelihood = pm.Normal('post_covid_likelihood', mu=post_covid_mean, sigma=post_covid_sd, observed=post_covid)

            trace = pm.sample(1000, tune=2000, return_inferencedata=True)
        
        az.plot_trace(trace)
        plt.show()
        
        return trace
    

data = lpa_data
print(data.head(2))
# Instantiate the DataAnalyzer
data_analyzer = DataAnalyzer(data)
data_analyzer.exploratory_data_analysis(data)
pre_covid, post_covid = data.loc[:'Mar-21', 'demand'], data.loc['Mar-21':, 'demand']
data_analyzer.determine_priors(pre_covid, post_covid)
trace= data_analyzer.bayesian_modeling(pre_covid, post_covid)
## ------------------


# %%

## ------------------Step 4: forecasting-------------------- ##
post_covid_data = data.loc[:'Mar-21', 'demand'], data.loc['Mar-21':, 'demand']



## ------------------

    
# %%

az.style.use("arviz-darkgrid")

RANDOM_SEED = 58
rng = np.random.default_rng(RANDOM_SEED)

model = pm.Model()
with model:
    pre_covid_mean = pm.Normal('pre_covid_mean', mu=np.mean(pre_covid), sigma=np.std(pre_covid))
    pre_covid_sd = pm.HalfNormal('pre_covid_sd', sigma=1)
    post_covid_mean = pm.Normal('post_covid_mean', mu=np.mean(post_covid), sigma=np.std(post_covid))
    post_covid_sd = pm.HalfNormal('post_covid_sd', sigma=1)

    pre_covid_likelihood = pm.Normal('pre_covid_likelihood', mu=pre_covid_mean, sigma=pre_covid_sd, observed=pre_covid)
    post_covid_likelihood = pm.Normal('post_covid_likelihood', mu=post_covid_mean, sigma=post_covid_sd, observed=post_covid)

    trace = pm.sample(1000, tune=2000, return_inferencedata=True)
    pm.sample_posterior_predictive(trace, extend_inferencedata=True, random_seed=rng)
    
print(trace.posterior_predictive)   #

az.plot_ppc(trace, num_pp_samples=100); 


# %%

## ------------------Step 5: Monte Carlo simulations for forecasting-------------------- ##
pre_covid_data, post_covid_data = data.loc[:'Mar-21', 'demand'], data.loc['Mar-21':, 'demand']

trace = data_analyzer.bayesian_modeling(pre_covid_data, post_covid_data)
pre_covid_data, post_covid_data = pre_covid.reset_index(), post_covid.reset_index() 
#trace = data_analyzer.bayesian_modeling(pre_covid_data, post_covid_data)
trace['pre_covid_pred'], trace['post_covid_pred'] = pre_covid_likelihood, post_covid_likelihood

with pm.Model() as model:
    post_pred = pm.sample_posterior_predictive(trace, var_names=['pre_covid_pred', 'post_covid_pred'])

# Plot the results
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
axes[0].plot(pre_covid_data['month_year'], pre_covid_data['demand'], label='Observed Pre-COVID')
axes[0].plot(pre_covid_data['month_year'], post_pred['pre_covid_pred'].mean(axis=0), label='Predicted Pre-COVID Mean')
axes[0].fill_between(pre_covid_data['month_year'], np.percentile(post_pred['pre_covid_pred'], 5, axis=0), 
                     np.percentile(post_pred['pre_covid_pred'], 95, axis=0), alpha=0.2, label='95% CI')
axes[0].legend()
axes[0].set_title('Pre-COVID Predictions')

axes[1].plot(post_covid_data['month_year'], post_covid_data['demand'], label='Observed Post-COVID')
axes[1].plot(post_covid_data['month_year'], post_pred['post_covid_pred'].mean(axis=0), label='Predicted Post-COVID Mean')
axes[1].fill_between(post_covid_data['month_year'], np.percentile(post_pred['post_covid_pred'], 5, axis=0), 
                     np.percentile(post_pred['post_covid_pred'], 95, axis=0), alpha=0.2, label='95% CI')
axes[1].legend()
axes[1].set_title('Post-COVID Predictions')

plt.tight_layout()
plt.show()
## ------------------
## ------------------


# %%


## ------------------Step 6: Specify the Model with PyMC-------------------- ##

# Convert data to numpy array for PyMC
demand_data = data['demand'].values

# Define a Bayesian model with potential structural break after March 2021
with pm.Model() as model:
    # Priors for pre-COVID parameters (Normal distribution)
    mu_pre = pm.Normal('mu_pre', mu=np.mean(pre_covid), sigma=np.std(pre_covid))
    sigma_pre = pm.HalfNormal('sigma_pre', sigma=np.std(pre_covid))
    
    # Priors for post-COVID parameters (Normal distribution)
    mu_post = pm.Normal('mu_post', mu=np.mean(post_covid), sigma=np.std(post_covid))
    sigma_post = pm.HalfNormal('sigma_post', sigma=np.std(post_covid))
    
    # Indicator for structural break
    switchpoint = len(pre_covid)
    idx = np.arange(len(demand_data))
    
    # Likelihood
    mu = pm.math.switch(idx < switchpoint, mu_pre, mu_post)
    sigma = pm.math.switch(idx < switchpoint, sigma_pre, sigma_post)
    
    y = pm.Normal('y', mu=mu, sigma=sigma, observed=demand_data)




## -----------------
#%%

## ------------------Step 5: Fit the Model Using MCMC-------------------- ##

with model:
    trace = pm.sample(2000, tune=1000, return_inferencedata=True)

# Inspect the trace
az.plot_trace(trace)
plt.show()



## -----------------
#%%


## ------------------Step 6: Posterior Predictive Checks-------------------- ##

with model:
    ppc = pm.sample_posterior_predictive(trace, var_names=['y'])

# Simulate future demands
n_future = 365  # Simulate for one year
future_simulations = []

for sample in ppc['y']:
    future_path = np.concatenate([demand_data, np.zeros(n_future)])
    for t in range(len(demand_data), len(demand_data) + n_future):
        future_path[t] = sample[t - len(demand_data)] + future_path[t - 1]
    future_simulations.append(future_path[-n_future:])

future_simulations = np.array(future_simulations)

# Analyze future simulations
mean_forecast = np.mean(future_simulations, axis=0)
cred_interval = np.percentile(future_simulations, [2.5, 97.5], axis=0)


## -----------------
#%%

## ------------------Step 7: Analyse and Visualize Results-------------------- ##

plt.figure(figsize=(14, 7))
plt.plot(data.index, demand_data, label='Historical Data')
plt.plot(pd.date_range(data.index[-1], periods=n_future, freq='D'), mean_forecast, label='Mean Forecast')
plt.fill_between(pd.date_range(data.index[-1], periods=n_future, freq='D'), cred_interval[0], cred_interval[1], color='gray', alpha=0.5, label='95% Credible Interval')
plt.axvline(data.index[switchpoint], color='r', linestyle='--', label='Post-COVID Impact')
plt.legend()
plt.title('Forecast of Daily Application Demands')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.show()


## -----------------

import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split

class LPAForecast:
    def __init__(self, post_covid_data, covid_data, historical_data): #, economic_data):
        self.post_covid_data = post_covid_data
        self.covid_data = covid_data
        #self.economic_data = economic_data
        self.historical_data = historical_data
        
    def preprocess_data(self):
        # Merge historical data with COVID-19 impact data and economic data
        self.data = historical_data #pd.merge(self.post_covid_data, self.covid_data, on='date', how='left')
        #self.data = pd.merge(self.data, self.economic_data, on='date', how='left')
        self.data.fillna(0, inplace=True)

    def short_term_forecast(self, order=(5,1,0)):
        # Split data into training and testing sets
        train, test = train_test_split(self.data, test_size=0.2, shuffle=False)
        
        # Fit ARIMA model
        model = ARIMA(train['demand'], order=order, exog=train[['covid_impact']]) #, 'economic_indicator']])
        self.model_fit = model.fit()
        
        # Forecast
        self.short_term_predictions = self.model_fit.forecast(steps=len(test), exog=test[['covid_impact']]) #, 'economic_indicator']])
        #baseline_forecast = self.model_fit.forecast(steps=24)  # Assuming 2 years monthly forecast
        self.test = test

    def quantify_uncertainty(self):
        # Calculate confidence intervals
        self.conf_int = self.model_fit.get_forecast(steps=len(self.test), exog=self.test[['covid_impact']]).conf_int() #, 'economic_indicator']]).conf_int()
        #self.conf_int = self.model_fit.get_forecast(steps=len(self.test)).conf_int()
        
    def long_term_forecast(self, adjustment_factor=1.2):
        # Baseline forecast using historical trend
        #future_economic_data = self.economic_data.tail(24)  # Assuming 2 years monthly forecast
        baseline_forecast = self.model_fit.forecast(steps=24, exog=future_economic_data[['covid_impact']]) #, 'economic_indicator']])
        #baseline_forecast = self.model_fit.forecast(steps=24)  # Assuming 2 years monthly forecast
        
        # Adjust for COVID-19 impact
        self.long_term_predictions = baseline_forecast * adjustment_factor

        
# Write a function to apply exponential smoothing applied to the age-specific short-term forecasts to incorporate covid impact and do sentivitity tests 
# follow the excel formula in LPA model and control assumption:
# =GB9*'LPA Control Assumptions'!$B$123+(1-'LPA Control Assumptions'!$B$123)*(GB9-GA9+GB9)
    #def age_specific_uncertainty(self)
    
    
    def forecast_by_age_group(self, age_groups):
        self.age_group_forecasts = {}
        for age_group in age_groups:
            # Filter data for the age group
            age_group_data = self.data[self.data['age_group'] == age_group]
            
            # Fit ARIMA model
            model = ARIMA(age_group_data['demand'], order=(5,1,0), exog=age_group_data[['covid_impact']]) #, 'economic_indicator']])
            model_fit = model.fit()
            
            # Forecast
            forecast = model_fit.forecast(steps=24, exog=future_economic_data[['covid_impact']]) #, 'economic_indicator']])
            self.age_group_forecasts[age_group] = forecast
#         for age_group in age_groups:
#             # Filter data for the age group
#             age_group_data = self.data[self.data['age_group'] == age_group]
#             # Fit ARIMA model
#             model = ARIMA(age_group_data['registrations'], order=(5,1,0))
#             model_fit = model.fit()
#             # Forecast
#             forecast = model_fit.forecast(steps=24)
#             self.age_group_forecasts[age_group] = forecast
    def validate_model(self):
        # Compare predictions with actual values
        self.validation_results = self.test['demand'] - self.short_term_predictions

    def aggregate_by_age_group(self, bins, labels):
        # Create a new column 'AgeGroup' with the age bins
        self.data['AgeGroup'] = pd.cut(self.data['age'], bins=bins, labels=labels, right=False)
        
        # Aggregate data within each age group
        aggregated_data = self.data.groupby('AgeGroup').agg({
            'demand': 'sum',
            'name': 'count'
        }).rename(columns={'name': 'count'})
        
        return aggregated_data

    def run_forecast(self, age_groups, bins, labels):
        self.preprocess_data()
        self.short_term_forecast()
        self.quantify_uncertainty()
        self.long_term_forecast()
        self.forecast_by_age_group(age_groups)
        self.validate_model()
        return self.aggregate_by_age_group(bins, labels)

# usage
#historical_data = pd.read_csv('historical_data.csv')  # Replace with actual data source
#covid_data = pd.read_csv('covid_data.csv')  # Replace with actual data source
#economic_data = pd.read_csv('economic_data.csv')  # Replace with actual data source
age_groups = ['18-20', '21-25', '26-30', '31-35', '36-40','41-45','46-50', '51-55', '56-60', '61-65', '66-70', '71+']
bins = [18, 20, 30, 40, 50, 60, 70, 100]
labels = ['18-20', '21-25', '26-30', '31-35', '36-40','41-45','46-50', '51-55', '56-60', '61-65', '66-70', '71+']

forecast = LPAForecast(post_covid_data, covid_data, historical_data)# , economic_data)
aggregated_data = forecast.run_forecast(age_groups, bins, labels)

print(aggregated_data)

#age_groups = ['18-30', '31-50', '51-70', '71+']

#forecast = LPAForecast(post_covid_data, covid_data)
#forecast.run_forecast(age_groups)
#print(forecast.long_term_predictions)
#print(forecast.age_group_forecasts)


In [ ]:
# Create a sample DataFrame
data = {'Column1': [1, 2, 3],
        'Column2': ['A', 'B', 'C']}

df = pd.DataFrame(data)

# Define a custom function to combine columns
def combine_columns(row):
    return str(row['Column1']) + row['Column2']

# Apply the custom function to create a new column 'Combined'
df['Combined'] = df.apply(combine_columns, axis=1)

In [ ]:
# Create a pivot table to compare the distribution
pivot_annual_unique_donors = pd.pivot_table(annual_unique_donors, 
                             values='demand', 
                             index=['year'])#, # ['Age Group 2017', 'Age Group Today'],
                             #aggfunc='sum') # count

pivot_annual_unique_donors.reset_index(inplace=True)
pivot_annual_unique_donors.columns = ['year', 'age'] # 'Age Group 2017', 'Age Group Today', 'Number of Individuals']

pivot_annual_unique_donors

In [ ]:

        lx = life_table.loc[(life_table['year'] == year) & (life_table['age'] == age), 'lx'].values[0]
        initial_customers = customers_df.loc[customers_df['year'] == cohort_year, 'Age 18'].values[0]
        lpa_surviving_donors_age = initial_customers * (lx / initial_lx)
        lpa_surviving_donors_year[f'Age {age}'] = lpa_surviving_donors_age
        total_lpa_surviving_donors += lpa_surviving_donors_age

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Mortality rates for ages 45 to 49 (example values)
qx_45_49 = qx.iloc['age' in range(45,50),:]
mortality_rates_45_49 = qx_45_49['qx'] #np.array([rate_45, rate_46, rate_47, rate_48, rate_49])
mortality_rates_45_49

# Step 1: Calculate the logarithm of the average mortality rate for ages 45-49
log_avg_mortality = np.log(np.mean(mortality_rates_45_49))
log_avg_mortality

# Ages and their corresponding logarithms of mortality rates (example values)
ages = np.array([49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94])
log_mortality_rates = np.array([...])  # Fill in the logarithms of mortality rates for these ages
log_mortality_rates

# Step 1: Load the qx.csv file
file_path = 'qx.csv'
df = pd.read_csv(file_path)

# Step 2: Define the age groups
age_groups = {
    "45-49": range(45, 50),
    "50-54": range(50, 55),
    "55-59": range(55, 60),
    "60-64": range(60, 65),
    "65-69": range(65, 70),
    "70-74": range(70, 75),
    "75-79": range(75, 80),
    "80-84": range(80, 85),
    "85-89": range(85, 90),
}

# Step 3: Calculate the average mortality rates for each age group
average_mortality_rates = {}
for group, ages in age_groups.items():
    average_mortality_rates[group] = df[df['age'].isin(ages)]['qx'].mean()

# Step 4: Calculate the logarithm of the average mortality rates
log_avg_mortality_rates = {group: np.log(rate) for group, rate in average_mortality_rates.items()}

# Use the midpoints of the age groups as representative ages for regression
midpoints = {
    "45-49": 47,
    "50-54": 52,
    "55-59": 57,
    "60-64": 62,
    "65-69": 67,
    "70-74": 72,
    "75-79": 77,
    "80-84": 82,
    "85-89": 87,
}

# Convert midpoints and log mortality rates into lists for regression
ages_for_regression = np.array(list(midpoints.values()))
log_mortality_for_regression = np.array(list(log_avg_mortality_rates.values()))

# Step 5: Prepare the data for linear regression
X = np.column_stack((ages_for_regression, ages_for_regression**2))

# Add a constant to the model for the intercept term
X = sm.add_constant(X)

# Step 6: Perform linear regression
model = sm.OLS(log_mortality_for_regression, X)
results = model.fit()
coefficients = results.params

# Step 7: Predict the logarithm of mortality rates for ages 90 to 106
predicted_log_mortality = {}
age_prediction_groups = {
    "90-94": range(90, 95),
    "95-99": range(95, 100),
    "100-104": range(100, 105),
    "105+": range(105, 107)
}

for group, ages in age_prediction_groups.items():
    predictions = []
    for age in ages:
        log_mortality = coefficients[0] + coefficients[1] * age + coefficients[2] * age**2
        predictions.append(log_mortality)
    predicted_log_mortality[group] = np.mean(predictions)

# Step 8: Apply exponential back-transformation
predicted_mortality_rates = {group: 10 ** log_mortality for group, log_mortality in predicted_log_mortality.items()}

# Display the results
print("Predicted mortality rates (original scale):")
for group, rate in predicted_mortality_rates.items():
    print(f"{group}: {rate}")


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# Step 1: Load the qx.csv file
file_path = 'qx.csv'
df = pd.read_csv(file_path)

# Step 2: Define the age groups
age_groups = {
    "45-49": range(45, 50),
    "50-54": range(50, 55),
    "55-59": range(55, 60),
    "60-64": range(60, 65),
    "65-69": range(65, 70),
    "70-74": range(70, 75),
    "75-79": range(75, 80),
    "80-84": range(80, 85),
    "85-89": range(85, 90),
}

# Step 3: Calculate the average mortality rates for each age group
average_mortality_rates = {}
for group, ages in age_groups.items():
    average_mortality_rates[group] = df[df['age'].isin(ages)]['qx'].mean()

# Step 4: Calculate the logarithm of the average mortality rates
log_avg_mortality_rates = {group: np.log(rate) for group, rate in average_mortality_rates.items()}

# Step 5: Perform linear regression based on these values
ages_for_regression = np.array(list(range(49, 95)))
X = np.column_stack((ages_for_regression, ages_for_regression**2))
y = np.array([log_avg_mortality_rates['45-49'], log_avg_mortality_rates['50-54'], 
              log_avg_mortality_rates['55-59'], log_avg_mortality_rates['60-64'], 
              log_avg_mortality_rates['65-69'], log_avg_mortality_rates['70-74'], 
              log_avg_mortality_rates['75-79'], log_avg_mortality_rates['80-84'], 
              log_avg_mortality_rates['85-89']])

# Fit the linear model
model = sm.OLS(y, X)
results = model.fit()
coefficients = results.params

# Step 6: Predict the logarithm of mortality rates for ages 90 to 106
predicted_log_mortality = {}
age_prediction_groups = {
    "90-94": range(90, 95),
    "95-99": range(95, 100),
    "100-104": range(100, 105),
    "105+": range(105, 107)
}

for group, ages in age_prediction_groups.items():
    predictions = []
    for age in ages:
        log_mortality = coefficients[0] * age + coefficients[1] * age**2
        predictions.append(log_mortality)
    predicted_log_mortality[group] = np.mean(predictions)

# Step 7: Apply exponential back-transformation
predicted_mortality_rates = {group: 10 ** log_mortality for group, log_mortality in predicted_log_mortality.items()}

# Display the results
print("Predicted mortality rates (original scale):")
for group, rate in predicted_mortality_rates.items():
    print(f"{group}: {rate}")


In [ ]:

# Step 2: Prepare the data for linear regression
X = np.column_stack((ages, ages**2))  # Include age and age^2 as independent variables

# Step 3: Perform linear regression
coefficients, _, _, _ = np.linalg.lstsq(X, log_mortality_rates, rcond=None)

# Step 4: Predict the log mortality rate for age 90
age_90 = 90
predicted_log_mortality = coefficients[0] * age_90 + coefficients[1] * age_90**2

# Step 5: Convert the predicted log mortality rate back to the original scale
predicted_mortality = 10 ** predicted_log_mortality

print(predicted_mortality)

# Cohort model to calculate Survival rate based on Demographic Ageing 

Initialize the Input Data:

Mortality rates (qx) for ages 18 to 106.
LPA data (e.g., donor demands) for years 2007 to 2023.
Mortality statistics for males and females.

Forecasted donor demands for years 2024 to 2030.
Create Survival Tables:

For each year (2007-2023), generate a survival table corresponding to that year using the actual donor demands.
Estimate Age-Specific Survival:

Use the mortality rates (qx) to create tables for estimated age-specific survival for both males and females.
Forecast Donors for Future Years:

Use the survival rates to forecast the number of donors for future years (2024-2030).
Aggregate Data into a Final Table:

Aggregate the data from the survival tables into a single table that tracks the total survival over the years.

In [ ]:


# Example LPA data for years 2007 to 2023 (random data for illustration)
lpa_data = pd.DataFrame(np.random.randint(100, 1000, size=(89, 17)), 
                        index=np.arange(18, 107), 
                        columns=np.arange(2007, 2024))  # Replace with actual LPA data

# Example mortality statistics for males and females (random data for illustration)
survival_male = np.random.random((89, 89))  # Replace with actual male survival rates
survival_female = np.random.random((89, 89))  # Replace with actual female survival rates

# Example forecasted donor demands for years 2024 to 2030 (random data for illustration)
forecast_data = pd.DataFrame(np.random.randint(100, 1000, size=(89, 7)), 
                             index=np.arange(18, 107), 
                             columns=np.arange(2024, 2031))  # Replace with actual forecast data

# Control assumption for gender
control_assumption_gender = 1  # 1 for male, 0 for female

# Step 2: Create survival tables for each year from 2007 to 2023

def create_survival_table(year):
    survival_table = pd.DataFrame(index=np.arange(18, 107), columns=[year])
    survival_table[year] = lpa_data[year]
    return survival_table

survival_tables = {year: create_survival_table(year) for year in range(2007, 2024)}

# Step 3: Estimate Age-Specific Survival for future years based on survival rates

def estimate_age_specific_survival(age, prior_year, current_year, gender):
    prior_number_of_year = current_year - 2007
    if gender == 1:  # Male
        survival_rate = survival_male[age - prior_number_of_year - 18, prior_year]
    else:  # Female
        survival_rate = survival_female[age - prior_number_of_year - 18, prior_year]
    return survival_rate

# Step 4: Forecast Donors for Future Years

def forecast_donors_for_year(year):
    prior_year_data = survival_tables[year - 1]
    forecasted_table = pd.DataFrame(index=np.arange(18, 107), columns=[year])

    for age in range(18, 107):
        prior_age = age - (year - 2007)
        if prior_age >= 18:
            survival_rate = estimate_age_specific_survival(age, year - 2007, year, control_assumption_gender)
            forecasted_table.at[age, year] = prior_year_data.at[prior_age, year - 1] * survival_rate
        else:
            forecasted_table.at[age, year] = 0

    return forecasted_table

for year in range(2024, 2031):
    if year in forecast_data.columns:
        survival_tables[year] = forecast_data[year]
    else:
        survival_tables[year] = forecast_donors_for_year(year)

# Step 5: Aggregate all survival tables into a single table

total_lpa_survival_table = pd.DataFrame(index=np.arange(18, 107), columns=np.arange(2007, 2031))

for year in range(2007, 2031):
    total_lpa_survival_table[year] = survival_tables[year].sum(axis=1)

# Example output of the total LPA Survival Table
print(total_lpa_survival_table)

# Step 6: Save or manipulate the final output as needed (e.g., export to Excel, further analysis, etc.)
# total_lpa_survival_table.to_excel("Total_LPA_Survival_Tables.xlsx")


In [ ]:
year=range(2007,2023)
print(create_survival_table(year))

age= 19 #range(18,107)
prior_year= 0 # This can be from 0 to 89 (aging from 18 to 106 years old)
current_year=2023
gender=1
print(estimate_age_specific_survival(age, prior_year, current_year, gender))
#print(forecast_donors_for_year(year))

# Dynamic Micro-simulation

Initialization:

We start by initializing a simulated population with random ages between 18 and 106 and assigning genders.
The population size (population_size) is chosen as 10,000 for this example, but it can be adjusted.
Simulating Life Paths:

For each individual, we simulate their life path year by year using mortality rates (qx).
The function simulate_life_path updates each individual's age and determines if they survive each year based on the given mortality rates.
Aggregating Results:

The survival status of each individual is tracked, and these are aggregated into a survival_table that records the number of survivors for each year and age group.
Incorporating Forecasts:

For years beyond 2023, we replace the survival counts with forecasted donor demands.
Output:

The final survival_table contains the simulated survival data, which can be saved or further analyzed.
Important Notes:
The mortality rates (qx), initial population attributes, and forecast data must be realistic and based on actual data.
The size of the population (population_size) can significantly impact the accuracy of the simulation.
This code provides a flexible framework that can be extended or modified based on specific needs or additional demographic factors.
This dynamic micro-simulation model provides a powerful tool to simulate demographic processes and forecast the survival rates of a population over time, accounting for individual-level variability and dynamic changes in the population structure.

In [ ]:
import numpy as np
import pandas as pd

# Step 1: Initialize the Population

# Parameters
start_year = 2007
end_year = 2030
initial_age = 18
max_age = 106
population_size = 10000  # Number of individuals to simulate

# Generate initial cohort
np.random.seed(42)
initial_population = pd.DataFrame({
    'age': np.random.randint(initial_age, max_age, population_size),
    'gender': np.random.choice(['male', 'female'], population_size, p=[0.5, 0.5])
})

# Step 2: Simulate Individual Life Paths

def simulate_life_path(individual, start_year, end_year, mortality_rates):
    life_path = {}
    for year in range(start_year, end_year + 1):
        current_age = individual['age'] + (year - start_year)
        if current_age > max_age:
            break
        if np.random.rand() < mortality_rates[individual['gender']][current_age - initial_age]:
            life_path[year] = 0  # Individual has died
            break
        life_path[year] = 1  # Individual survives this year
    return life_path

# Example mortality rates (random for illustration)
mortality_rates = {
    'male': np.random.rand(max_age - initial_age + 1),
    'female': np.random.rand(max_age - initial_age + 1)
}

# Simulate life paths for the entire population
life_paths = initial_population.apply(lambda ind: simulate_life_path(ind, start_year, end_year, mortality_rates), axis=1)

# Step 3: Aggregate Results

# Initialize an empty DataFrame to store survival counts for each year and age
survival_table = pd.DataFrame(0, index=np.arange(initial_age, max_age + 1), columns=np.arange(start_year, end_year + 1))

# Aggregate individual life paths into the survival table
for path in life_paths:
    for year, status in path.items():
        age = initial_age + (year - start_year)
        survival_table.at[age, year] += status

# Step 4: Incorporate Forecasts (For years 2024 to 2030)

# Example forecasted data for donor demands (random for illustration)
forecasted_donors = pd.DataFrame(np.random.randint(100, 1000, size=(max_age - initial_age + 1, end_year - 2023)),
                                 index=np.arange(initial_age, max_age + 1), 
                                 columns=np.arange(2024, end_year + 1))

# Replace survival table data for forecast years with the forecasted data
for year in range(2024, end_year + 1):
    survival_table[year] = forecasted_donors[year]

# Step 5: Output the Results

# The final survival table contains the simulated survival counts for each year and age
print(survival_table)

# Optionally, save the results to an Excel file or further manipulate the data
# survival_table.to_excel("Simulated_LPA_Survival_Tables.xlsx")


In [ ]:
import numpy as np
import pandas as pd

# Assuming qx is the mortality rate array for ages 18 to 106
# Example: qx = np.array([...]) # 89 elements corresponding to ages 18 to 106
qx = np.random.random(89)  # This should be replaced with actual mortality rates

# Assuming we have actual LPA data in a dataframe, with years as columns and ages 18-106 as rows
# Example: lpa_data = pd.DataFrame([...], index=np.arange(18, 107), columns=np.arange(2007, 2024))
lpa_data = pd.DataFrame(np.random.randint(100, 1000, size=(89, 17)), 
                        index=np.arange(18, 107), index
                        columns=np.arange(2007, 2024))  # Replace with actual LPA data

# Placeholder for male and female survival tables
# Example: survival_male = np.array([...]) # Male survival rates
# Example: survival_female = np.array([...]) # Female survival rates
survival_male = np.random.random((89, 89))  # Replace with actual survival rates
survival_female = np.random.random((89, 89))  # Replace with actual survival rates

# Control assumptions for gender (1 = Male, 0 = Female)
control_assumption_gender = 1  # Example value, should be set based on the control assumptions

# Initialize dataframe to hold the forecast data
forecast_years = np.arange(2024, 2031)
forecast_data = pd.DataFrame(index=np.arange(18, 107), columns=forecast_years)

# Copy actual LPA data to forecast data for years 2007-2023
all_data = lpa_data.copy()

x=17 # start age -1
# Iterate through years from 2007 to 2030
for year in range(2007, 2031):
    if year > 2023:
        # Forecast years - use the previous year's data and survival rates
        prior_year = year - 2007
        for age in range(18, 107):
            if age - prior_year >= 18:  # Ensure valid age after subtracting years
                if control_assumption_gender == 1:
                    # Use male survival rates
                    survival_rate = survival_male[age - prior_year - 18, prior_year]
                else:
                    # Use female survival rates
                    survival_rate = survival_female[age - prior_year - 18, prior_year]

                # Estimate donors
                #num_donors = all_data.at[age - prior_year, year - 1] * survival_rate
                num_donors = all_data.iloc[age - prior_year - 1, prior_year - 1] * survival_rate
                print(prior_year)
                print(year)
                print(age)
                forecast_data.at[age, year] = num_donors
            else:
                forecast_data.at[age, year] = 0  # No data available for this combination of year and age
    else:
        # Use actual LPA data for years up to 2023
        forecast_data[year] = all_data[year]

# Append forecasted data to the full dataset
all_data = pd.concat([all_data, forecast_data], axis=1)

# Example: output the full dataset (could be written to an Excel or CSV file)
print(all_data)


# Calculate Total Numeber of Demands:
calculate_total_demand() sums up the values based on the year.

# Calculate Age Distribution:
calculate_age_distribution() aggregates and normalizes the age distribution data.

# Monte Carlo Simulation:
monte_carlo_simulation() runs the simulation for 2024, generating multiple forecasts.

# Output Results:
output_results() writes the results to a new or existing worksheet.

# Calculate Population and Donors:
calculate_total_population_and_donors() computes total population and donors age-specific.

# Extend Forecast:
extend_forecast() continues the forecast for 2025-2029 using the cohort survival model.

# Finding Values for Previous Years:
Use Find to locate the corresponding rows for the current age group in the last two years.
lastYearRow corresponds to the row in the worksheet for the previous year (year - 1).
secondLastYearRow corresponds to the row in the worksheet for the year before the previous year (year - 2).

# Calculating prevYearMean:
The mean of the forecasts from the last two years is calculated using the values found from lastYearMean and secondLastYearMean.
This average is then used as prevYearMean in the forecast calculations.

# Using prevYearMean in Forecast:
The prevYearMean is multiplied by the cohort survival rate to get the forecasted donors for the current year and age group.
This method ensures that the forecast for each year is informed by the trends observed in the previous two years, providing a more stable and potentially more accurate prediction.
To apply the assumedPotentialMarketShape for each age into the long-term forecast along with the cohortSurvivalRate, we can modify the calculation of forecastedDonors by multiplying the prevYearMean (the average of the forecasts from the last two years) by both the cohortSurvivalRate and the assumedPotentialMarketShape for the specific age.

# Multiplying by assumedPotentialMarketShape:
The forecastedDonors calculation is modified to include assumedPotentialMarketShape(age) / 100. This converts the percentage value into a multiplier.
By applying this adjustment, the forecasted number of donors for each age group is scaled according to the assumed potential market shape for that age group.

# Integrating assumedPotentialMarketShape:
The potential market shape is used to adjust the number of forecasted donors, reflecting the varying market potential across different age groups.

# Impact on the Forecast:
This approach ensures that the forecast not only considers the survival rate but also adjusts based on the market potential for each age group, providing a more nuanced and potentially more accurate prediction.
By incorporating both the cohortSurvivalRate and assumedPotentialMarketShape, the forecast becomes more refined, taking into account both demographic survival probabilities and market potential variations by age group.

In [ ]:
import pandas as pd
import numpy as np

class LongTermForecastLPA:
    def __init__(self, donors_df, population_forecast_df, survival_rate_df):
        # Initialize dataframes
        self.donors_df = donors_df
        self.population_forecast_df = population_forecast_df
        self.survival_rate_df = survival_rate_df

        # Fixed variables
        self.iterations = 1000
        self.lower_bound = 4000
        self.upper_bound = 7000
        self.num_working_days_2024 = 256

        # Fixed total receipts for 2022 and 2023
        self.total_receipts_2022 = 751967.534626039
        self.total_receipts_2023 = 970035.028248588

        # Initialize variables
        self.total_population_age_specific = np.zeros(107)
        self.total_donors_age_specific = np.zeros(107)
        self.assumed_market_shape = np.zeros(107)
        self.assumed_potential_market_shape = np.zeros(107)

    def run_forecast(self):
        # Calculate total demands for 2022 and 2023
        total_demand_2022 = self.calculate_total_demand(2022)
        total_demand_2023 = self.calculate_total_demand(2023)

        # Aggregate age distributions for 2022 and 2023
        age_distribution_2022 = self.calculate_age_distribution(2022, total_demand_2022)
        age_distribution_2023 = self.calculate_age_distribution(2023, total_demand_2023)

        # Average age distribution across 2022 and 2023
        average_age_distribution = self.calculate_average_age_distribution(age_distribution_2022, age_distribution_2023)

        # Monte Carlo Simulation for 2024
        monte_carlo_results = self.monte_carlo_simulation(average_age_distribution, total_demand_2022, total_demand_2023)

        # Output the results to a DataFrame
        result_df = self.output_results(monte_carlo_results, average_age_distribution)

        # Calculate total population and total donors age-specific for 2023 and 2024
        self.calculate_total_population_and_donors()

        # Extend forecast to the next 5 years (2025-2029) using cohort model
        extended_forecast_df = self.extend_forecast(result_df)

        # Combine all results
        final_result_df = pd.concat([result_df, extended_forecast_df], ignore_index=True)

        return final_result_df

    def calculate_total_demand(self, year):
        return self.donors_df[self.donors_df['year'] == year]['demand'].sum()

    def calculate_age_distribution(self, year, total_demand):
        age_distribution = self.donors_df[self.donors_df['year'] == year].groupby('age')['demand'].sum()
        return age_distribution / total_demand

    def calculate_average_age_distribution(self, dist_2022, dist_2023):
        common_ages = dist_2022.index.intersection(dist_2023.index)
        return (dist_2022[common_ages] + dist_2023[common_ages]) / 2

    def monte_carlo_simulation(self, average_age_distribution, total_demand_2022, total_demand_2023):
        monte_carlo_results = []
        for _ in range(self.iterations):
            random_receipt = np.random.randint(self.lower_bound, self.upper_bound)
            total_receipts_2024 = random_receipt * self.num_working_days_2024
            ratio_receipt_donor_2022 = self.total_receipts_2022 / total_demand_2022
            ratio_receipt_donor_2023 = self.total_receipts_2023 / total_demand_2023
            estimated_donors_2024 = total_receipts_2024 / ((ratio_receipt_donor_2022 + ratio_receipt_donor_2023) / 2)
            monte_carlo_results.append(estimated_donors_2024)
        return np.array(monte_carlo_results)

    def output_results(self, monte_carlo_results, average_age_distribution):
        result_data = []

        for age in average_age_distribution.index:
            sum_values = np.sum(monte_carlo_results * average_age_distribution[age])
            sum_squared_values = np.sum((monte_carlo_results * average_age_distribution[age]) ** 2)
            forecasted_donors_mean = sum_values / self.iterations
            forecasted_donors_std_dev = np.sqrt(sum_squared_values / self.iterations - (forecasted_donors_mean ** 2))

            result_data.append({
                'year': 2024,
                'Age Group': age,
                'Age Distribution 2022': average_age_distribution[age],
                'Age Distribution 2023': 'N/A',
                'Random Receipt': np.mean(monte_carlo_results),
                'Final Forecast (Mean)': forecasted_donors_mean,
                'Final Forecast (StdDev)': forecasted_donors_std_dev
            })

        return pd.DataFrame(result_data)

    def calculate_total_population_and_donors(self):
        for age in range(18, 107):
            total_population = self.population_forecast_df[(self.population_forecast_df['age'] == age) & 
                                                           (self.population_forecast_df['year'].isin([2023, 2024]))]['PopulationForecastInput'].sum()
            total_donors = self.total_population_age_specific[age] + self.total_donors_age_specific[age]
            self.assumed_market_shape[age] = total_donors / total_population
        self.max_assumed_market_shape = np.max(self.assumed_market_shape)

        for age in range(18, 107):
            self.assumed_potential_market_shape[age] = self.assumed_market_shape[age] / self.max_assumed_market_shape * 100

    def extend_forecast(self, result_df):
        extended_result_data = []
        survival_dict = self.survival_rate_df.set_index(['age', 'year'])['SurvivalRate'].to_dict()

        for year in range(2025, 2030):
            for age in range(18, 107):
                key = (age, year)
                cohort_survival_rate = survival_dict.get(key, 0)
                prev_year_mean = result_df.loc[(result_df['year'] == year - 1) & (result_df['Age Group'] == age), 'Final Forecast (Mean)'].values[0]
                forecasted_donors_mean = prev_year_mean * cohort_survival_rate
                variance = np.var([forecasted_donors_mean for _ in range(self.iterations)])
                forecasted_donors_std_dev = np.sqrt(variance) if variance >= 0 else 0

                extended_result_data.append({
                    'year': year,
                    'Age Group': age,
                    'Age Distribution 2022': 'N/A',
                    'Age Distribution 2023': 'N/A',
                    'Random Receipt': 'N/A',
                    'Final Forecast (Mean)': forecasted_donors_mean,
                    'Final Forecast (StdDev)': forecasted_donors_std_dev
                })

        return pd.DataFrame(extended_result_data)

# Assuming you have loaded your data into pandas DataFrames:
donors_df = pd.read_csv('annual_unique_donors.csv')  # Replace with your DataFrame
population_forecast_df = pd.read_csv('UnpivotPopulationForecast.csv')  # Replace with your DataFrame
survival_rate_df = pd.read_csv('UnpivotSurvivalRate.csv')  # Replace with your DataFrame

# Initialize the forecast class
forecast = LongTermForecastLPA(donors_df, population_forecast_df, survival_rate_df)

# Run the forecast
result_df = forecast.run_forecast()

# Save the result to an Excel file
result_df.to_excel('long_term_forecast_results.xlsx', index=False)


In [ ]:


# Define a function to perform Naïve extrapolation
def naive_extrapolation(data, forecast_periods, freq='Y'):
    last_observed_value = data.iloc[-1]
    forecast_values = np.repeat(last_observed_value, forecast_periods)
    #data.set_index('year', inplace=True)
    #forecast_index = pd.date_range(start=f"{data.index[-1].year + 1}-01-01", periods=forecast_periods, freq='YS')
    last_year = data.index[-1] #data['year'].iloc[-1] #
    
    # Modified to generate forecasts for different frequencies (Y, M, D).
    if freq == 'Y': 
        forecast_index = pd.date_range(start=f"{last_year + 1}", periods=forecast_periods, freq='YS')
        #forecast_index = pd.date_range(start=datetime(last_year, 1, 1).strftime('%Y'), periods=forecast_periods, freq='YS')
        
    elif freq == 'M': 
        tup = data.index[-1]
        first, *middle, last = tup
        middle = last
        last = 1
        # Calculate the date 6 months from today's date and print it in ISO 8601 format
        #print((dt.date.today() + dt.timedelta(1 * 365 / 12)).isoformat())
        forecast_index = pd.date_range(start=f"{(dt.date(first, middle, last) + dt.timedelta((1 * 365 / 12)+1)).isoformat()}", periods=forecast_periods, freq='MS')
        #forecast_index = pd.date_range(start=f"{last_year}-01", periods=forecast_periods, freq='MS')
        #forecast_index = pd.date_range(start=datetime(last_year, 1, 1).strftime('%Y-%m'), periods=forecast_periods, freq='MS')
    elif freq == 'D':
        tup = data.index[-1]
        first, middle, last = tup 
        forecast_index = pd.date_range(start=f"{(dt.date(first, middle, last) + dt.timedelta(3)).isoformat()}", periods=forecast_periods, freq='D')
        #forecast_index = pd.date_range(start=f"{last_year}-01-01", periods=forecast_periods, freq='D')
        #forecast_index = pd.date_range(start=datetime(last_year, 1, 1).strftime('%Y-%m-%d'), periods=forecast_periods, freq='D')
        
    forecast_df = pd.DataFrame({'Forecast': forecast_values}, index=forecast_index)
    return forecast_df

In [ ]:
# Forecast demand using SARIMA
#start_year = '2024-04'
#forecast_periods = 2 #24
# Forecast overall demand using Naïve extrapolation
forecast_periods_annual = 5  # 2 years of forecast
forecast_periods_monthly = 24  # 24 months of forecast (2 years)
forecast_periods_daily = 730  # 730 days of forecast (2 years)

# Annual forecast
historical_demand = historical_data_df.groupby('year')['demand'].sum()
forecast_df_annual = naive_extrapolation(historical_demand, forecast_periods_annual, freq='Y')

# Monthly forecast
historical_demand_mon = historical_demand_monthly.reset_index()
historical_demand_mon = historical_demand_monthly.groupby(['year', 'month'])['demand'].sum()
forecast_df_monthly = naive_extrapolation(historical_demand_mon, forecast_periods_monthly, freq='M')


# Daily forecast
historical_demand_day = historical_demand_daily.reset_index()
historical_demand_day = historical_demand_day.groupby(['year', 'month', 'day'])['demand'].sum() #.groupby(['year', 'month', 'day'])['demand'].sum()
forecast_df_daily = naive_extrapolation(historical_demand_day, forecast_periods_daily, freq='D')


In [ ]:



#forecast_df = fit_sarima_and_forecast(historical_data_df.set_index('year')['demand'], start_year, forecast_periods)
#forecast_df = fit_sarima_and_forecast(historical_demand, start_year, forecast_periods)
#forecast_df = naive_extrapolation(historical_demand, forecast_periods)

# Calculate age-specific demand ratios
age_specific_ratios = {}
for year in historical_data_df['year'].unique():
    print(f"year: {year}")
    age_demand = annual_donor_demands_age_specific[annual_donor_demands_age_specific['year'] == year]
    #print(f"age_demand: {age_demand}")
    total_demand = age_demand['demand'].sum()
    #print(f"total_demand: {total_demand}")
    if total_demand > 0:
        age_specific_ratios[year] = age_demand.set_index('age')['demand'] / total_demand
        #print(f"age_specific_ratios[year]: {age_specific_ratios[year]}")
    else:
        age_specific_ratios[year] = pd.Series(0, index=range(18, 107))
        #print(f" Zero age: age_specific_ratios[year]: {pd.Series(0, index=range(18, 107))}")
# Function to apply uncertainty adjustments to the forecasts
def apply_uncertainty_adjustments(forecast_df, annual_donor_receipts, annual_donor_demands, age_specific_ratios, forecast_start_year, freq='Y'):
#def apply_uncertainty_adjustments(forecast_df, annual_donor_receipts, annual_donor_demands, age_specific_ratios, forecast_start_year):
    # Filter Ages: We filter the data to include only ages between 18 and 106.
    #adjusted_forecast = forecast_df.copy()
    adjusted_forecast = pd.DataFrame(index=forecast_df.index, columns=range(18, 107))
    
    start_year = pd.to_datetime(forecast_start_year).year -1
    
    month_ratio_list = []#list()
    
    for i in range(len(forecast_df)):
        if freq == 'Y':
            forecast_year = start_year - i
            print(f" i: {i}  forecast_year: {forecast_year}")
        elif freq == 'M':
            forecast_year = start_year - (i // 12)
            print(f" i: {i}  forecast_year: {forecast_year}")
        elif freq == 'D':
            forecast_year = start_year - (i // 365)
            print(f" i: {i}  forecast_year: {forecast_year}")
    # Forecasting: We fit the SARIMA model to the historical data and obtain the overall forecast.
#         Check Year Existence: Before calculating the month_ratio, we check if the forecast_year exists in both annual_donor_receipts and annual_donor_demands.
#         Assign Zero Demand: If the year does not exist in either DataFrame, we assign zero demand for that period.
        if forecast_year in annual_donor_receipts['year'].values and forecast_year in annual_donor_demands.index:
            print(f"## ------------------- ##")
            #print(f"## ------------------- ##")
            #print(f"## ------------------- ##")
            #print(f"forecast_year for receipts: {annual_donor_receipts['year'].values}")
            #print(f"forecast_year for donors: {annual_donor_demands.index}")
            #print(f"forecast_year: {forecast_year}")
            month_ratio = round((annual_donor_receipts[annual_donor_receipts['year'] == forecast_year]['Yearly Receipt'].values[0] / ##annual_donor_receipts[annual_donor_receipts['year'] == 2023]['Yearly Receipt'].values[0] / annual_donor_demands[2023] 
                           annual_donor_demands[forecast_year]), 2)
            #print(f"month_ratio: {month_ratio}")
            
            # Age-Specific Ratios: We calculate the demand ratios for each age group.
            #age_specific_ratio = age_specific_ratios.get(year, pd.Series(0, index=range(18, 107)))
            for age in range(18, 107):
                age_specific_ratio = age_specific_ratios.get(forecast_year, pd.Series(0, index=range(18, 107)))
                #print(f"age_specific_ratio: {age_specific_ratio}")
                # Adjusting the Forecast:
                # Initialize an empty DataFrame to hold the adjusted forecast values.
                # For each forecasted period, calculate the monthly ratio.
                if age in age_specific_ratio:
                     # Distribute the forecasted demand among different age groups using the calculated age-specific ratios.
                    #month_ratio_list = month_ratio_list.append(month_ratio) 
                    adjusted_forecast.iloc[i, age - 18] = (forecast_df.iloc[i]['Forecast'] * month_ratio * age_specific_ratio[age])
                    print(f" for i: {i}, year: {forecast_year}, age: {age} ==> month_ratio = {month_ratio}, age_specific_ratio[age]: {age_specific_ratio[age]}, adjusted_forecast: {adjusted_forecast.iloc[i, age - 18]}")
#                     if age not in dict:
#                         dict[age] = []
#                         dict[age].append(month_ratio)
#                         print(dict)
                   
                    #.to_csv(f"month_ratio_year{forecast_year}_age{age}.csv", index=False)
                    #age_specific_ratio[age].to_csv(f"age_specific_ratio_year{forecast_year}_age{age}.csv", index=False)
                else: # If a specific age has no data, we assign a zero demand.
                    adjusted_forecast.iloc[i, age - 18] = 0   
        else:
            # If the year is not found, assign zero demand
            adjusted_forecast.iloc[i, :] = 0
        
        #adjusted_forecast.iloc[i] = adjusted_forecast.iloc[i] * month_ratio * age_specific_ratio.values
        #forecast_value = adjusted_forecast.iloc[i]['Forecast']
        #adjusted_values = forecast_value * month_ratio * age_specific_ratio.values
        #adjusted_forecast.iloc[i]['Forecast'] = adjusted_values.sum()
    return adjusted_forecast #, month_ratio_list)

# Apply the uncertainty adjustments to the forecasts
#adjusted_forecast_df = apply_uncertainty_adjustments(forecast_df, annual_donor_receipts, annual_donor_demands, age_specific_ratios, start_year)

# Apply the uncertainty adjustments to the forecasts

start_year = dt.date.today() #2024
forecast_start_year = start_year
pd.to_datetime(forecast_start_year).year -1

# Adjusted annual forecast
adjusted_forecast_annual_df = apply_uncertainty_adjustments(forecast_df_annual, annual_donor_receipts, annual_donor_demands, age_specific_ratios, start_year, freq='Y')
adjusted_forecast_annual_df.to_csv(f"adjusted_forecast_annual.csv", index=False)

# Adjusted monthly forecast
#adjusted_forecast_monthly_df = apply_uncertainty_adjustments(forecast_df_monthly, annual_donor_receipts, annual_donor_demands, age_specific_ratios, start_year, freq='M')
#adjusted_forecast_monthly_df.to_csv(f"adjusted_forecast_monthly.csv", index=False)

# Adjusted daily forecast
#adjusted_forecast_daily_df = apply_uncertainty_adjustments(forecast_df_daily, annual_donor_receipts, annual_donor_demands, age_specific_ratios, start_year, freq='D')
#adjusted_forecast_daily_df.to_csv(f"adjusted_forecast_daily.csv", index=False)

# Display the adjusted forecasts
print("Adjusted Annual Forecast:")
print(adjusted_forecast_annual_df.head())

#print("Adjusted Monthly Forecast:")
#print(adjusted_forecast_monthly_df.head())

#print("Adjusted Daily Forecast:")
#print(adjusted_forecast_daily_df.head())

In [ ]:
# Load the historical data
historical_data_df = annual_unique_donors


# Filter the data to include only ages from 18 to 106
filtered_data_df = historical_data_df[(historical_data_df['age'] >= 18) & (historical_data_df['age'] <= 106)]

# Calculate annual donor receipts and demands
annual_donor_receipts = filtered_data_df.groupby('year')['demand'].sum().reset_index(name='Yearly Receipt')
annual_donor_demands = filtered_data_df.groupby('year')['demand'].sum()

# Calculate age-specific demands
annual_donor_demands_age_specific = filtered_data_df.groupby(['year', 'age'])['demand'].sum().reset_index()

# Define a function to perform Naïve extrapolation
def naive_extrapolation(data, forecast_periods):
    last_observed_value = data.iloc[-1]
    forecast_values = np.repeat(last_observed_value, forecast_periods)
    #forecast_index = pd.date_range(start=f"{data.index[-1].year + 1}-01-01", periods=forecast_periods, freq='YS')
    forecast_index = pd.date_range(start=f"{data.index[-1] + 1}-01-01", periods=forecast_periods, freq='YS')

    forecast_df = pd.DataFrame({'Forecast': forecast_values}, index=forecast_index)
    return forecast_df

# Forecast overall demand using Naïve extrapolation
forecast_periods = 2  # 2 years of forecast
historical_demand = filtered_data_df.groupby('year')['demand'].sum()
historical_demand

In [ ]:

forecast_df = naive_extrapolation(historical_demand, forecast_periods)
forecast_df

In [ ]:


# # Calculate age-specific demand ratios
# age_specific_ratios = {}
# for year in filtered_data_df['year'].unique():
#     age_demand = annual_donor_demands_age_specific[annual_donor_demands_age_specific['year'] == year]
#     total_demand = age_demand['demand'].sum()
#     if total_demand > 0:
#         age_specific_ratios[year] = age_demand.set_index('age')['demand'] / total_demand
#     else:
#         age_specific_ratios[year] = pd.Series(0, index=range(18, 107))

# # Function to apply uncertainty adjustments to the forecasts
# def apply_uncertainty_adjustments(forecast_df, annual_donor_receipts, annual_donor_demands, age_specific_ratios, forecast_start_year):
#     adjusted_forecast = pd.DataFrame(index=forecast_df.index, columns=range(18, 107))
#     start_year = pd.to_datetime(forecast_start_year).year
#     for i in range(len(forecast_df)):
#         forecast_year = start_year + i
#         # Check if the year exists in both DataFrames
#         if forecast_year in annual_donor_receipts['year'].values and forecast_year in annual_donor_demands.index:
#             month_ratio = (annual_donor_receipts[annual_donor_receipts['year'] == forecast_year]['Yearly Receipt'].values[0] /
#                            annual_donor_demands[forecast_year])
#             for age in range(18, 107):
#                 age_specific_ratio = age_specific_ratios.get(forecast_year, pd.Series(0, index=range(18, 107)))
#                 if age in age_specific_ratio:
#                     adjusted_forecast.iloc[i, age - 18] = (forecast_df.iloc[i]['Forecast'] * month_ratio * age_specific_ratio[age])
#                 else:
#                     adjusted_forecast.iloc[i, age - 18] = 0
#         else:
#             # If the year is not found, assign zero demand
#             adjusted_forecast.iloc[i, :] = 0
#     return adjusted_forecast

# # Apply the uncertainty adjustments to the forecasts
# start_year = 2024
# adjusted_forecast_df = apply_uncertainty_adjustments(forecast_df, annual_donor_receipts, annual_donor_demands, age_specific_ratios, start_year)

# # Display the adjusted forecast
# print(adjusted_forecast_df.head())


In [ ]:
# Creates a pivot table with age as the index and year as the columns.
# Calculates the growth rate for each age cohort.
# Computes the average growth rate for each age cohort.
# Uses the last year’s data to forecast the next year’s demand.
# Incorporates uncertainty by multiplying the forecast by the average Uncertainty_Ratio.
# Creates a pivot table with age as the index and year as the columns.
# Calculates the growth rate for each age cohort.
# Computes the average growth rate for each age cohort.
# Uses the last year’s data to forecast the next year’s demand.
# Incorporates uncertainty by multiplying the forecast by the average Uncertainty_Ratio.

# Creates a pivot table with age as the index and year as the columns.
# Applies seasonal decomposition to separate the trend and seasonal components.
# Calculates the average growth rate for the trend component.
# Forecasts the next year’s trend component and combines it with the seasonal component.
# Incorporates uncertainty by multiplying the forecast by the average Uncertainty_Ratio.

def cohort_analysis_forecast(df):
    # Create a pivot table with 'year' as columns, 'age' as index, and 'demand' as values
    pivot = df.pivot_table(index='age', columns='year', values='demand', aggfunc='sum')
    
    # Calculate the growth rate for each age cohort
    growth_rates = pivot.pct_change(axis=1)
    
    # Calculate the average growth rate for each age cohort
    avg_growth_rates = growth_rates.mean(axis=1)
    
    # Get the last year of data for each age cohort
    last_year_data = pivot.iloc[:, -1]
    
    # Forecast the next year's demand by applying the average growth rate to the last year's data
    forecast = last_year_data * (1 + avg_growth_rates)
    
    # Incorporate uncertainty by multiplying the forecast by the 'Uncertainty_Ratio'
    # Assuming 'Uncertainty_Ratio' is a column in the dataframe
    uncertainty_ratio = df['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in df.columns else 1
    forecast_with_uncertainty = forecast * uncertainty_ratio
    
    # Return the forecast with uncertainty
    return forecast_with_uncertainty

df = annual_donor_demands
# Call the function and print the forecast
forecast = cohort_analysis_forecast(df)
print(forecast)


In [ ]:
# def apply_uncertainty(forecast, annual_donor_receipts, annual_donor_demands, age_specific_demands):
#     # Calculate the annual ratios
#     annual_ratios = annual_donor_receipts['Yearly Receipt'] / annual_donor_demands

#     # Calculate the age-specific demand ratios
#     age_specific_ratios = age_specific_demands.groupby('age')['demand'].sum() / age_specific_demands['demand'].sum()

#     # Create a DataFrame to store the adjusted forecast
#     adjusted_forecast = pd.DataFrame(index=forecast.index, columns=forecast.columns)

#     for year in range(2024, 2026):
#         # Get the annual ratio for the current year
#         annual_ratio = annual_ratios.loc[year]

#         for age in range(18, 107):
#             # Get the age-specific ratio for the current age
#             age_ratio = age_specific_ratios.get(age, 0)

#             # Apply the uncertainty to the forecast
#             adjusted_forecast.loc[(forecast.index.year == year) & (forecast.columns == age), age] = (
#                 forecast.loc[(forecast.index.year == year) & (forecast.columns == age), age] * annual_ratio * age_ratio
#             )

#     # Fill NaN values with 0
#     adjusted_forecast = adjusted_forecast.fillna(0)

#     return adjusted_forecast


# def cohort_analysis_forecast_with_seasonality(df):
#     # Create a pivot table with 'year' as columns, 'age' as index, and 'demand' as values
#     pivot = df.pivot_table(index='age', columns='year', values='demand', aggfunc='sum')
    
#     # Initialize a DataFrame to store the forecast
#     forecast_df = pd.DataFrame(index=pivot.index)
    
#     # Loop through each age cohort to apply seasonal decomposition and forecast
#     for age in pivot.index:
#         # Get the time series for the current age cohort
#         ts = pivot.loc[age].dropna()
        
#         # Apply seasonal decomposition
#         decomposition = seasonal_decompose(ts, model='multiplicative', period=12)
        
#         # Extract the trend and seasonal components
#         trend = decomposition.trend.dropna()
#         seasonal = decomposition.seasonal.dropna()
        
#         # Calculate the average growth rate for the trend component
#         growth_rate = trend.pct_change().mean()
        
#         # Forecast the next year's trend component
#         last_trend_value = trend.iloc[-1]
#         forecast_trend = last_trend_value * (1 + growth_rate)
        
#         # Get the seasonal component for the next period (assuming seasonality repeats)
#         next_seasonal_value = seasonal.iloc[-12 % len(seasonal)]
        
#         # Combine the forecasted trend and seasonal components
#         forecast_value = forecast_trend * next_seasonal_value
        
#         # Incorporate uncertainty by multiplying the forecast by the 'Uncertainty_Ratio'
#         uncertainty_ratio = df['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in df.columns else 1
#         forecast_with_uncertainty = forecast_value * uncertainty_ratio
        
#         # Store the forecast in the DataFrame
#         forecast_df.loc[age, 'Forecast'] = forecast_with_uncertainty
    
#     # Return the forecast DataFrame
#     return forecast_df

# # Call the function and print the forecast
# forecast = cohort_analysis_forecast_with_seasonality(df)
# print(forecast)


# Creates a pivot table with age as the index and year as the columns.
# Applies STL decomposition to separate the trend and seasonal components.
# Calculates the average growth rate for the trend component.
# Forecasts the next year’s trend component and combines it with the seasonal component.
# Incorporates uncertainty by multiplying the forecast by the average Uncertainty_Ratio.

# import pandas as pd
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.holtwinters import ExponentialSmoothing

# # Load the dataset
# df = pd.read_csv('annual_unique_donors.csv')

# # Assuming the dataframe has a 'year' column and a 'demand' column
# df['year'] = pd.to_datetime(df['year'], format='%Y')
# df.set_index('year', inplace=True)

# # Decompose the time series to observe seasonal patterns
# decomposition = seasonal_decompose(df['demand'], model='additive', period=1)

# # Fit the TSL model with seasonal components
# model = ExponentialSmoothing(df['demand'], trend='add', seasonal='add', seasonal_periods=1)
# fit_model = model.fit()

# # Forecast the next period
# forecast = fit_model.forecast(1)

# # Add the forecast to the original function
# def cohort_analysis_forecast(df, forecast_value):
#     # Create a pivot table with 'year' as columns, 'age' as index, and 'demand' as values
#     pivot = df.pivot_table(index='age', columns='year', values='demand', aggfunc='sum')
    
#     # Calculate the growth rate for each age cohort
#     growth_rates = pivot.pct_change(axis=1)
    
#     # Calculate the average growth rate for each age cohort
#     avg_growth_rates = growth_rates.mean(axis=1)
    
#     # Get the last year of data for each age cohort
#     last_year_data = pivot.iloc[:, -1]
    
#     # Forecast the next year's demand by applying the average growth rate to the last year's data
#     forecast = last_year_data * (1 + avg_growth_rates)
    
#     # Incorporate uncertainty by multiplying the forecast by the 'Uncertainty_Ratio'
#     # Assuming 'Uncertainty_Ratio' is a column in the dataframe
#     uncertainty_ratio = df['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in df.columns else 1
#     forecast_with_uncertainty = forecast * uncertainty_ratio
    
#     # Add the TSL forecast value
#     forecast_with_uncertainty['TSL_Forecast'] = forecast_value
    
#     # Return the forecast with uncertainty
#     return forecast_with_uncertainty

# # Call the function and print the forecast with TSL seasonal forecasting
# forecast_with_tsl = cohort_analysis_forecast(df, forecast.values[0])
# print(forecast_with_tsl)

# This code integrates STL seasonal forecasting into your existing cohort analysis forecast function. 
# It uses the STL decomposition from the statsmodels library to decompose the time series data and 
# then applies the Holt-Winters method for forecasting. 
# The forecast value is then adjusted by the age-specific Uncertainty_Ratio.

# import pandas as pd
# from statsmodels.tsa.seasonal import STL
# from statsmodels.tsa.holtwinters import ExponentialSmoothing

# # Load the dataset
# df = pd.read_csv('annual_unique_donors.csv')

# # Assuming the dataframe has 'year' and 'age' columns and a 'demand' column
# df['year'] = pd.to_datetime(df['year'], format='%Y')
# df.set_index(['age', 'year'], inplace=True)

# # Function to apply STL and forecast
# def cohort_analysis_forecast_with_stl(df):
#     # Initialize a DataFrame to store the forecast
#     forecast_df = pd.DataFrame()

#     # Loop through each age cohort to apply STL and forecast
#     for age in df.index.get_level_values(0).unique():
#         # Get the time series for the current age cohort
#         ts = df.xs(age, level='age')['demand'].dropna()
        
#         # Apply STL decomposition
#         stl = STL(ts, seasonal=13)
#         result = stl.fit()
        
#         # Forecast the next year's demand using Holt-Winters method
#         model = ExponentialSmoothing(result.trend, trend='add', seasonal='add', seasonal_periods=13)
#         fit_model = model.fit()
#         forecast_value = fit_model.forecast(1).iloc[0]
        
#         # Get the corresponding 'Uncertainty_Ratio' for the specific age and year
#         uncertainty_ratio = df.xs(age, level='age')['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in df.columns else 1
        
#         # Multiply the forecast value by the 'Uncertainty_Ratio'
#         forecast_with_uncertainty = forecast_value * uncertainty_ratio
        
#         # Store the forecast in the DataFrame
#         forecast_df.loc[age, 'Forecast'] = forecast_with_uncertainty
    
#     # Return the forecast DataFrame
#     return forecast_df

# # Call the function and print the forecast
# forecast = cohort_analysis_forecast_with_stl(df)
# print(forecast)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.holtwinters import ExponentialSmoothing

class DemandForecasting:
    def __init__(self, file_path):
        self.df = pd.read_csv(file_path)
        self.df['year'] = pd.to_datetime(self.df['year'], format='%Y')
        self.df = self.df[self.df['year'].isin([2020, 2021, 2022, 2023])]
        self.df.set_index(['age', 'year'], inplace=True)
        self.variability_measures = None
        self.forecast_df = None

    def calculate_variability_measures(self):
        measures = pd.DataFrame()
        for age in self.df.index.get_level_values(0).unique():
            ts = self.df.xs(age, level='age')['demand'].dropna()
            std_dev = ts.std()
            variance = ts.var()
            range_val = ts.max() - ts.min()
            iqr = ts.quantile(0.75) - ts.quantile(0.25)
            cv = (std_dev / ts.mean()) * 100 if ts.mean() != 0 else 0
            mad = ts.mad()
            qd = iqr / 2
            measures.loc[age, 'Standard Deviation'] = std_dev
            measures.loc[age, 'Variance'] = variance
            measures.loc[age, 'Range'] = range_val
            measures.loc[age, 'IQR'] = iqr
            measures.loc[age, 'CV'] = cv
            measures.loc[age, 'MAD'] = mad
            measures.loc[age, 'QD'] = qd
        self.variability_measures = measures
        return measures

    def cohort_analysis_forecast_with_stl(self):
        forecast_df = pd.DataFrame()
        for age in self.df.index.get_level_values(0).unique():
            print(f"age: {age}")
            ts = self.df.xs(age, level='age')['demand'].dropna()
            # Ensure that the time series has a frequency for the STL decomposition
            if not ts.index.inferred_freq:
                ts = ts.asfreq('Y')
            stl = STL(endog=ts, period=None, seasonal=13)
            result = stl.fit()
            model = ExponentialSmoothing(result.trend, trend='add', seasonal='add', seasonal_periods=13)
            fit_model = model.fit()
            forecast_value = fit_model.forecast(1).iloc[0]
            uncertainty_ratio = self.df.xs(age, level='age')['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in self.df.columns else 1
            forecast_with_uncertainty = forecast_value * uncertainty_ratio
            forecast_df.loc[age, 'Forecast'] = forecast_with_uncertainty
        self.forecast_df = forecast_df
        return forecast_df

    def visualize_forecast_with_uncertainty(self):
        plt.figure(figsize=(14, 7))
        for age in self.df.index.get_level_values(0).unique():
            ts = self.df.xs(age, level='age')['demand']
            plt.plot(ts.index, ts, label=f'Original Demand (Age {age})', marker='o')
        for age in self.forecast_df.index:
            plt.plot(self.forecast_df.columns, self.forecast_df.loc[age], label=f'Forecasted Demand (Age {age})', marker='x')
        for age in self.forecast_df.index:
            adjusted_forecast = self.forecast_df.loc[age] * self.df.xs(age, level='age')['Uncertainty_Ratio'].mean()
            plt.plot(self.forecast_df.columns, adjusted_forecast, label=f'Adjusted Forecast (Age {age})', linestyle='--')
        plt.xlabel('Year')
        plt.ylabel('Demand')
        plt.title('Demand Forecast with Uncertainty Adjustment')
        plt.legend()
        plt.show()

    def visualize_variability_measures(self):
        # Reset index to use 'age' as a column for the boxplot
        df_reset = self.df.reset_index()
        
        # Check if there are any NaN values in 'age' or 'demand' columns
        if df_reset['age'].isnull().any() or df_reset['demand'].isnull().any():
            # Drop rows with NaN values in 'age' or 'demand' columns
            df_reset = df_reset.dropna(subset=['age', 'demand'])
        
        # Check if there are enough data points to create a boxplot
        if len(df_reset) > 0:
            plt.figure(figsize=(12, 6))
            sns.boxplot(x='age', y='demand', data=df_reset)
            plt.title('Box Plot of Demand by Age')
            plt.xlabel('age')
            plt.ylabel('Demand')
            plt.show()
        else:
            print("Not enough data to create a boxplot.")


        plt.figure(figsize=(12, 6))
        sns.histplot(self.df['demand'], bins=20, kde=True)
        plt.title('Histogram of Demand')
        plt.xlabel('Demand')
        plt.ylabel('Frequency')
        plt.show()

        plt.figure(figsize=(12, 6))
        sns.violinplot(x='age', y='demand', data=self.df.reset_index())
        plt.title('Violin Plot of Demand by Age')
        plt.xlabel('age')
        plt.ylabel('Demand')
        plt.show()

        mean_demand = self.df.groupby('age')['demand'].mean()
        std_demand = self.df.groupby('age')['demand'].std()
        plt.figure(figsize=(12, 6))
        plt.errorbar(mean_demand.index, mean_demand, yerr=std_demand, fmt='o', capsize=5)
        plt.title('Mean Demand with Standard Deviation by Age')
        plt.xlabel('age')
        plt.ylabel('Demand')
        plt.show()

        pivot_table = self.df.reset_index().pivot_table(index='age', columns='year', values='demand', aggfunc='mean')
        plt.figure(figsize=(12, 6))
        sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".1f")
        plt.title('Heatmap of Mean Demand by Age and Year')
        plt.xlabel('Year')
        plt.ylabel('age')
        plt.show()

    def apply_uncertainty_to_forecast(self,start_period,end_period,average_annual_receipts,total_annual_donors,annual_donor_demands_age_specific):
        # Load additional data
        average_annual_receipts = average_annual_receipts
        total_annual_donors = total_annual_donors 
        annual_donor_demands_age_specific = annual_donor_demands_age_specific

        # Calculate the ratios
        ratio_receipts_to_demands = round((average_annual_receipts['demand']) / (total_annual_donors), 2)
        ratio_age_specific = annual_donor_demands_age_specific.groupby(['year', 'age'])['demand'].sum() / annual_donor_demands_age_specific.groupby('year')['demand'].sum()

        # Naive extrapolation for the specified period
        forecast_dates = pd.date_range(start=start_period, end=end_period, freq='D')
        last_known_value = self.df['demand'].iloc[-1]
        forecast_values = pd.Series(last_known_value, index=forecast_dates)

        # Apply the ratios to the forecasted values
        forecast_with_uncertainty = forecast_values.copy()
        for date in forecast_dates:
            year = date.year
            age = self.df.loc[self.df.index.get_level_values('year') == date.year, 'age'].unique()[0] if 'age' in self.df.columns else None
            ratio_1 = ratio_receipts_to_demands.loc[year]
            ratio_2 = ratio_age_specific.loc[(year, age)] if age in ratio_age_specific.index.get_level_values('age') else 0
            forecast_with_uncertainty[date] = forecast_values[date] * ratio_1 * ratio_2

        # Return the forecast with applied uncertainty
        return forecast_with_uncertainty
    
# Calculate variability measures
variability_measures = forecasting.calculate_variability_measures()
print(variability_measures)

# Perform cohort analysis forecast with STL
forecast = forecasting.cohort_analysis_forecast_with_stl()
print(forecast)

# Visualize the forecast with uncertainty
forecasting.visualize_forecast_with_uncertainty()

# Visualize variability measures
#forecasting.visualize_variability_measures()

file_path = 'annual_unique_donors.csv'
forecasting = DemandForecasting(file_path)

# Apply uncertainty to forecast using naive extrapolation
start_period = pd.Timestamp('2024-04-01')
end_period = pd.Timestamp('2026-03-31')

annual_donor_receipts = pd.read_csv('annual_receipts.csv')
average_annual_receipts = annual_donor_receipts.rename(columns={'Yearly Receipt': 'demand'})
average_annual_receipts = average_annual_receipts[average_annual_receipts['year'].isin([2020, 2021, 2022, 2023])]
average_annual_receipts.set_index('year', inplace=True)
#pd.DataFrame({'year': [2022, 2023, 2024],'Yearly Receipt': [7.519675e+05, 9.700350e+05, 1.124615e+06]})

annual_donor_demands = pd.read_csv('annual_unique_donors.csv') #pd.Series([601254, 752898, 505219], index=[2022, 2023, 2024])
total_annual_donors = annual_donor_demands
total_annual_donors = total_annual_donors[total_annual_donors['year'].isin([2020, 2021, 2022, 2023])]
total_annual_donors = total_annual_donors.groupby('year')['demand'].sum()

annual_donor_demands_age_specific = pd.read_csv('annual_demands_age_specific.csv') #pd.DataFrame({})
annual_donor_demands_age_specific = annual_donor_demands_age_specific[annual_donor_demands_age_specific['year'].isin([2020, 2021, 2022, 2023])]

forecast_with_uncertainty = forecasting.apply_uncertainty_to_forecast(start_period, end_period, average_annual_receipts, total_annual_donors, annual_donor_demands_age_specific)
print(forecast_with_uncertainty)

In [ ]:
annual_donor_demands.xs(18, level='age')['demand'].dropna()

In [ ]:
annual_donor_demands['demand']

In [ ]:
# import pandas as pd
# import numpy as np
# from statsmodels.tsa.statespace.sarimax import SARIMAX

# class DemandForecasting:
#     def __init__(self, file_path):
#         self.df = pd.read_csv(file_path)
#         self.df['year'] = pd.to_datetime(self.df['year'], format='%Y')
#         self.df = self.df[self.df['year'].isin([2020, 2021, 2022, 2023])]
#         self.df.set_index(['age', 'year'], inplace=True)
#         self.variability_measures = None
#         self.forecast_df = None

#     def calculate_variability_measures(self):
#         measures = pd.DataFrame()
#         for age in self.df.index.get_level_values(0).unique():
#             ts = self.df.xs(age, level='age')['demand'].dropna()
#             std_dev = ts.std()
#             variance = ts.var()
#             range_val = ts.max() - ts.min()
#             iqr = ts.quantile(0.75) - ts.quantile(0.25)
#             cv = (std_dev / ts.mean()) * 100 if ts.mean() != 0 else 0
#             mad = ts.mad()
#             qd = iqr / 2
#             measures.loc[age, 'Standard Deviation'] = std_dev
#             measures.loc[age, 'Variance'] = variance
#             measures.loc[age, 'Range'] = range_val
#             measures.loc[age, 'IQR'] = iqr
#             measures.loc[age, 'CV'] = cv
#             measures.loc[age, 'MAD'] = mad
#             measures.loc[age, 'QD'] = qd
#         self.variability_measures = measures
#         return measures

#     def cohort_analysis_forecast_with_stl(self):
#         forecast_df = pd.DataFrame()
#         for age in self.df.index.get_level_values(0).unique():
#             print(f"age: {age}")
#             ts = self.df.xs(age, level='age')['demand'].dropna()
#             stl = STL(ts, seasonal=13)
#             result = stl.fit()
#             model = ExponentialSmoothing(result.trend, trend='add', seasonal='add', seasonal_periods=13)
#             fit_model = model.fit()
#             forecast_value = fit_model.forecast(1).iloc[0]
#             uncertainty_ratio = self.df.xs(age, level='age')['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in self.df.columns else 1
#             forecast_with_uncertainty = forecast_value * uncertainty_ratio
#             forecast_df.loc[age, 'Forecast'] = forecast_with_uncertainty
#         self.forecast_df = forecast_df
#         return forecast_df

#     def visualize_forecast_with_uncertainty(self):
#         plt.figure(figsize=(14, 7))
#         for age in self.df.index.get_level_values(0).unique():
#             ts = self.df.xs(age, level='age')['demand']
#             plt.plot(ts.index, ts, label=f'Original Demand (Age {age})', marker='o')
#         for age in self.forecast_df.index:
#             plt.plot(self.forecast_df.columns, self.forecast_df.loc[age], label=f'Forecasted Demand (Age {age})', marker='x')
#         for age in self.forecast_df.index:
#             adjusted_forecast = self.forecast_df.loc[age] * self.df.xs(age, level='age')['Uncertainty_Ratio'].mean()
#             plt.plot(self.forecast_df.columns, adjusted_forecast, label=f'Adjusted Forecast (Age {age})', linestyle='--')
#         plt.xlabel('Year')
#         plt.ylabel('Demand')
#         plt.title('Demand Forecast with Uncertainty Adjustment')
#         plt.legend()
#         plt.show()

#     def visualize_variability_measures(self):
#         plt.figure(figsize=(12, 6))
#         sns.boxplot(x='age', y='demand', data=self.df.reset_index())
#         plt.title('Box Plot of Demand by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         plt.figure(figsize=(12, 6))
#         sns.histplot(self.df['demand'], bins=20, kde=True)
#         plt.title('Histogram of Demand')
#         plt.xlabel('Demand')
#         plt.ylabel('Frequency')
#         plt.show()

#         plt.figure(figsize=(12, 6))
#         sns.violinplot(x='age', y='demand', data=self.df.reset_index())
#         plt.title('Violin Plot of Demand by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         mean_demand = self.df.groupby('age')['demand'].mean()
#         std_demand = self.df.groupby('age')['demand'].std()
#         plt.figure(figsize=(12, 6))
#         plt.errorbar(mean_demand.index, mean_demand, yerr=std_demand, fmt='o', capsize=5)
#         plt.title('Mean Demand with Standard Deviation by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         pivot_table = self.df.reset_index().pivot_table(index='age', columns='year', values='demand', aggfunc='mean')
#         plt.figure(figsize=(12, 6))
#         sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".1f")
#         plt.title('Heatmap of Mean Demand by Age and Year')
#         plt.xlabel('Year')
#         plt.ylabel('Age')
#         plt.show()

#     def apply_uncertainty_to_forecast(
#         self, sarima_order, seasonal_order, start_period, end_period, average_annual_receipts, total_annual_donors, annual_donor_demands_age_specific):
#         # Load additional data
#         average_annual_receipts = average_annual_receipts
#         total_annual_donors = total_annual_donors 
#         annual_donor_demands_age_specific = annual_donor_demands_age_specific

#         # Calculate the ratios
#         ratio_receipts_to_demands = round((average_annual_receipts['demand']) / (total_annual_donors), 2)
#         ratio_age_specific = annual_donor_demands_age_specific.groupby(['year', 'age'])['demand'].sum() / annual_donor_demands_age_specific.groupby('year')['demand'].sum()

#         # Fit SARIMA model
#         sarima_model = SARIMAX(self.df['demand'], order=sarima_order, seasonal_order=seasonal_order)
#         sarima_results = sarima_model.fit()

#         # Forecast for the specified period
#         #forecast = sarima_results.get_forecast(steps=(end_period - start_period).days)
#         forecast_dates = pd.date_range(start=start_period, end=end_period, freq='D')
#         forecast = sarima_results.get_forecast(steps=len(forecast_dates))
#         forecast_values = forecast.predicted_mean

#         # Apply the ratios to the forecasted values
#         forecast_with_uncertainty = forecast_values.copy()
#         for date in forecast_dates: #forecast_values.index:
#             year = date.year
#             print(f"For year: {year}")
#             age = self.df.loc[self.df.index.get_level_values('year') == date.year, 'age'].unique()[0] if 'age' in self.df.columns else None
#             ratio_1 = ratio_receipts_to_demands.loc[year]
#             ratio_2 = ratio_age_specific.loc[(year, age)] if age in ratio_age_specific.index.get_level_values('age') else 0
#             forecast_with_uncertainty[date] = forecast_values[date] * ratio_1 * ratio_2
#             print(f"""For year: {year}, the ratio_receipts_to_demands is: {ratio_1} 
#             and the ratio_receipts_to_demands is: {ratio_2}""")           
#             #age = self.df.loc[date, 'age'] if 'age' in self.df.columns else None
#             #age = self.df.loc[self.df.index.get_level_values('year') == date.year, 'age'].unique()[0]
#             #ratio_1 = ratio_receipts_to_demands.loc[year]
#             #ratio_1 = ratio_receipts_to_demands.reset_index().loc[:].demand #.year
#             #ratio_2 = ratio_age_specific.loc[(year, age)] if age in ratio_age_specific.index.get_level_values('age') else 0

#         # Return the forecast with applied uncertainty
#         return forecast_with_uncertainty

# # Example usage
# file_path = 'annual_unique_donors.csv'
# forecasting = DemandForecasting(file_path)
# sarima_order = (1, 1, 1)
# seasonal_order = (1, 1, 1, 12)
# start_period = pd.Timestamp('2024-01-01')  #('2024-04-01')
# end_period = pd.Timestamp('2026-01-01')
# annual_donor_receipts = pd.read_csv('annual_receipts.csv')
# average_annual_receipts = annual_donor_receipts.rename(columns={'Yearly Receipt': 'demand'})
# average_annual_receipts = average_annual_receipts[average_annual_receipts['year'].isin([2020, 2021, 2022, 2023])]
# average_annual_receipts.set_index('year', inplace=True)
# #pd.DataFrame({'year': [2022, 2023, 2024],'Yearly Receipt': [7.519675e+05, 9.700350e+05, 1.124615e+06]})

# annual_donor_demands = pd.read_csv('annual_unique_donors.csv') #pd.Series([601254, 752898, 505219], index=[2022, 2023, 2024])
# total_annual_donors = annual_donor_demands
# total_annual_donors = total_annual_donors[total_annual_donors['year'].isin([2020, 2021, 2022, 2023])]
# total_annual_donors = total_annual_donors.groupby('year')['demand'].sum()

# annual_donor_demands_age_specific = pd.read_csv('annual_demands_age_specific.csv') #pd.DataFrame({})
# annual_donor_demands_age_specific = annual_donor_demands_age_specific[annual_donor_demands_age_specific['year'].isin([2020, 2021, 2022, 2023])]

# forecast_with_uncertainty = forecasting.apply_uncertainty_to_forecast(
#     sarima_order, seasonal_order, start_period, end_period, average_annual_receipts, total_annual_donors, annual_donor_demands_age_specific)
# print(forecast_with_uncertainty)


In [ ]:
forecast_dates

In [ ]:
ratio_receipts_to_demands = round((average_annual_receipts['demand']) / (total_annual_donors), 2)
#ratio_receipts_to_demands.reset_index()
#ratio_receipts_to_demands.loc['year']
#ratio_receipts_to_demands.reset_index()
ratio_1 = ratio_receipts_to_demands.reset_index().loc[:].year #ratio_receipts_to_demands.loc[year]
ratio_1 = ratio_receipts_to_demands.reset_index().loc[:].year

#ratio_age_specific = annual_donor_demands_age_specific.groupby(['year', 'age'])['demand'].sum() / annual_donor_demands_age_specific.groupby('year')['demand'].sum()
#ratio_2 = ratio_age_specific.loc[(year, age)] if age in ratio_age_specific.index.get_level_values('age') else 0
#forecast_with_uncertainty[date] = forecast_values[date] * ratio_1 * ratio_2


In [ ]:
ratio_receipts_to_demands.loc[2020]

In [ ]:
pd.date_range(start=start_period, end=end_period, freq='D')

In [ ]:
forecast_values

In [ ]:
# import pandas as pd
# import numpy as np
# from statsmodels.tsa.statespace.sarimax import SARIMAX

# class DemandForecasting:
#     def __init__(self, file_path):
#         self.df = pd.read_csv(file_path)
#         self.df['year'] = pd.to_datetime(self.df['year'], format='%Y')
#         self.df.set_index(['age', 'year'], inplace=True)
#         self.variability_measures = None
#         self.forecast_df = None

#     def calculate_variability_measures(self):
#         measures = pd.DataFrame()
#         for age in self.df.index.get_level_values(0).unique():
#             ts = self.df.xs(age, level='age')['demand'].dropna()
#             std_dev = ts.std()
#             variance = ts.var()
#             range_val = ts.max() - ts.min()
#             iqr = ts.quantile(0.75) - ts.quantile(0.25)
#             cv = (std_dev / ts.mean()) * 100 if ts.mean() != 0 else 0
#             mad = ts.mad()
#             qd = iqr / 2
#             measures.loc[age, 'Standard Deviation'] = std_dev
#             measures.loc[age, 'Variance'] = variance
#             measures.loc[age, 'Range'] = range_val
#             measures.loc[age, 'IQR'] = iqr
#             measures.loc[age, 'CV'] = cv
#             measures.loc[age, 'MAD'] = mad
#             measures.loc[age, 'QD'] = qd
#         self.variability_measures = measures
#         return measures

#     def cohort_analysis_forecast_with_stl(self):
#         forecast_df = pd.DataFrame()
#         for age in self.df.index.get_level_values(0).unique():
#             ts = self.df.xs(age, level='age')['demand'].dropna()
#             stl = STL(ts, seasonal=13)
#             result = stl.fit()
#             model = ExponentialSmoothing(result.trend, trend='add', seasonal='add', seasonal_periods=13)
#             fit_model = model.fit()
#             forecast_value = fit_model.forecast(1).iloc[0]
#             uncertainty_ratio = self.df.xs(age, level='age')['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in self.df.columns else 1
#             forecast_with_uncertainty = forecast_value * uncertainty_ratio
#             forecast_df.loc[age, 'Forecast'] = forecast_with_uncertainty
#         self.forecast_df = forecast_df
#         return forecast_df

#     def visualize_forecast_with_uncertainty(self):
#         plt.figure(figsize=(14, 7))
#         for age in self.df.index.get_level_values(0).unique():
#             ts = self.df.xs(age, level='age')['demand']
#             plt.plot(ts.index, ts, label=f'Original Demand (Age {age})', marker='o')
#         for age in self.forecast_df.index:
#             plt.plot(self.forecast_df.columns, self.forecast_df.loc[age], label=f'Forecasted Demand (Age {age})', marker='x')
#         for age in self.forecast_df.index:
#             adjusted_forecast = self.forecast_df.loc[age] * self.df.xs(age, level='age')['Uncertainty_Ratio'].mean()
#             plt.plot(self.forecast_df.columns, adjusted_forecast, label=f'Adjusted Forecast (Age {age})', linestyle='--')
#         plt.xlabel('Year')
#         plt.ylabel('Demand')
#         plt.title('Demand Forecast with Uncertainty Adjustment')
#         plt.legend()
#         plt.show()

#     def visualize_variability_measures(self):
#         plt.figure(figsize=(12, 6))
#         sns.boxplot(x='age', y='demand', data=self.df.reset_index())
#         plt.title('Box Plot of Demand by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         plt.figure(figsize=(12, 6))
#         sns.histplot(self.df['demand'], bins=20, kde=True)
#         plt.title('Histogram of Demand')
#         plt.xlabel('Demand')
#         plt.ylabel('Frequency')
#         plt.show()

#         plt.figure(figsize=(12, 6))
#         sns.violinplot(x='age', y='demand', data=self.df.reset_index())
#         plt.title('Violin Plot of Demand by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         mean_demand = self.df.groupby('age')['demand'].mean()
#         std_demand = self.df.groupby('age')['demand'].std()
#         plt.figure(figsize=(12, 6))
#         plt.errorbar(mean_demand.index, mean_demand, yerr=std_demand, fmt='o', capsize=5)
#         plt.title('Mean Demand with Standard Deviation by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         pivot_table = self.df.reset_index().pivot_table(index='age', columns='year', values='demand', aggfunc='mean')
#         plt.figure(figsize=(12, 6))
#         sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".1f")
#         plt.title('Heatmap of Mean Demand by Age and Year')
#         plt.xlabel('Year')
#         plt.ylabel('Age')
#         plt.show()

#     def apply_uncertainty_to_forecast(self, sarima_order, seasonal_order, start_period, end_period):
#         # Load additional data
#         annual_donor_receipts = pd.DataFrame({
#             'year': [2022, 2023, 2024],
#             'Yearly Receipt': [7.519675e+05, 9.700350e+05, 1.124615e+06]
#         })
#         annual_donor_demands = pd.Series([601254, 752898, 505219], index=[2022, 2023, 2024])
#         annual_donor_demands_age_specific = pd.DataFrame({
#             # Assuming this DataFrame contains the age-specific demand data as shown in the context
#         })

#         # Calculate the ratios
#         ratio_receipts_to_demands = annual_donor_receipts['Yearly Receipt'] / annual_donor_demands
#         ratio_age_specific = annual_donor_demands_age_specific.groupby(['year', 'age'])['demand'].sum() / annual_donor_demands_age_specific.groupby('year')['demand'].sum()

#         # Fit SARIMA model
#         sarima_model = SARIMAX(self.df['demand'], order=sarima_order, seasonal_order=seasonal_order)
#         sarima_results = sarima_model.fit()

#         # Forecast for the specified period
#         forecast = sarima_results.get_forecast(steps=(end_period - start_period).days)
#         forecast_values = forecast.predicted_mean

#         # Apply the ratios to the forecasted values
#         forecast_with_uncertainty = forecast_values.copy()
#         for date in forecast_values.index:
#             year = date.year
#             age = self.df.loc[date, 'age'] if 'age' in self.df.columns else None
#             ratio_1 = ratio_receipts_to_demands.loc[year]
#             ratio_2 = ratio_age_specific.loc[(year, age)] if age in ratio_age_specific.index.get_level_values('age') else 0
#             forecast_with_uncertainty[date] = forecast_values[date] * ratio_1 * ratio_2

#         # Return the forecast with applied uncertainty
#         return forecast_with_uncertainty

# # Example usage
# file_path = 'annual_unique_donors.csv'
# forecasting = DemandForecasting(file_path)
# sarima_order = (1, 1, 1)
# seasonal_order = (1, 1, 1, 12)
# start_period = pd.Timestamp('2024-04-01')
# end_period = pd.Timestamp('2026-03-31')
# forecast_with_uncertainty = forecasting.apply_uncertainty_to_forecast(sarima_order, seasonal_order, start_period, end_period)
# print(forecast_with_uncertainty)

In [ ]:
end_period

In [ ]:
annual_donor_demands_age_specific.groupby(['year', 'age'])['demand'].sum() / annual_donor_demands_age_specific.groupby('year')['demand'].sum()

In [ ]:
# # Explanation
# # Class Initialization:
# # Loads the dataset and sets up the DataFrame.
# # Calculate Variability Measures:
# # Computes various measures of variability for each age cohort.
# # Cohort Analysis Forecast with STL:
# # Applies STL decomposition and Holt-Winters forecasting, adjusting the forecast with the uncertainty ratio.
# # Visualize Forecast with Uncertainty:
# # Plots the original demand, forecasted demand, and adjusted forecast with uncertainty.
# # Visualize Variability Measures:
# # Creates various plots (box plot, histogram, violin plot, scatter plot with error bars, and heatmap) to visualize the variability measures.
# # You can use this class to load your data, perform the analysis, and visualize the results. Feel free to run the code and let me know if you need any further assistance!

# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# from statsmodels.tsa.seasonal import STL
# from statsmodels.tsa.holtwinters import ExponentialSmoothing

# class DemandForecasting:
#     def __init__(self, historical_data):
#         self.df = historical_data
#         self.df['year'] = pd.to_datetime(self.df['year'], format='%Y')
#         self.df.set_index(['age', 'year'], inplace=True)
#         self.variability_measures = None
#         self.forecast_df = None

#     def calculate_variability_measures(self):
#         measures = pd.DataFrame()
#         for age in self.df.index.get_level_values(0).unique():
#             ts = self.df.xs(age, level='age')['demand'].dropna()
#             std_dev = ts.std()
#             variance = ts.var()
#             range_val = ts.max() - ts.min()
#             iqr = ts.quantile(0.75) - ts.quantile(0.25)
#             cv = (std_dev / ts.mean()) * 100 if ts.mean() != 0 else 0
#             mad = ts.mad()
#             qd = iqr / 2
#             measures.loc[age, 'Standard Deviation'] = std_dev
#             measures.loc[age, 'Variance'] = variance
#             measures.loc[age, 'Range'] = range_val
#             measures.loc[age, 'IQR'] = iqr
#             measures.loc[age, 'CV'] = cv
#             measures.loc[age, 'MAD'] = mad
#             measures.loc[age, 'QD'] = qd
#         self.variability_measures = measures
#         return measures

#     def cohort_analysis_forecast_with_stl(self):
#         forecast_df = pd.DataFrame()
#         for age in self.df.index.get_level_values(0).unique():
#             ts = self.df.xs(age, level='age')['demand'].dropna()
#             stl = STL(ts, seasonal=13)
#             result = stl.fit()
#             model = ExponentialSmoothing(result.trend, trend='add', seasonal='add', seasonal_periods=13)
#             fit_model = model.fit()
#             forecast_value = fit_model.forecast(1).iloc[0]
#             uncertainty_ratio = self.df.xs(age, level='age')['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in self.df.columns else 1
#             forecast_with_uncertainty = forecast_value * uncertainty_ratio
#             forecast_df.loc[age, 'Forecast'] = forecast_with_uncertainty
#         self.forecast_df = forecast_df
#         return forecast_df

#     def visualize_forecast_with_uncertainty(self):
#         plt.figure(figsize=(14, 7))
#         for age in self.df.index.get_level_values(0).unique():
#             ts = self.df.xs(age, level='age')['demand']
#             plt.plot(ts.index, ts, label=f'Original Demand (Age {age})', marker='o')
#         for age in self.forecast_df.index:
#             plt.plot(self.forecast_df.columns, self.forecast_df.loc[age], label=f'Forecasted Demand (Age {age})', marker='x')
#         for age in self.forecast_df.index:
#             adjusted_forecast = self.forecast_df.loc[age] * self.df.xs(age, level='age')['Uncertainty_Ratio'].mean()
#             plt.plot(self.forecast_df.columns, adjusted_forecast, label=f'Adjusted Forecast (Age {age})', linestyle='--')
#         plt.xlabel('Year')
#         plt.ylabel('Demand')
#         plt.title('Demand Forecast with Uncertainty Adjustment')
#         plt.legend()
#         plt.show()

#     def visualize_variability_measures(self):
#         plt.figure(figsize=(12, 6))
#         sns.boxplot(x='age', y='demand', data=self.df.reset_index())
#         plt.title('Box Plot of Demand by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         plt.figure(figsize=(12, 6))
#         sns.histplot(self.df['demand'], bins=20, kde=True)
#         plt.title('Histogram of Demand')
#         plt.xlabel('Demand')
#         plt.ylabel('Frequency')
#         plt.show()

#         plt.figure(figsize=(12, 6))
#         sns.violinplot(x='age', y='demand', data=self.df.reset_index())
#         plt.title('Violin Plot of Demand by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         mean_demand = self.df.groupby('age')['demand'].mean()
#         std_demand = self.df.groupby('age')['demand'].std()
#         plt.figure(figsize=(12, 6))
#         plt.errorbar(mean_demand.index, mean_demand, yerr=std_demand, fmt='o', capsize=5)
#         plt.title('Mean Demand with Standard Deviation by Age')
#         plt.xlabel('Age')
#         plt.ylabel('Demand')
#         plt.show()

#         pivot_table = self.df.reset_index().pivot_table(index='age', columns='year', values='demand', aggfunc='mean')
#         plt.figure(figsize=(12, 6))
#         sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".1f")
#         plt.title('Heatmap of Mean Demand by Age and Year')
#         plt.xlabel('Year')
#         plt.ylabel('Age')
#         plt.show()

# # Example usage
# historical_data = annual_unique_donors
# forecasting = DemandForecasting(historical_data)
# variability_measures = forecasting.calculate_variability_measures()
# forecast = forecasting.cohort_analysis_forecast_with_stl()
# forecasting.visualize_forecast_with_uncertainty()
# forecasting.visualize_variability_measures()


In [ ]:
can you add TSL seasonal forecasting to the code below based on the attched data?
text
annual_unique_donors.csv

Copilot
Executing Task:

add TSL seasonal forecasting to the code based on the attached data


You
Sent by you:
can you add TSL seasonal forecasting to the code below based on the attched data? def cohort_analysis_forecast(df):
    # Create a pivot table with 'year' as columns, 'age' as index, and 'demand' as values
    pivot = df.pivot_table(index='age', columns='year', values='demand', aggfunc='sum')
    
    # Calculate the growth rate for each age cohort
    growth_rates = pivot.pct_change(axis=1)
    
    # Calculate the average growth rate for each age cohort
    avg_growth_rates = growth_rates.mean(axis=1)
    
    # Get the last year of data for each age cohort
    last_year_data = pivot.iloc[:, -1]
    
    # Forecast the next year's demand by applying the average growth rate to the last year's data
    forecast = last_year_data * (1 + avg_growth_rates)
    
    # Incorporate uncertainty by multiplying the forecast by the 'Uncertainty_Ratio'
    # Assuming 'Uncertainty_Ratio' is a column in the dataframe
    uncertainty_ratio = df['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in df.columns else 1
    forecast_with_uncertainty = forecast * uncertainty_ratio
    
    # Return the forecast with uncertainty
    return forecast_with_uncertainty

# Call the function and print the forecast
forecast = cohort_analysis_forecast(df)
print(forecast)
text
annual_unique_donors.csv
Copilot
Executing Task:

add TSL seasonal forecasting to the code based on the attached data


Sent by Copilot:
It looks like there was an issue with the code execution because the dataset does not contain a ‘Date’ column. Let’s adjust the code to add TSL seasonal forecasting based on the data you provided.

Here’s the updated code:

Python

import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load the dataset
df = pd.read_csv('annual_unique_donors.csv')

# Assuming the dataframe has a 'year' column and a 'demand' column
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.set_index('year', inplace=True)

# Decompose the time series to observe seasonal patterns
decomposition = seasonal_decompose(df['demand'], model='additive', period=1)

# Fit the TSL model with seasonal components
model = ExponentialSmoothing(df['demand'], trend='add', seasonal='add', seasonal_periods=1)
fit_model = model.fit()

# Forecast the next period
forecast = fit_model.forecast(1)

# Add the forecast to the original function
def cohort_analysis_forecast(df, forecast_value):
    # Create a pivot table with 'year' as columns, 'age' as index, and 'demand' as values
    pivot = df.pivot_table(index='age', columns='year', values='demand', aggfunc='sum')
    
    # Calculate the growth rate for each age cohort
    growth_rates = pivot.pct_change(axis=1)
    
    # Calculate the average growth rate for each age cohort
    avg_growth_rates = growth_rates.mean(axis=1)
    
    # Get the last year of data for each age cohort
    last_year_data = pivot.iloc[:, -1]
    
    # Forecast the next year's demand by applying the average growth rate to the last year's data
    forecast = last_year_data * (1 + avg_growth_rates)
    
    # Incorporate uncertainty by multiplying the forecast by the 'Uncertainty_Ratio'
    # Assuming 'Uncertainty_Ratio' is a column in the dataframe
    uncertainty_ratio = df['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in df.columns else 1
    forecast_with_uncertainty = forecast * uncertainty_ratio
    
    # Add the TSL forecast value
    forecast_with_uncertainty['TSL_Forecast'] = forecast_value
    
    # Return the forecast with uncertainty
    return forecast_with_uncertainty

# Call the function and print the forecast with TSL seasonal forecasting
forecast_with_tsl = cohort_analysis_forecast(df, forecast.values[0])
print(forecast_with_tsl)
AI-generated code. Review and use carefully. More info on FAQ.
This code integrates TSL seasonal forecasting into your existing cohort analysis forecast function. It uses the ExponentialSmoothing model from the statsmodels library to forecast the next period based on the seasonal decomposition of the demand data. The forecast value is then incorporated into the final forecast with uncertainty.

Feel free to run this code and let me know if you encounter any issues or need further adjustments!

Learn more

1

annual_unique_donors.csv
You
Sent by you:
How can I visualize the forecast results?
Copilot
Sent by Copilot:
You can visualize the forecast results using various plotting libraries in Python, such as matplotlib or seaborn. Here’s an example of how you can visualize the forecast results using matplotlib:

Python

import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'forecast_with_tsl' is the DataFrame returned by the cohort_analysis_forecast function
# and it includes the 'TSL_Forecast' column

def visualize_forecast(df, forecast_with_tsl):
    # Plot the original demand data
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df['demand'], label='Original Demand', marker='o')
    
    # Plot the forecasted demand
    plt.plot(forecast_with_tsl.index, forecast_with_tsl, label='Forecasted Demand', marker='x')
    
    # Highlight the TSL forecast
    plt.axvline(x=forecast_with_tsl.index[-1], color='r', linestyle='--', label='TSL Forecast')
    
    # Add labels and title
    plt.xlabel('Year')
    plt.ylabel('Demand')
    plt.title('Demand Forecast with TSL Seasonal Forecasting')
    plt.legend()
    
    # Show the plot
    plt.show()

# Call the visualization function
visualize_forecast(df, forecast_with_tsl)
AI-generated code. Review and use carefully. More info on FAQ.
This code will create a line plot showing the original demand data and the forecasted demand, with a vertical line highlighting the TSL forecast. You can customize the plot further by adding more details or adjusting the style to suit your needs.

Feel free to run this code and let me know if you need any further assistance!

You
Sent by you:
can you add STL for forecasting time series data forecasting model and adjust and change the above code to ensure that multiplying the forecast value for a specific age in a specific year to be corresponded to the related age and year when choosing age_specific Uncertainty_Ratio to the code below based on the attched data? # Load the dataset
df = pd.read_csv('annual_unique_donors.csv')                                                                                                         def cohort_analysis_forecast_with_seasonality(df):
    # Create a pivot table with 'year' as columns, 'age' as index, and 'demand' as values
    pivot = df.pivot_table(index='age', columns='year', values='demand', aggfunc='sum')
    
    # Initialize a DataFrame to store the forecast
    forecast_df = pd.DataFrame(index=pivot.index)
    
    # Loop through each age cohort to apply seasonal decomposition and forecast
    for age in pivot.index:
        # Get the time series for the current age cohort
        ts = pivot.loc[age].dropna()
        
        # Apply seasonal decomposition
        decomposition = seasonal_decompose(ts, model='multiplicative', period=12)
        
        # Extract the trend and seasonal components
        trend = decomposition.trend.dropna()
        seasonal = decomposition.seasonal.dropna()
        
        # Calculate the average growth rate for the trend component
        growth_rate = trend.pct_change().mean()
        
        # Forecast the next year's trend component
        last_trend_value = trend.iloc[-1]
        forecast_trend = last_trend_value * (1 + growth_rate)
        
        # Get the seasonal component for the next period (assuming seasonality repeats)
        next_seasonal_value = seasonal.iloc[-12 % len(seasonal)]
        
        # Combine the forecasted trend and seasonal components
        forecast_value = forecast_trend * next_seasonal_value
        
        # Incorporate uncertainty by multiplying the forecast by the 'Uncertainty_Ratio'
        uncertainty_ratio = df['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in df.columns else 1
        forecast_with_uncertainty = forecast_value * uncertainty_ratio
        
        # Store the forecast in the DataFrame
        forecast_df.loc[age, 'Forecast'] = forecast_with_uncertainty
    
    # Return the forecast DataFrame
    return forecast_df

# Call the function and print the forecast
forecast = cohort_analysis_forecast_with_seasonality(df)
print(forecast)
text
annual_unique_donors.csv
Copilot
Executing Task:

add STL for forecasting time series data forecasting model and adjust and change the code to ensure that multiplying the forecast value for a specific age in a specific year to be corresponded to the related age and year when choosing age_specific Uncertainty_Ratio to the code based on the attached data


Sent by Copilot:
It looks like there was an issue with the code execution because the dataset does not contain a ‘Date’ column. Let’s adjust the code to add STL seasonal forecasting based on the data you provided.

Here’s the updated code:

Python

import pandas as pd
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load the dataset
df = pd.read_csv('annual_unique_donors.csv')

# Assuming the dataframe has 'year' and 'age' columns and a 'demand' column
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.set_index(['age', 'year'], inplace=True)

# Function to apply STL and forecast
def cohort_analysis_forecast_with_stl(df):
    # Initialize a DataFrame to store the forecast
    forecast_df = pd.DataFrame()

    # Loop through each age cohort to apply STL and forecast
    for age in df.index.get_level_values(0).unique():
        # Get the time series for the current age cohort
        ts = df.xs(age, level='age')['demand'].dropna()
        
        # Apply STL decomposition
        stl = STL(ts, seasonal=13)
        result = stl.fit()
        
        # Forecast the next year's demand using Holt-Winters method
        model = ExponentialSmoothing(result.trend, trend='add', seasonal='add', seasonal_periods=13)
        fit_model = model.fit()
        forecast_value = fit_model.forecast(1).iloc[0]
        
        # Get the corresponding 'Uncertainty_Ratio' for the specific age and year
        uncertainty_ratio = df.xs(age, level='age')['Uncertainty_Ratio'].mean() if 'Uncertainty_Ratio' in df.columns else 1
        
        # Multiply the forecast value by the 'Uncertainty_Ratio'
        forecast_with_uncertainty = forecast_value * uncertainty_ratio
        
        # Store the forecast in the DataFrame
        forecast_df.loc[age, 'Forecast'] = forecast_with_uncertainty
    
    # Return the forecast DataFrame
    return forecast_df

# Call the function and print the forecast
forecast = cohort_analysis_forecast_with_stl(df)
print(forecast)
AI-generated code. Review and use carefully. More info on FAQ.
This code integrates STL seasonal forecasting into your existing cohort analysis forecast function. It uses the STL decomposition from the statsmodels library to decompose the time series data and then applies the Holt-Winters method for forecasting. The forecast value is then adjusted by the age-specific Uncertainty_Ratio.

Feel free to run this code and let me know if you encounter any issues or need further adjustments!

Learn more

1

annual_unique_donors.csv
You
Sent by you:
How can I interpret the forecast results?
Copilot
Sent by Copilot:
Interpreting the forecast results involves understanding the different components and what they represent. Here’s a breakdown of how to interpret the forecast results from the code provided:

Original Demand Data:
This represents the historical demand data for each age cohort over the years. It’s the baseline data used for forecasting.
STL Decomposition:
Seasonal Component: This shows the repeating patterns or cycles in the data, such as yearly seasonality.
Trend Component: This indicates the long-term progression of the data, showing whether the demand is generally increasing or decreasing over time.
Residual Component: This represents the random noise or irregular fluctuations in the data that are not explained by the trend or seasonal components.
Forecasted Demand:
This is the predicted demand for the next period based on the historical data and the STL decomposition. It combines the trend and seasonal components to project future demand.
Uncertainty Ratio:
This factor adjusts the forecasted demand to account for uncertainty. It can vary by age and year, reflecting the variability in the data.
Forecast with Uncertainty:
This is the final forecasted demand after adjusting for uncertainty. It provides a more realistic prediction by incorporating potential variations.
Example Interpretation
Let’s say you have the following forecast results for a specific age cohort:

Original Demand Data: Shows an increasing trend over the past 5 years.
Seasonal Component: Indicates higher demand during certain months of the year.
Trend Component: Shows a steady upward trend.
Residual Component: Contains some random fluctuations but no significant outliers.
Forecasted Demand: Predicts an increase in demand for the next year.
Uncertainty Ratio: Adjusts the forecast slightly higher due to observed variability.
Visualization
To visualize these results, you can plot the original demand data, the decomposed components (seasonal, trend, residual), and the forecasted demand. This will help you see how the forecast is derived and how the different components contribute to the final prediction.

Here’s an example of how to visualize the forecast results using matplotlib:

Python

import matplotlib.pyplot as plt

def visualize_forecast(df, forecast_df):
    # Plot the original demand data
    plt.figure(figsize=(14, 7))
    for age in df.index.get_level_values(0).unique():
        ts = df.xs(age, level='age')['demand']
        plt.plot(ts.index, ts, label=f'Original Demand (Age {age})', marker='o')
    
    # Plot the forecasted demand
    for age in forecast_df.index:
        plt.plot(forecast_df.columns, forecast_df.loc[age], label=f'Forecasted Demand (Age {age})', marker='x')
    
    # Add labels and title
    plt.xlabel('Year')
    plt.ylabel('Demand')
    plt.title('Demand Forecast with STL Seasonal Forecasting')
    plt.legend()
    
    # Show the plot
    plt.show()

# Call the visualization function
visualize_forecast(df, forecast)
AI-generated code. Review and use carefully. More info on FAQ.
This code will create a line plot showing the original demand data and the forecasted demand for each age cohort. You can customize the plot further to highlight specific components or add more details.

Feel free to run this code and let me know if you need any further assistance!

You
Sent by you:
How do I interpret the uncertainty ratio?
Copilot
Sent by Copilot:
The uncertainty ratio is a factor used to adjust the forecasted demand to account for variability and potential errors in the prediction. Here’s how you can interpret it:

Understanding the Uncertainty Ratio
Purpose:
The uncertainty ratio is used to incorporate the inherent uncertainty in the forecast. It adjusts the forecasted values to reflect the possible range of outcomes, considering the variability in the historical data.
Calculation:
In your code, the uncertainty ratio is calculated as the mean of the ‘Uncertainty_Ratio’ column for each specific age cohort. If this column is not present, a default value of 1 is used, indicating no adjustment.
Impact on Forecast:
The forecasted demand is multiplied by the uncertainty ratio. If the uncertainty ratio is greater than 1, it increases the forecasted value, indicating higher variability and potential for greater demand. If it is less than 1, it decreases the forecasted value, indicating lower variability and potential for lesser demand.
Example Interpretation
Let’s say you have a forecasted demand of 100 units for a specific age cohort, and the uncertainty ratio for that cohort is 1.2. The adjusted forecast would be:

[ \text{Adjusted Forecast} = \text{Forecasted Demand} \times \text{Uncertainty Ratio} ] [ \text{Adjusted Forecast} = 100 \times 1.2 = 120 ]

This means that, considering the variability in the data, the forecasted demand is adjusted to 120 units.

Practical Use
Higher Uncertainty Ratio: Indicates more variability in the historical data, suggesting that the forecasted demand could be higher than initially predicted.
Lower Uncertainty Ratio: Indicates less variability, suggesting that the forecasted demand could be lower or more stable.
Visualization
To visualize the impact of the uncertainty ratio, you can plot the original forecasted values and the adjusted values side by side. This will help you see how the uncertainty ratio modifies the forecast.

Here’s an example of how to visualize this using matplotlib:

Python

import matplotlib.pyplot as plt

def visualize_forecast_with_uncertainty(df, forecast_df):
    # Plot the original demand data
    plt.figure(figsize=(14, 7))
    for age in df.index.get_level_values(0).unique():
        ts = df.xs(age, level='age')['demand']
        plt.plot(ts.index, ts, label=f'Original Demand (Age {age})', marker='o')
    
    # Plot the forecasted demand
    for age in forecast_df.index:
        plt.plot(forecast_df.columns, forecast_df.loc[age], label=f'Forecasted Demand (Age {age})', marker='x')
    
    # Highlight the adjusted forecast with uncertainty
    for age in forecast_df.index:
        adjusted_forecast = forecast_df.loc[age] * df.xs(age, level='age')['Uncertainty_Ratio'].mean()
        plt.plot(forecast_df.columns, adjusted_forecast, label=f'Adjusted Forecast (Age {age})', linestyle='--')
    
    # Add labels and title
    plt.xlabel('Year')
    plt.ylabel('Demand')
    plt.title('Demand Forecast with Uncertainty Adjustment')
    plt.legend()
    
    # Show the plot
    plt.show()

# Call the visualization function
visualize_forecast_with_uncertainty(df, forecast)
AI-generated code. Review and use carefully. More info on FAQ.
This code will create a line plot showing the original demand data, the forecasted demand, and the adjusted forecast with uncertainty for each age cohort. This visualization helps in understanding how the uncertainty ratio affects the forecasted values.

Feel free to run this code and let me know if you need any further assistance!

You
Sent by you:
How can I calculate the uncertainty ratio for my specific data?
Copilot
Sent by Copilot:
To calculate the uncertainty ratio for your specific data, you need to quantify the variability or uncertainty in your historical demand data. This can be done using statistical measures such as the standard deviation or variance. Here’s a step-by-step guide on how to calculate the uncertainty ratio:

Step-by-Step Guide
Load Your Data: Ensure your data is loaded into a DataFrame and properly formatted.
Calculate the Standard Deviation: Compute the standard deviation of the demand for each age cohort. The standard deviation measures the amount of variation or dispersion in the data.
Calculate the Mean Demand: Compute the mean (average) demand for each age cohort.
Calculate the Uncertainty Ratio: The uncertainty ratio can be calculated as the ratio of the standard deviation to the mean demand. This gives a relative measure of variability.
Example Code
Here’s an example of how you can calculate the uncertainty ratio for each age cohort in your dataset:

Python

import pandas as pd

# Load the dataset
df = pd.read_csv('annual_unique_donors.csv')

# Assuming the dataframe has 'year', 'age', and 'demand' columns
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.set_index(['age', 'year'], inplace=True)

# Calculate the uncertainty ratio for each age cohort
def calculate_uncertainty_ratio(df):
    # Initialize a DataFrame to store the uncertainty ratios
    uncertainty_ratios = pd.DataFrame()

    # Loop through each age cohort
    for age in df.index.get_level_values(0).unique():
        # Get the demand time series for the current age cohort
        ts = df.xs(age, level='age')['demand'].dropna()
        
        # Calculate the standard deviation and mean demand
        std_dev = ts.std()
        mean_demand = ts.mean()
        
        # Calculate the uncertainty ratio
        uncertainty_ratio = std_dev / mean_demand if mean_demand != 0 else 0
        
        # Store the uncertainty ratio in the DataFrame
        uncertainty_ratios.loc[age, 'Uncertainty_Ratio'] = uncertainty_ratio
    
    return uncertainty_ratios

# Call the function and print the uncertainty ratios
uncertainty_ratios = calculate_uncertainty_ratio(df)
print(uncertainty_ratios)
AI-generated code. Review and use carefully. More info on FAQ.
Explanation
Standard Deviation (std_dev): Measures the spread of the demand values around the mean. A higher standard deviation indicates more variability.
Mean Demand (mean_demand): The average demand for the age cohort.
Uncertainty Ratio (uncertainty_ratio): The ratio of the standard deviation to the mean demand. This provides a relative measure of the variability in the demand data.
Using the Uncertainty Ratio
Once you have calculated the uncertainty ratios, you can use them to adjust your forecasted demand values. This will help you account for the variability in the historical data and provide a more realistic forecast.

Feel free to run this code with your data and let me know if you need any further assistance!

You
Sent by you:
How can I interpret the uncertainty ratio in my specific context?
Copilot
Sent by Copilot:
Interpreting the uncertainty ratio in your specific context involves understanding how it reflects the variability and reliability of your forecasted demand. Here’s a detailed guide to help you interpret it:

What the Uncertainty Ratio Represents
Variability in Historical Data:
The uncertainty ratio is a measure of how much the demand has varied historically for a specific age cohort. A higher ratio indicates more variability, while a lower ratio indicates more stability.
Adjustment to Forecast:
The ratio adjusts the forecasted demand to account for this variability. It helps in providing a range of possible outcomes rather than a single point estimate, making the forecast more realistic.
Interpreting Different Values
High Uncertainty Ratio (> 1):
Implication: There is significant variability in the historical demand data.
Interpretation: The forecasted demand is adjusted upwards to account for potential higher demand. This suggests that the future demand could be more volatile and less predictable.
Example: If the forecasted demand is 100 units and the uncertainty ratio is 1.5, the adjusted forecast would be 150 units.
Low Uncertainty Ratio (< 1):
Implication: There is less variability in the historical demand data.
Interpretation: The forecasted demand is adjusted downwards, indicating more stable and predictable future demand.
Example: If the forecasted demand is 100 units and the uncertainty ratio is 0.8, the adjusted forecast would be 80 units.
Uncertainty Ratio = 1:
Implication: The variability in the historical data is average or negligible.
Interpretation: The forecasted demand remains unchanged, suggesting that the historical data is a good predictor of future demand without significant adjustments.
Example: If the forecasted demand is 100 units and the uncertainty ratio is 1, the adjusted forecast remains 100 units.
Practical Application
Decision Making: Use the adjusted forecast to make informed decisions about inventory, resource allocation, and planning. A higher uncertainty ratio might prompt you to prepare for a wider range of outcomes.
Risk Management: Understand the potential risks associated with high variability. A higher uncertainty ratio indicates a need for contingency plans to handle unexpected changes in demand.
Performance Evaluation: Evaluate the performance of your forecasting model by comparing the adjusted forecasts with actual outcomes. This can help in refining the model and improving accuracy.
Visualization
Visualizing the impact of the uncertainty ratio can help in better understanding its effect on the forecast. Here’s an example of how to visualize the original forecast and the adjusted forecast:

Python

import matplotlib.pyplot as plt

def visualize_forecast_with_uncertainty(df, forecast_df, uncertainty_ratios):
    plt.figure(figsize=(14, 7))
    
    for age in df.index.get_level_values(0).unique():
        ts = df.xs(age, level='age')['demand']
        plt.plot(ts.index, ts, label=f'Original Demand (Age {age})', marker='o')
    
    for age in forecast_df.index:
        plt.plot(forecast_df.columns, forecast_df.loc[age], label=f'Forecasted Demand (Age {age})', marker='x')
    
    for age in forecast_df.index:
        adjusted_forecast = forecast_df.loc[age] * uncertainty_ratios.loc[age, 'Uncertainty_Ratio']
        plt.plot(forecast_df.columns, adjusted_forecast, label=f'Adjusted Forecast (Age {age})', linestyle='--')
    
    plt.xlabel('Year')
    plt.ylabel('Demand')
    plt.title('Demand Forecast with Uncertainty Adjustment')
    plt.legend()
    plt.show()

# Assuming 'uncertainty_ratios' is the DataFrame containing the calculated uncertainty ratios
visualize_forecast_with_uncertainty(df, forecast, uncertainty_ratios)
AI-generated code. Review and use carefully. More info on FAQ.
This visualization will help you see how the uncertainty ratio modifies the forecasted values and provides a clearer picture of potential future demand.

Feel free to run this code and let me know if you need any further assistance!

You
Sent by you:
What other statistical measures can help assess variability?
Copilot
Sent by Copilot:
In addition to the standard deviation and mean, several other statistical measures can help assess variability in your data. Here are some key measures:

1. Variance
Definition: Variance is the average of the squared differences from the mean. It provides a measure of how much the data points differ from the mean.
Formula:
Variance
(
σ
2
)
=
∑
(
x
i
−
μ
)
2
N
Variance(σ2)=N∑(xi​−μ)2​

Interpretation: A higher variance indicates more spread out data points.
2. Range
Definition: The range is the difference between the maximum and minimum values in the dataset.
Formula:
Range
=
Max
−
Min
Range=Max−Min

Interpretation: A larger range indicates a wider spread of data points.
3. Interquartile Range (IQR)
Definition: The IQR is the range between the first quartile (Q1) and the third quartile (Q3). It measures the spread of the middle 50% of the data.
Formula:
IQR
=
Q
3
−
Q
1
IQR=Q3−Q1

Interpretation: A larger IQR indicates more variability in the middle 50% of the data.
4. Coefficient of Variation (CV)
Definition: The CV is the ratio of the standard deviation to the mean, expressed as a percentage. It provides a standardized measure of dispersion.
Formula:
CV
=
σ
μ
×
100
CV=μσ​×100

Interpretation: A higher CV indicates greater relative variability.
5. Mean Absolute Deviation (MAD)
Definition: MAD is the average of the absolute differences between each data point and the mean.
Formula:
MAD
=
∑
∣
x
i
−
μ
∣
N
MAD=N∑∣xi​−μ∣​

Interpretation: A higher MAD indicates more variability.
6. Quartile Deviation (QD)
Definition: QD is half of the IQR and provides a measure of the spread of the middle 50% of the data.
Formula:
QD
=
Q
3
−
Q
1
2
QD=2Q3−Q1​

Interpretation: A larger QD indicates more variability in the middle 50% of the data.
Example Code to Calculate These Measures
Here’s an example of how you can calculate these measures using Python:

Python

import pandas as pd

# Load the dataset
df = pd.read_csv('annual_unique_donors.csv')

# Assuming the dataframe has 'year', 'age', and 'demand' columns
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.set_index(['age', 'year'], inplace=True)

# Function to calculate variability measures
def calculate_variability_measures(df):
    variability_measures = pd.DataFrame()

    for age in df.index.get_level_values(0).unique():
        ts = df.xs(age, level='age')['demand'].dropna()
        
        # Calculate measures
        std_dev = ts.std()
        variance = ts.var()
        range_val = ts.max() - ts.min()
        iqr = ts.quantile(0.75) - ts.quantile(0.25)
        cv = (std_dev / ts.mean()) * 100 if ts.mean() != 0 else 0
        mad = ts.mad()
        qd = iqr / 2
        
        # Store measures in DataFrame
        variability_measures.loc[age, 'Standard Deviation'] = std_dev
        variability_measures.loc[age, 'Variance'] = variance
        variability_measures.loc[age, 'Range'] = range_val
        variability_measures.loc[age, 'IQR'] = iqr
        variability_measures.loc[age, 'CV'] = cv
        variability_measures.loc[age, 'MAD'] = mad
        variability_measures.loc[age, 'QD'] = qd
    
    return variability_measures

# Call the function and print the variability measures
variability_measures = calculate_variability_measures(df)
print(variability_measures)
AI-generated code. Review and use carefully. More info on FAQ.
Interpretation of Results
Standard Deviation and Variance: Indicate the overall spread of the data.
Range and IQR: Provide insights into the spread of the data, with IQR focusing on the middle 50%.
Coefficient of Variation: Standardizes the measure of dispersion, making it easier to compare across different datasets.
Mean Absolute Deviation and Quartile Deviation: Offer alternative measures of variability that are less sensitive to outliers.
These measures can help you better understand the variability in your data and make more informed decisions based on the forecasted demand.

Feel free to run this code and let me know if you need any further assistance!

You
Sent by you:
How can I choose the most appropriate variability measure for my specific analysis?
Copilot
Sent by Copilot:
Choosing the most appropriate variability measure for your specific analysis depends on several factors, including the nature of your data, the goals of your analysis, and the specific characteristics you want to capture. Here are some guidelines to help you make an informed decision:

1. Nature of the Data
Continuous Data: Measures like standard deviation, variance, and coefficient of variation are suitable for continuous data, as they provide insights into the spread and relative variability.
Categorical Data: For categorical data, measures like range and interquartile range (IQR) might be more appropriate, as they can highlight the spread of categories.
2. Goals of the Analysis
Understanding Overall Spread: If your goal is to understand the overall spread of the data, standard deviation and variance are good choices. They provide a comprehensive measure of variability.
Comparing Relative Variability: If you need to compare the variability of different datasets or groups, the coefficient of variation (CV) is useful. It standardizes the measure of dispersion, making it easier to compare across different scales.
Identifying Outliers: If you are interested in identifying outliers or understanding the spread of the middle 50% of the data, the interquartile range (IQR) and mean absolute deviation (MAD) are helpful.
3. Specific Characteristics
Sensitivity to Outliers: Standard deviation and variance are sensitive to outliers, which can skew the results. If your data contains outliers, you might prefer measures like IQR or MAD, which are less affected by extreme values.
Ease of Interpretation: Some measures are easier to interpret than others. For example, the range is straightforward and easy to understand, while the coefficient of variation provides a relative measure that can be more informative in certain contexts.
Practical Examples
Standard Deviation and Variance:
Use When: You want to understand the overall spread of continuous data.
Example: Analyzing the variability in monthly sales figures.
Coefficient of Variation (CV):
Use When: You need to compare the relative variability of different datasets.
Example: Comparing the variability of sales figures across different regions.
Interquartile Range (IQR):
Use When: You want to focus on the spread of the middle 50% of the data and identify potential outliers.
Example: Analyzing the distribution of test scores to identify students who are performing significantly above or below the median.
Mean Absolute Deviation (MAD):
Use When: You need a measure of variability that is less sensitive to outliers.
Example: Assessing the consistency of delivery times for a logistics company.
Example Code to Calculate and Compare Measures
Here’s an example of how you can calculate and compare different variability measures using Python:

Python

import pandas as pd

# Load the dataset
df = pd.read_csv('annual_unique_donors.csv')

# Assuming the dataframe has 'year', 'age', and 'demand' columns
df['year'] = pd.to_datetime(df['year'], format='%Y')
df.set_index(['age', 'year'], inplace=True)

# Function to calculate and compare variability measures
def compare_variability_measures(df):
    measures = pd.DataFrame()

    for age in df.index.get_level_values(0).unique():
        ts = df.xs(age, level='age')['demand'].dropna()
        
        # Calculate measures
        std_dev = ts.std()
        variance = ts.var()
        range_val = ts.max() - ts.min()
        iqr = ts.quantile(0.75) - ts.quantile(0.25)
        cv = (std_dev / ts.mean()) * 100 if ts.mean() != 0 else 0
        mad = ts.mad()
        
        # Store measures in DataFrame
        measures.loc[age, 'Standard Deviation'] = std_dev
        measures.loc[age, 'Variance'] = variance
        measures.loc[age, 'Range'] = range_val
        measures.loc[age, 'IQR'] = iqr
        measures.loc[age, 'CV'] = cv
        measures.loc[age, 'MAD'] = mad
    
    return measures

# Call the function and print the measures
variability_measures = compare_variability_measures(df)
print(variability_measures)
AI-generated code. Review and use carefully. More info on FAQ.
Summary
Standard Deviation and Variance: Best for understanding overall spread.
Coefficient of Variation (CV): Ideal for comparing relative variability.
Interquartile Range (IQR) and Mean Absolute Deviation (MAD): Useful for identifying outliers and understanding the spread of the middle 50% of the data.
By considering these factors, you can choose the most appropriate variability measure for your specific analysis, ensuring that your insights are accurate and meaningful. If you have any specific questions or need further assistance, feel free to ask!

You
Sent by you:
How can I visualize different variability measures for better insights?
Copilot
Sent by Copilot:
Visualizing different variability measures can provide a clearer understanding of the data’s spread and help identify patterns or outliers. Here are some common ways to visualize these measures using Python’s matplotlib and seaborn libraries:

1. Box Plot
A box plot (or whisker plot) is useful for visualizing the distribution of data and highlighting the interquartile range (IQR), median, and potential outliers.

Python

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('annual_unique_donors.csv')

# Assuming the dataframe has 'year', 'age', and 'demand' columns
df['year'] = pd.to_datetime(df['year'], format='%Y')

# Create a box plot
plt.figure(figsize=(12, 6))
sns.boxplot(x='age', y='demand', data=df)
plt.title('Box Plot of Demand by Age')
plt.xlabel('Age')
plt.ylabel('Demand')
plt.show()
AI-generated code. Review and use carefully. More info on FAQ.
2. Histogram
A histogram shows the frequency distribution of the data, which helps in understanding the spread and central tendency.

Python

plt.figure(figsize=(12, 6))
sns.histplot(df['demand'], bins=20, kde=True)
plt.title('Histogram of Demand')
plt.xlabel('Demand')
plt.ylabel('Frequency')
plt.show()
AI-generated code. Review and use carefully. More info on FAQ.
3. Violin Plot
A violin plot combines aspects of a box plot and a kernel density plot. It shows the distribution of the data across different categories.

Python

plt.figure(figsize=(12, 6))
sns.violinplot(x='age', y='demand', data=df)
plt.title('Violin Plot of Demand by Age')
plt.xlabel('Age')
plt.ylabel('Demand')
plt.show()
AI-generated code. Review and use carefully. More info on FAQ.
4. Scatter Plot with Error Bars
A scatter plot with error bars can show the mean demand and the variability (e.g., standard deviation) for each age cohort.

Python

# Calculate mean and standard deviation for each age cohort
mean_demand = df.groupby('age')['demand'].mean()
std_demand = df.groupby('age')['demand'].std()

# Create a scatter plot with error bars
plt.figure(figsize=(12, 6))
plt.errorbar(mean_demand.index, mean_demand, yerr=std_demand, fmt='o', capsize=5)
plt.title('Mean Demand with Standard Deviation by Age')
plt.xlabel('Age')
plt.ylabel('Demand')
plt.show()
AI-generated code. Review and use carefully. More info on FAQ.
5. Heatmap
A heatmap can visualize the variability measures across different age cohorts and years.

Python

# Pivot the data to create a matrix for the heatmap
pivot_table = df.pivot_table(index='age', columns='year', values='demand', aggfunc='mean')

# Create a heatmap
plt.figure(figsize=(12, 6))
sns.heatmap(pivot_table, cmap='coolwarm', annot=True, fmt=".1f")
plt.title('Heatmap of Mean Demand by Age and Year')
plt.xlabel('Year')
plt.ylabel('Age')
plt.show()
AI-generated code. Review and use carefully. More info on FAQ.
Summary
Box Plot: Visualizes the distribution, IQR, median, and outliers.
Histogram: Shows the frequency distribution of the data.
Violin Plot: Combines box plot and kernel density plot features.
Scatter Plot with Error Bars: Displays mean demand with variability.
Heatmap: Visualizes variability measures across different categories.
These visualizations can help you gain better insights into the variability of your data and make more informed decisions based on the analysis. Feel free to run these examples with your data and let me know if you need any further assistance!

In [ ]:


import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
import statsmodels.api as sm
# Suppress warnings from statsmodels
warnings.filterwarnings("ignore")


def apply_uncertainty_to_forecast(forecast, annual_receipts, annual_demands, age_specific_demands):
    # Calculate the annual donor receipt to demand ratio for each year
    annual_ratio = annual_receipts['Yearly Receipt'] / annual_demands
    
    # Calculate the age-specific demands ratio
    age_specific_ratio = age_specific_demands.groupby(['year', 'age']).sum().reset_index()
    age_specific_ratio['total_demands'] = age_specific_ratio.groupby('year')['demand'].transform('sum')
    age_specific_ratio['age_ratio'] = age_specific_ratio['demand'] / age_specific_ratio['total_demands']
    
    # Create an empty DataFrame to hold the adjusted forecast
    adjusted_forecast = pd.DataFrame(columns=['date', 'age', 'adjusted_forecast'])
    
    # Apply the ratios to the forecasted values
    for index, row in forecast.iterrows():
        year = row['date'].year
        age = row['age']
        forecast_value = row['forecast']

        # Calculate the adjusted forecast value
        if year in annual_ratio.index and year in age_specific_ratio['year'].unique():
            age_ratio = age_specific_ratio[(age_specific_ratio['year'] == year) & (age_specific_ratio['age'] == age)]
            if not age_ratio.empty:
                age_ratio_value = age_ratio['age_ratio'].values[0]
            else:
                age_ratio_value = 0
            adjusted_value = forecast_value * annual_ratio.loc[year] * age_ratio_value
        else:
            adjusted_value = 0
        
        # Append the adjusted value to the DataFrame
        adjusted_forecast = adjusted_forecast.append({'date': row['date'], 'age': age, 'adjusted_forecast': adjusted_value}, ignore_index=True)
    
    return adjusted_forecast

def forecast_demand_with_sarima(historical_data, steps, seasonal_order=(1,1,1,12)):
    # Fit a SARIMA model
    model = SARIMAX(historical_data, order=(1,1,1), seasonal_order=seasonal_order)
    model_fit = model.fit(disp=False)
    
    # Forecast future values
    forecast = model_fit.forecast(steps=steps)
    return forecast

# Lpa historical data for fitting SARIMA
historical_data = pd.Series(np.random.randint(100, 1000, 36), 
                            index=pd.date_range(start='2021-01-01', periods=36, freq='M'))

# Forecast for the next 24 months (2 years)
sarima_forecast = forecast_demand_with_sarima(historical_data, steps=24)

# Prepare the forecast DataFrame for the next 24 months for ages 18 to 106
forecast_dates = pd.date_range(start='2024-04-01', periods=24, freq='M')
forecast_ages = np.random.randint(18, 106, 24)
forecast_df = pd.DataFrame({'date': forecast_dates, 'age': forecast_ages, 'forecast': sarima_forecast})

# Annual receipts and demands DataFrames
annual_receipts = annual_donor_receipts  #pd.DataFrame({'year': [2022, 2023, 2024], 'Yearly Receipt': [7.519675e+05, 9.700350e+05, 1.124615e+06]})

annual_demands = annual_donor_demands  #pd.Series({2022: 601254, 2023: 752898, 2024: 505219}, name='unique_key')

# Age-specific demands DataFrame
age_specific_demands = annual_donor_demands_age_specific #pd.DataFrame({
#     'year': [2022, 2022, 2022, 2023, 2023, 2023, 2024, 2024, 2024],
#     'age': [18, 19, 20, 18, 19, 20, 18, 19, 20],
#     'demand': [199, 286, 192, 249, 345, 243, 122, 267, 196]})

# Apply uncertainty to the SARIMA forecast
adjusted_forecast = apply_uncertainty_to_forecast(forecast_df, annual_receipts, annual_demands, age_specific_demands)

print(adjusted_forecast)


In [ ]:


#def uncertainty_ratio(self, lpa_df_index, month_year):


# Estimate the Uncertainty:


#covid_impact_ratio = pre_covid_mean_demand / post_covid_mean_demand
impact_ratio = filtered_annual_demands / filtered_annual_receipts 

# Apply the Uncertainty to Future Long-Term Forecasts:
# Multiply the future long-term forecasts by the estimated uncertainty to incorporate the impact of the short-term period.

print(f"""The ratio of {impact_ratio} indicates that the short-term demand during COVID 
        was {impact_ratio * 100}% higher than the long-term average.""")

# # Apply the Uncertainty:
# Adjusted_Long-Term_Forecast = Future_Long_Term_Forecast * covid_impact_ratio

   
# Extract and save data into a csv file
lpa_data = monthly_unique_donors.rename(columns={'count': 'demand'})

print(lpa_data.head(2))